In [26]:
!pip install google-cloud-core
!pip install google-cloud-bigtable

import pandas as pd 
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family, row_filters
#crear frame
p1_gendata= pd.read_csv("Plant_1_Generation_Data.csv")

    
print(p1_gendata.shape)

(68778, 7)


In [28]:
p1_gendata.dropna(inplace = True)
p1_gendata_dict = p1_gendata.to_dict()
p1_gendata_dict

{'DATE_TIME': {0: '15-05-2020 00:00',
  1: '15-05-2020 00:00',
  2: '15-05-2020 00:00',
  3: '15-05-2020 00:00',
  4: '15-05-2020 00:00',
  5: '15-05-2020 00:00',
  6: '15-05-2020 00:00',
  7: '15-05-2020 00:00',
  8: '15-05-2020 00:00',
  9: '15-05-2020 00:00',
  10: '15-05-2020 00:00',
  11: '15-05-2020 00:00',
  12: '15-05-2020 00:00',
  13: '15-05-2020 00:00',
  14: '15-05-2020 00:00',
  15: '15-05-2020 00:00',
  16: '15-05-2020 00:00',
  17: '15-05-2020 00:00',
  18: '15-05-2020 00:00',
  19: '15-05-2020 00:00',
  20: '15-05-2020 00:00',
  21: '15-05-2020 00:15',
  22: '15-05-2020 00:15',
  23: '15-05-2020 00:15',
  24: '15-05-2020 00:15',
  25: '15-05-2020 00:15',
  26: '15-05-2020 00:15',
  27: '15-05-2020 00:15',
  28: '15-05-2020 00:15',
  29: '15-05-2020 00:15',
  30: '15-05-2020 00:15',
  31: '15-05-2020 00:15',
  32: '15-05-2020 00:15',
  33: '15-05-2020 00:15',
  34: '15-05-2020 00:15',
  35: '15-05-2020 00:15',
  36: '15-05-2020 00:15',
  37: '15-05-2020 00:15',
  38: '15

In [27]:
p2_gendata = pd.read_csv("Plant_2_Generation_Data.csv")
print(p2_gendata.shape)
print(p2_gendata)

(67698, 7)
                 DATE_TIME  PLANT_ID       SOURCE_KEY  DC_POWER  AC_POWER  \
0      2020-05-15 00:00:00   4136001  4UPUqMRk7TRMgml       0.0       0.0   
1      2020-05-15 00:00:00   4136001  81aHJ1q11NBPMrL       0.0       0.0   
2      2020-05-15 00:00:00   4136001  9kRcWv60rDACzjR       0.0       0.0   
3      2020-05-15 00:00:00   4136001  Et9kgGMDl729KT4       0.0       0.0   
4      2020-05-15 00:00:00   4136001  IQ2d7wF4YD8zU1Q       0.0       0.0   
...                    ...       ...              ...       ...       ...   
67693  2020-06-17 23:45:00   4136001  q49J1IKaHRwDQnt       0.0       0.0   
67694  2020-06-17 23:45:00   4136001  rrq4fwE8jgrTyWY       0.0       0.0   
67695  2020-06-17 23:45:00   4136001  vOuJvMaM2sgwLmb       0.0       0.0   
67696  2020-06-17 23:45:00   4136001  xMbIugepa2P7lBB       0.0       0.0   
67697  2020-06-17 23:45:00   4136001  xoJJ8DcxJEcupym       0.0       0.0   

       DAILY_YIELD   TOTAL_YIELD  
0      9425.000000  2.429011e

In [5]:
p1_meteriodata = pd.read_csv("Plant_1_Weather_Sensor_Data.csv")
print(p1_meteriodata)

                DATE_TIME  PLANT_ID       SOURCE_KEY  AMBIENT_TEMPERATURE  \
0     2020-05-15 00:00:00   4135001  HmiyD2TTLFNqkNe            25.184316   
1     2020-05-15 00:15:00   4135001  HmiyD2TTLFNqkNe            25.084589   
2     2020-05-15 00:30:00   4135001  HmiyD2TTLFNqkNe            24.935753   
3     2020-05-15 00:45:00   4135001  HmiyD2TTLFNqkNe            24.846130   
4     2020-05-15 01:00:00   4135001  HmiyD2TTLFNqkNe            24.621525   
...                   ...       ...              ...                  ...   
3177  2020-06-17 22:45:00   4135001  HmiyD2TTLFNqkNe            22.150570   
3178  2020-06-17 23:00:00   4135001  HmiyD2TTLFNqkNe            22.129816   
3179  2020-06-17 23:15:00   4135001  HmiyD2TTLFNqkNe            22.008275   
3180  2020-06-17 23:30:00   4135001  HmiyD2TTLFNqkNe            21.969495   
3181  2020-06-17 23:45:00   4135001  HmiyD2TTLFNqkNe            21.909288   

      MODULE_TEMPERATURE  IRRADIATION  
0              22.857507          0

In [7]:
p2_meteriodata = pd.read_csv("Plant_2_Weather_Sensor_Data.csv")
print(p2_meteriodata)

                DATE_TIME  PLANT_ID       SOURCE_KEY  AMBIENT_TEMPERATURE  \
0     2020-05-15 00:00:00   4136001  iq8k7ZNt4Mwm3w0            27.004764   
1     2020-05-15 00:15:00   4136001  iq8k7ZNt4Mwm3w0            26.880811   
2     2020-05-15 00:30:00   4136001  iq8k7ZNt4Mwm3w0            26.682055   
3     2020-05-15 00:45:00   4136001  iq8k7ZNt4Mwm3w0            26.500589   
4     2020-05-15 01:00:00   4136001  iq8k7ZNt4Mwm3w0            26.596148   
...                   ...       ...              ...                  ...   
3254  2020-06-17 22:45:00   4136001  iq8k7ZNt4Mwm3w0            23.511703   
3255  2020-06-17 23:00:00   4136001  iq8k7ZNt4Mwm3w0            23.482282   
3256  2020-06-17 23:15:00   4136001  iq8k7ZNt4Mwm3w0            23.354743   
3257  2020-06-17 23:30:00   4136001  iq8k7ZNt4Mwm3w0            23.291048   
3258  2020-06-17 23:45:00   4136001  iq8k7ZNt4Mwm3w0            23.202871   

      MODULE_TEMPERATURE  IRRADIATION  
0              25.060789          0

In [15]:
INSTANCE_ID = 'bigtable-1'
TABLE1_ID = 'plant_{}'.format(time.time())
TABLE_ID = 'meterio_{}'.format(time.time())

In [16]:
TABLE_ID

'meterio_1649621494.9262319'

In [17]:
TABLE1_ID

'plant_1649621494.9262319'

In [20]:
def print_row(row):
    print("Reading data for {}:".format(row.row_key.decode('utf-8')))
    for cf, cols in sorted(row.cells.items()):
        print("Column Family {}".format(cf))
        for col, cells in sorted(cols.items()):
            for cell in cells:
                labels = " [{}]".format(",".join(cell.labels)) \
                    if len(cell.labels) else ""
                print(
                    "\t{}: {} @{}{}".format(col.decode('utf-8'),
                                            cell.value.decode('utf-8'),
                                            cell.timestamp, labels))
    print("")

In [21]:
client = bigtable.Client.from_service_account_json('unbosque-service-account.json', admin = True)

In [22]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

Connecting to database instance


In [23]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

Creating the meterio_1649621494.9262319 table


In [25]:
print('Creating the {} table'.format(TABLE1_ID))
table = instance.table(TABLE1_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE1_ID))

Creating the plant_1649621494.9262319 table


In [30]:
print('Creating all family columns')

planta_plId = 'planta'
planta_pl = table.column_family(planta_plId)
planta_pl.create()




Creating all family columns


AlreadyExists: 409 Cannot create existing family planta

In [31]:
produ_prId = 'produccion'
produ_prId = table.column_family(produ_prId)
produ_prId.create()

In [32]:
dt = dt.utcnow()
rows = []

print('Writing orders to the table')
for order in orders:
    
    row_key = 'SOURCE_KEY#{}'.format(order['SOURCE_KEY']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(planta_plId, 'PLANT_ID'.encode(), order['planta']['PLANT_ID'], timestamp = dt)
    
    for item in order['produccion']:
        row.set_cell(produ_prId, '{}'.format(produccion['DC_POWER']).encode(), (produccion['AC_POWER'], (produccion['DAILY_YIELD'] + ':' + produccion['TOTAL_YIELD']+ ':' + produccion['DATE_TIME']), timestamp = dt)
       
    rows.append(row)

table.mutate_rows(rows)

SyntaxError: invalid syntax (Temp/ipykernel_21132/1073409285.py, line 13)

In [ ]:
planta = [
    {'DATE_TIME': {0: '15-05-2020 00:00',
  1: '15-05-2020 00:00',
  2: '15-05-2020 00:00',
  3: '15-05-2020 00:00',
  4: '15-05-2020 00:00',
  5: '15-05-2020 00:00',
  6: '15-05-2020 00:00',
  7: '15-05-2020 00:00',
  8: '15-05-2020 00:00',
  9: '15-05-2020 00:00',
  10: '15-05-2020 00:00',
  11: '15-05-2020 00:00',
  12: '15-05-2020 00:00',
  13: '15-05-2020 00:00',
  14: '15-05-2020 00:00',
  15: '15-05-2020 00:00',
  16: '15-05-2020 00:00',
  17: '15-05-2020 00:00',
  18: '15-05-2020 00:00',
  19: '15-05-2020 00:00',
  20: '15-05-2020 00:00',
  21: '15-05-2020 00:15',
  22: '15-05-2020 00:15',
  23: '15-05-2020 00:15',
  24: '15-05-2020 00:15',
  25: '15-05-2020 00:15',
  26: '15-05-2020 00:15',
  27: '15-05-2020 00:15',
  28: '15-05-2020 00:15',
  29: '15-05-2020 00:15',
  30: '15-05-2020 00:15',
  31: '15-05-2020 00:15',
  32: '15-05-2020 00:15',
  33: '15-05-2020 00:15',
  34: '15-05-2020 00:15',
  35: '15-05-2020 00:15',
  36: '15-05-2020 00:15',
  37: '15-05-2020 00:15',
  38: '15-05-2020 00:15',
  39: '15-05-2020 00:15',
  40: '15-05-2020 00:15',
  41: '15-05-2020 00:15',
  42: '15-05-2020 00:30',
  43: '15-05-2020 00:30',
  44: '15-05-2020 00:30',
  45: '15-05-2020 00:30',
  46: '15-05-2020 00:30',
  47: '15-05-2020 00:30',
  48: '15-05-2020 00:30',
  49: '15-05-2020 00:30',
  50: '15-05-2020 00:30',
  51: '15-05-2020 00:30',
  52: '15-05-2020 00:30',
  53: '15-05-2020 00:30',
  54: '15-05-2020 00:30',
  55: '15-05-2020 00:30',
  56: '15-05-2020 00:30',
  57: '15-05-2020 00:30',
  58: '15-05-2020 00:30',
  59: '15-05-2020 00:30',
  60: '15-05-2020 00:30',
  61: '15-05-2020 00:30',
  62: '15-05-2020 00:30',
  63: '15-05-2020 00:45',
  64: '15-05-2020 00:45',
  65: '15-05-2020 00:45',
  66: '15-05-2020 00:45',
  67: '15-05-2020 00:45',
  68: '15-05-2020 00:45',
  69: '15-05-2020 00:45',
  70: '15-05-2020 00:45',
  71: '15-05-2020 00:45',
  72: '15-05-2020 00:45',
  73: '15-05-2020 00:45',
  74: '15-05-2020 00:45',
  75: '15-05-2020 00:45',
  76: '15-05-2020 00:45',
  77: '15-05-2020 00:45',
  78: '15-05-2020 00:45',
  79: '15-05-2020 00:45',
  80: '15-05-2020 00:45',
  81: '15-05-2020 00:45',
  82: '15-05-2020 00:45',
  83: '15-05-2020 00:45',
  84: '15-05-2020 01:00',
  85: '15-05-2020 01:00',
  86: '15-05-2020 01:00',
  87: '15-05-2020 01:00',
  88: '15-05-2020 01:00',
  89: '15-05-2020 01:00',
  90: '15-05-2020 01:00',
  91: '15-05-2020 01:00',
  92: '15-05-2020 01:00',
  93: '15-05-2020 01:00',
  94: '15-05-2020 01:00',
  95: '15-05-2020 01:00',
  96: '15-05-2020 01:00',
  97: '15-05-2020 01:00',
  98: '15-05-2020 01:00',
  99: '15-05-2020 01:00',
  100: '15-05-2020 01:00',
  101: '15-05-2020 01:00',
  102: '15-05-2020 01:00',
  103: '15-05-2020 01:00',
  104: '15-05-2020 01:00',
  105: '15-05-2020 01:00',
  106: '15-05-2020 01:15',
  107: '15-05-2020 01:15',
  108: '15-05-2020 01:15',
  109: '15-05-2020 01:15',
  110: '15-05-2020 01:15',
  111: '15-05-2020 01:15',
  112: '15-05-2020 01:15',
  113: '15-05-2020 01:15',
  114: '15-05-2020 01:15',
  115: '15-05-2020 01:15',
  116: '15-05-2020 01:15',
  117: '15-05-2020 01:15',
  118: '15-05-2020 01:15',
  119: '15-05-2020 01:15',
  120: '15-05-2020 01:15',
  121: '15-05-2020 01:15',
  122: '15-05-2020 01:15',
  123: '15-05-2020 01:15',
  124: '15-05-2020 01:15',
  125: '15-05-2020 01:15',
  126: '15-05-2020 01:15',
  127: '15-05-2020 01:30',
  128: '15-05-2020 01:30',
  129: '15-05-2020 01:30',
  130: '15-05-2020 01:30',
  131: '15-05-2020 01:30',
  132: '15-05-2020 01:30',
  133: '15-05-2020 01:30',
  134: '15-05-2020 01:30',
  135: '15-05-2020 01:30',
  136: '15-05-2020 01:30',
  137: '15-05-2020 01:30',
  138: '15-05-2020 01:30',
  139: '15-05-2020 01:30',
  140: '15-05-2020 01:30',
  141: '15-05-2020 01:30',
  142: '15-05-2020 01:30',
  143: '15-05-2020 01:30',
  144: '15-05-2020 01:30',
  145: '15-05-2020 01:30',
  146: '15-05-2020 01:30',
  147: '15-05-2020 01:30',
  148: '15-05-2020 01:45',
  149: '15-05-2020 01:45',
  150: '15-05-2020 01:45',
  151: '15-05-2020 01:45',
  152: '15-05-2020 01:45',
  153: '15-05-2020 01:45',
  154: '15-05-2020 01:45',
  155: '15-05-2020 01:45',
  156: '15-05-2020 01:45',
  157: '15-05-2020 01:45',
  158: '15-05-2020 01:45',
  159: '15-05-2020 01:45',
  160: '15-05-2020 01:45',
  161: '15-05-2020 01:45',
  162: '15-05-2020 01:45',
  163: '15-05-2020 01:45',
  164: '15-05-2020 01:45',
  165: '15-05-2020 01:45',
  166: '15-05-2020 01:45',
  167: '15-05-2020 01:45',
  168: '15-05-2020 01:45',
  169: '15-05-2020 01:45',
  170: '15-05-2020 02:00',
  171: '15-05-2020 02:00',
  172: '15-05-2020 02:00',
  173: '15-05-2020 02:00',
  174: '15-05-2020 02:00',
  175: '15-05-2020 02:00',
  176: '15-05-2020 02:00',
  177: '15-05-2020 02:00',
  178: '15-05-2020 02:00',
  179: '15-05-2020 02:00',
  180: '15-05-2020 02:00',
  181: '15-05-2020 02:00',
  182: '15-05-2020 02:00',
  183: '15-05-2020 02:00',
  184: '15-05-2020 02:00',
  185: '15-05-2020 02:00',
  186: '15-05-2020 02:00',
  187: '15-05-2020 02:00',
  188: '15-05-2020 02:00',
  189: '15-05-2020 02:00',
  190: '15-05-2020 02:00',
  191: '15-05-2020 02:15',
  192: '15-05-2020 02:15',
  193: '15-05-2020 02:15',
  194: '15-05-2020 02:15',
  195: '15-05-2020 02:15',
  196: '15-05-2020 02:15',
  197: '15-05-2020 02:15',
  198: '15-05-2020 02:15',
  199: '15-05-2020 02:15',
  200: '15-05-2020 02:15',
  201: '15-05-2020 02:15',
  202: '15-05-2020 02:15',
  203: '15-05-2020 02:15',
  204: '15-05-2020 02:15',
  205: '15-05-2020 02:15',
  206: '15-05-2020 02:15',
  207: '15-05-2020 02:15',
  208: '15-05-2020 02:15',
  209: '15-05-2020 02:15',
  210: '15-05-2020 02:15',
  211: '15-05-2020 02:15',
  212: '15-05-2020 02:30',
  213: '15-05-2020 02:30',
  214: '15-05-2020 02:30',
  215: '15-05-2020 02:30',
  216: '15-05-2020 02:30',
  217: '15-05-2020 02:30',
  218: '15-05-2020 02:30',
  219: '15-05-2020 02:30',
  220: '15-05-2020 02:30',
  221: '15-05-2020 02:30',
  222: '15-05-2020 02:30',
  223: '15-05-2020 02:30',
  224: '15-05-2020 02:30',
  225: '15-05-2020 02:30',
  226: '15-05-2020 02:30',
  227: '15-05-2020 02:30',
  228: '15-05-2020 02:30',
  229: '15-05-2020 02:30',
  230: '15-05-2020 02:30',
  231: '15-05-2020 02:30',
  232: '15-05-2020 02:30',
  233: '15-05-2020 02:45',
  234: '15-05-2020 02:45',
  235: '15-05-2020 02:45',
  236: '15-05-2020 02:45',
  237: '15-05-2020 02:45',
  238: '15-05-2020 02:45',
  239: '15-05-2020 02:45',
  240: '15-05-2020 02:45',
  241: '15-05-2020 02:45',
  242: '15-05-2020 02:45',
  243: '15-05-2020 02:45',
  244: '15-05-2020 02:45',
  245: '15-05-2020 02:45',
  246: '15-05-2020 02:45',
  247: '15-05-2020 02:45',
  248: '15-05-2020 02:45',
  249: '15-05-2020 02:45',
  250: '15-05-2020 02:45',
  251: '15-05-2020 02:45',
  252: '15-05-2020 02:45',
  253: '15-05-2020 02:45',
  254: '15-05-2020 03:00',
  255: '15-05-2020 03:00',
  256: '15-05-2020 03:00',
  257: '15-05-2020 03:00',
  258: '15-05-2020 03:00',
  259: '15-05-2020 03:00',
  260: '15-05-2020 03:00',
  261: '15-05-2020 03:00',
  262: '15-05-2020 03:00',
  263: '15-05-2020 03:00',
  264: '15-05-2020 03:00',
  265: '15-05-2020 03:00',
  266: '15-05-2020 03:00',
  267: '15-05-2020 03:00',
  268: '15-05-2020 03:00',
  269: '15-05-2020 03:00',
  270: '15-05-2020 03:00',
  271: '15-05-2020 03:00',
  272: '15-05-2020 03:00',
  273: '15-05-2020 03:00',
  274: '15-05-2020 03:00',
  275: '15-05-2020 03:15',
  276: '15-05-2020 03:15',
  277: '15-05-2020 03:15',
  278: '15-05-2020 03:15',
  279: '15-05-2020 03:15',
  280: '15-05-2020 03:15',
  281: '15-05-2020 03:15',
  282: '15-05-2020 03:15',
  283: '15-05-2020 03:15',
  284: '15-05-2020 03:15',
  285: '15-05-2020 03:15',
  286: '15-05-2020 03:15',
  287: '15-05-2020 03:15',
  288: '15-05-2020 03:15',
  289: '15-05-2020 03:15',
  290: '15-05-2020 03:15',
  291: '15-05-2020 03:15',
  292: '15-05-2020 03:15',
  293: '15-05-2020 03:15',
  294: '15-05-2020 03:15',
  295: '15-05-2020 03:15',
  296: '15-05-2020 03:30',
  297: '15-05-2020 03:30',
  298: '15-05-2020 03:30',
  299: '15-05-2020 03:30',
  300: '15-05-2020 03:30',
  301: '15-05-2020 03:30',
  302: '15-05-2020 03:30',
  303: '15-05-2020 03:30',
  304: '15-05-2020 03:30',
  305: '15-05-2020 03:30',
  306: '15-05-2020 03:30',
  307: '15-05-2020 03:30',
  308: '15-05-2020 03:30',
  309: '15-05-2020 03:30',
  310: '15-05-2020 03:30',
  311: '15-05-2020 03:30',
  312: '15-05-2020 03:30',
  313: '15-05-2020 03:30',
  314: '15-05-2020 03:30',
  315: '15-05-2020 03:30',
  316: '15-05-2020 03:30',
  317: '15-05-2020 03:45',
  318: '15-05-2020 03:45',
  319: '15-05-2020 03:45',
  320: '15-05-2020 03:45',
  321: '15-05-2020 03:45',
  322: '15-05-2020 03:45',
  323: '15-05-2020 03:45',
  324: '15-05-2020 03:45',
  325: '15-05-2020 03:45',
  326: '15-05-2020 03:45',
  327: '15-05-2020 03:45',
  328: '15-05-2020 03:45',
  329: '15-05-2020 03:45',
  330: '15-05-2020 03:45',
  331: '15-05-2020 03:45',
  332: '15-05-2020 03:45',
  333: '15-05-2020 03:45',
  334: '15-05-2020 03:45',
  335: '15-05-2020 03:45',
  336: '15-05-2020 03:45',
  337: '15-05-2020 03:45',
  338: '15-05-2020 04:00',
  339: '15-05-2020 04:00',
  340: '15-05-2020 04:00',
  341: '15-05-2020 04:00',
  342: '15-05-2020 04:00',
  343: '15-05-2020 04:00',
  344: '15-05-2020 04:00',
  345: '15-05-2020 04:00',
  346: '15-05-2020 04:00',
  347: '15-05-2020 04:00',
  348: '15-05-2020 04:00',
  349: '15-05-2020 04:00',
  350: '15-05-2020 04:00',
  351: '15-05-2020 04:00',
  352: '15-05-2020 04:00',
  353: '15-05-2020 04:00',
  354: '15-05-2020 04:00',
  355: '15-05-2020 04:00',
  356: '15-05-2020 04:00',
  357: '15-05-2020 04:00',
  358: '15-05-2020 04:00',
  359: '15-05-2020 04:15',
  360: '15-05-2020 04:15',
  361: '15-05-2020 04:15',
  362: '15-05-2020 04:15',
  363: '15-05-2020 04:15',
  364: '15-05-2020 04:15',
  365: '15-05-2020 04:15',
  366: '15-05-2020 04:15',
  367: '15-05-2020 04:15',
  368: '15-05-2020 04:15',
  369: '15-05-2020 04:15',
  370: '15-05-2020 04:15',
  371: '15-05-2020 04:15',
  372: '15-05-2020 04:15',
  373: '15-05-2020 04:15',
  374: '15-05-2020 04:15',
  375: '15-05-2020 04:15',
  376: '15-05-2020 04:15',
  377: '15-05-2020 04:15',
  378: '15-05-2020 04:15',
  379: '15-05-2020 04:15',
  380: '15-05-2020 04:15',
  381: '15-05-2020 04:30',
  382: '15-05-2020 04:30',
  383: '15-05-2020 04:30',
  384: '15-05-2020 04:30',
  385: '15-05-2020 04:30',
  386: '15-05-2020 04:30',
  387: '15-05-2020 04:30',
  388: '15-05-2020 04:30',
  389: '15-05-2020 04:30',
  390: '15-05-2020 04:30',
  391: '15-05-2020 04:30',
  392: '15-05-2020 04:30',
  393: '15-05-2020 04:30',
  394: '15-05-2020 04:30',
  395: '15-05-2020 04:30',
  396: '15-05-2020 04:30',
  397: '15-05-2020 04:30',
  398: '15-05-2020 04:30',
  399: '15-05-2020 04:30',
  400: '15-05-2020 04:30',
  401: '15-05-2020 04:30',
  402: '15-05-2020 04:30',
  403: '15-05-2020 04:45',
  404: '15-05-2020 04:45',
  405: '15-05-2020 04:45',
  406: '15-05-2020 04:45',
  407: '15-05-2020 04:45',
  408: '15-05-2020 04:45',
  409: '15-05-2020 04:45',
  410: '15-05-2020 04:45',
  411: '15-05-2020 04:45',
  412: '15-05-2020 04:45',
  413: '15-05-2020 04:45',
  414: '15-05-2020 04:45',
  415: '15-05-2020 04:45',
  416: '15-05-2020 04:45',
  417: '15-05-2020 04:45',
  418: '15-05-2020 04:45',
  419: '15-05-2020 04:45',
  420: '15-05-2020 04:45',
  421: '15-05-2020 04:45',
  422: '15-05-2020 04:45',
  423: '15-05-2020 04:45',
  424: '15-05-2020 04:45',
  425: '15-05-2020 05:00',
  426: '15-05-2020 05:00',
  427: '15-05-2020 05:00',
  428: '15-05-2020 05:00',
  429: '15-05-2020 05:00',
  430: '15-05-2020 05:00',
  431: '15-05-2020 05:00',
  432: '15-05-2020 05:00',
  433: '15-05-2020 05:00',
  434: '15-05-2020 05:00',
  435: '15-05-2020 05:00',
  436: '15-05-2020 05:00',
  437: '15-05-2020 05:00',
  438: '15-05-2020 05:00',
  439: '15-05-2020 05:00',
  440: '15-05-2020 05:00',
  441: '15-05-2020 05:00',
  442: '15-05-2020 05:00',
  443: '15-05-2020 05:00',
  444: '15-05-2020 05:00',
  445: '15-05-2020 05:00',
  446: '15-05-2020 05:00',
  447: '15-05-2020 05:15',
  448: '15-05-2020 05:15',
  449: '15-05-2020 05:15',
  450: '15-05-2020 05:15',
  451: '15-05-2020 05:15',
  452: '15-05-2020 05:15',
  453: '15-05-2020 05:15',
  454: '15-05-2020 05:15',
  455: '15-05-2020 05:15',
  456: '15-05-2020 05:15',
  457: '15-05-2020 05:15',
  458: '15-05-2020 05:15',
  459: '15-05-2020 05:15',
  460: '15-05-2020 05:15',
  461: '15-05-2020 05:15',
  462: '15-05-2020 05:15',
  463: '15-05-2020 05:15',
  464: '15-05-2020 05:15',
  465: '15-05-2020 05:15',
  466: '15-05-2020 05:15',
  467: '15-05-2020 05:15',
  468: '15-05-2020 05:30',
  469: '15-05-2020 05:30',
  470: '15-05-2020 05:30',
  471: '15-05-2020 05:30',
  472: '15-05-2020 05:30',
  473: '15-05-2020 05:30',
  474: '15-05-2020 05:30',
  475: '15-05-2020 05:30',
  476: '15-05-2020 05:30',
  477: '15-05-2020 05:30',
  478: '15-05-2020 05:30',
  479: '15-05-2020 05:30',
  480: '15-05-2020 05:30',
  481: '15-05-2020 05:30',
  482: '15-05-2020 05:30',
  483: '15-05-2020 05:30',
  484: '15-05-2020 05:30',
  485: '15-05-2020 05:30',
  486: '15-05-2020 05:30',
  487: '15-05-2020 05:30',
  488: '15-05-2020 05:30',
  489: '15-05-2020 05:45',
  490: '15-05-2020 05:45',
  491: '15-05-2020 05:45',
  492: '15-05-2020 05:45',
  493: '15-05-2020 05:45',
  494: '15-05-2020 05:45',
  495: '15-05-2020 05:45',
  496: '15-05-2020 05:45',
  497: '15-05-2020 05:45',
  498: '15-05-2020 05:45',
  499: '15-05-2020 05:45',
  500: '15-05-2020 05:45',
  501: '15-05-2020 05:45',
  502: '15-05-2020 05:45',
  503: '15-05-2020 05:45',
  504: '15-05-2020 05:45',
  505: '15-05-2020 05:45',
  506: '15-05-2020 05:45',
  507: '15-05-2020 05:45',
  508: '15-05-2020 05:45',
  509: '15-05-2020 05:45',
  510: '15-05-2020 06:00',
  511: '15-05-2020 06:00',
  512: '15-05-2020 06:00',
  513: '15-05-2020 06:00',
  514: '15-05-2020 06:00',
  515: '15-05-2020 06:00',
  516: '15-05-2020 06:00',
  517: '15-05-2020 06:00',
  518: '15-05-2020 06:00',
  519: '15-05-2020 06:00',
  520: '15-05-2020 06:00',
  521: '15-05-2020 06:00',
  522: '15-05-2020 06:00',
  523: '15-05-2020 06:00',
  524: '15-05-2020 06:00',
  525: '15-05-2020 06:00',
  526: '15-05-2020 06:00',
  527: '15-05-2020 06:00',
  528: '15-05-2020 06:00',
  529: '15-05-2020 06:00',
  530: '15-05-2020 06:00',
  531: '15-05-2020 06:15',
  532: '15-05-2020 06:15',
  533: '15-05-2020 06:15',
  534: '15-05-2020 06:15',
  535: '15-05-2020 06:15',
  536: '15-05-2020 06:15',
  537: '15-05-2020 06:15',
  538: '15-05-2020 06:15',
  539: '15-05-2020 06:15',
  540: '15-05-2020 06:15',
  541: '15-05-2020 06:15',
  542: '15-05-2020 06:15',
  543: '15-05-2020 06:15',
  544: '15-05-2020 06:15',
  545: '15-05-2020 06:15',
  546: '15-05-2020 06:15',
  547: '15-05-2020 06:15',
  548: '15-05-2020 06:15',
  549: '15-05-2020 06:15',
  550: '15-05-2020 06:15',
  551: '15-05-2020 06:15',
  552: '15-05-2020 06:30',
  553: '15-05-2020 06:30',
  554: '15-05-2020 06:30',
  555: '15-05-2020 06:30',
  556: '15-05-2020 06:30',
  557: '15-05-2020 06:30',
  558: '15-05-2020 06:30',
  559: '15-05-2020 06:30',
  560: '15-05-2020 06:30',
  561: '15-05-2020 06:30',
  562: '15-05-2020 06:30',
  563: '15-05-2020 06:30',
  564: '15-05-2020 06:30',
  565: '15-05-2020 06:30',
  566: '15-05-2020 06:30',
  567: '15-05-2020 06:30',
  568: '15-05-2020 06:30',
  569: '15-05-2020 06:30',
  570: '15-05-2020 06:30',
  571: '15-05-2020 06:30',
  572: '15-05-2020 06:30',
  573: '15-05-2020 06:30',
  574: '15-05-2020 06:45',
  575: '15-05-2020 06:45',
  576: '15-05-2020 06:45',
  577: '15-05-2020 06:45',
  578: '15-05-2020 06:45',
  579: '15-05-2020 06:45',
  580: '15-05-2020 06:45',
  581: '15-05-2020 06:45',
  582: '15-05-2020 06:45',
  583: '15-05-2020 06:45',
  584: '15-05-2020 06:45',
  585: '15-05-2020 06:45',
  586: '15-05-2020 06:45',
  587: '15-05-2020 06:45',
  588: '15-05-2020 06:45',
  589: '15-05-2020 06:45',
  590: '15-05-2020 06:45',
  591: '15-05-2020 06:45',
  592: '15-05-2020 06:45',
  593: '15-05-2020 06:45',
  594: '15-05-2020 06:45',
  595: '15-05-2020 06:45',
  596: '15-05-2020 07:00',
  597: '15-05-2020 07:00',
  598: '15-05-2020 07:00',
  599: '15-05-2020 07:00',
  600: '15-05-2020 07:00',
  601: '15-05-2020 07:00',
  602: '15-05-2020 07:00',
  603: '15-05-2020 07:00',
  604: '15-05-2020 07:00',
  605: '15-05-2020 07:00',
  606: '15-05-2020 07:00',
  607: '15-05-2020 07:00',
  608: '15-05-2020 07:00',
  609: '15-05-2020 07:00',
  610: '15-05-2020 07:00',
  611: '15-05-2020 07:00',
  612: '15-05-2020 07:00',
  613: '15-05-2020 07:00',
  614: '15-05-2020 07:00',
  615: '15-05-2020 07:00',
  616: '15-05-2020 07:00',
  617: '15-05-2020 07:00',
  618: '15-05-2020 07:15',
  619: '15-05-2020 07:15',
  620: '15-05-2020 07:15',
  621: '15-05-2020 07:15',
  622: '15-05-2020 07:15',
  623: '15-05-2020 07:15',
  624: '15-05-2020 07:15',
  625: '15-05-2020 07:15',
  626: '15-05-2020 07:15',
  627: '15-05-2020 07:15',
  628: '15-05-2020 07:15',
  629: '15-05-2020 07:15',
  630: '15-05-2020 07:15',
  631: '15-05-2020 07:15',
  632: '15-05-2020 07:15',
  633: '15-05-2020 07:15',
  634: '15-05-2020 07:15',
  635: '15-05-2020 07:15',
  636: '15-05-2020 07:15',
  637: '15-05-2020 07:15',
  638: '15-05-2020 07:15',
  639: '15-05-2020 07:15',
  640: '15-05-2020 07:30',
  641: '15-05-2020 07:30',
  642: '15-05-2020 07:30',
  643: '15-05-2020 07:30',
  644: '15-05-2020 07:30',
  645: '15-05-2020 07:30',
  646: '15-05-2020 07:30',
  647: '15-05-2020 07:30',
  648: '15-05-2020 07:30',
  649: '15-05-2020 07:30',
  650: '15-05-2020 07:30',
  651: '15-05-2020 07:30',
  652: '15-05-2020 07:30',
  653: '15-05-2020 07:30',
  654: '15-05-2020 07:30',
  655: '15-05-2020 07:30',
  656: '15-05-2020 07:30',
  657: '15-05-2020 07:30',
  658: '15-05-2020 07:30',
  659: '15-05-2020 07:30',
  660: '15-05-2020 07:30',
  661: '15-05-2020 07:30',
  662: '15-05-2020 07:45',
  663: '15-05-2020 07:45',
  664: '15-05-2020 07:45',
  665: '15-05-2020 07:45',
  666: '15-05-2020 07:45',
  667: '15-05-2020 07:45',
  668: '15-05-2020 07:45',
  669: '15-05-2020 07:45',
  670: '15-05-2020 07:45',
  671: '15-05-2020 07:45',
  672: '15-05-2020 07:45',
  673: '15-05-2020 07:45',
  674: '15-05-2020 07:45',
  675: '15-05-2020 07:45',
  676: '15-05-2020 07:45',
  677: '15-05-2020 07:45',
  678: '15-05-2020 07:45',
  679: '15-05-2020 07:45',
  680: '15-05-2020 07:45',
  681: '15-05-2020 07:45',
  682: '15-05-2020 07:45',
  683: '15-05-2020 07:45',
  684: '15-05-2020 08:00',
  685: '15-05-2020 08:00',
  686: '15-05-2020 08:00',
  687: '15-05-2020 08:00',
  688: '15-05-2020 08:00',
  689: '15-05-2020 08:00',
  690: '15-05-2020 08:00',
  691: '15-05-2020 08:00',
  692: '15-05-2020 08:00',
  693: '15-05-2020 08:00',
  694: '15-05-2020 08:00',
  695: '15-05-2020 08:00',
  696: '15-05-2020 08:00',
  697: '15-05-2020 08:00',
  698: '15-05-2020 08:00',
  699: '15-05-2020 08:00',
  700: '15-05-2020 08:00',
  701: '15-05-2020 08:00',
  702: '15-05-2020 08:00',
  703: '15-05-2020 08:00',
  704: '15-05-2020 08:00',
  705: '15-05-2020 08:00',
  706: '15-05-2020 08:15',
  707: '15-05-2020 08:15',
  708: '15-05-2020 08:15',
  709: '15-05-2020 08:15',
  710: '15-05-2020 08:15',
  711: '15-05-2020 08:15',
  712: '15-05-2020 08:15',
  713: '15-05-2020 08:15',
  714: '15-05-2020 08:15',
  715: '15-05-2020 08:15',
  716: '15-05-2020 08:15',
  717: '15-05-2020 08:15',
  718: '15-05-2020 08:15',
  719: '15-05-2020 08:15',
  720: '15-05-2020 08:15',
  721: '15-05-2020 08:15',
  722: '15-05-2020 08:15',
  723: '15-05-2020 08:15',
  724: '15-05-2020 08:15',
  725: '15-05-2020 08:15',
  726: '15-05-2020 08:15',
  727: '15-05-2020 08:15',
  728: '15-05-2020 08:30',
  729: '15-05-2020 08:30',
  730: '15-05-2020 08:30',
  731: '15-05-2020 08:30',
  732: '15-05-2020 08:30',
  733: '15-05-2020 08:30',
  734: '15-05-2020 08:30',
  735: '15-05-2020 08:30',
  736: '15-05-2020 08:30',
  737: '15-05-2020 08:30',
  738: '15-05-2020 08:30',
  739: '15-05-2020 08:30',
  740: '15-05-2020 08:30',
  741: '15-05-2020 08:30',
  742: '15-05-2020 08:30',
  743: '15-05-2020 08:30',
  744: '15-05-2020 08:30',
  745: '15-05-2020 08:30',
  746: '15-05-2020 08:30',
  747: '15-05-2020 08:30',
  748: '15-05-2020 08:30',
  749: '15-05-2020 08:30',
  750: '15-05-2020 08:45',
  751: '15-05-2020 08:45',
  752: '15-05-2020 08:45',
  753: '15-05-2020 08:45',
  754: '15-05-2020 08:45',
  755: '15-05-2020 08:45',
  756: '15-05-2020 08:45',
  757: '15-05-2020 08:45',
  758: '15-05-2020 08:45',
  759: '15-05-2020 08:45',
  760: '15-05-2020 08:45',
  761: '15-05-2020 08:45',
  762: '15-05-2020 08:45',
  763: '15-05-2020 08:45',
  764: '15-05-2020 08:45',
  765: '15-05-2020 08:45',
  766: '15-05-2020 08:45',
  767: '15-05-2020 08:45',
  768: '15-05-2020 08:45',
  769: '15-05-2020 08:45',
  770: '15-05-2020 08:45',
  771: '15-05-2020 08:45',
  772: '15-05-2020 09:00',
  773: '15-05-2020 09:00',
  774: '15-05-2020 09:00',
  775: '15-05-2020 09:00',
  776: '15-05-2020 09:00',
  777: '15-05-2020 09:00',
  778: '15-05-2020 09:00',
  779: '15-05-2020 09:00',
  780: '15-05-2020 09:00',
  781: '15-05-2020 09:00',
  782: '15-05-2020 09:00',
  783: '15-05-2020 09:00',
  784: '15-05-2020 09:00',
  785: '15-05-2020 09:00',
  786: '15-05-2020 09:00',
  787: '15-05-2020 09:00',
  788: '15-05-2020 09:00',
  789: '15-05-2020 09:00',
  790: '15-05-2020 09:00',
  791: '15-05-2020 09:00',
  792: '15-05-2020 09:00',
  793: '15-05-2020 09:00',
  794: '15-05-2020 09:15',
  795: '15-05-2020 09:15',
  796: '15-05-2020 09:15',
  797: '15-05-2020 09:15',
  798: '15-05-2020 09:15',
  799: '15-05-2020 09:15',
  800: '15-05-2020 09:15',
  801: '15-05-2020 09:15',
  802: '15-05-2020 09:15',
  803: '15-05-2020 09:15',
  804: '15-05-2020 09:15',
  805: '15-05-2020 09:15',
  806: '15-05-2020 09:15',
  807: '15-05-2020 09:15',
  808: '15-05-2020 09:15',
  809: '15-05-2020 09:15',
  810: '15-05-2020 09:15',
  811: '15-05-2020 09:15',
  812: '15-05-2020 09:15',
  813: '15-05-2020 09:15',
  814: '15-05-2020 09:15',
  815: '15-05-2020 09:15',
  816: '15-05-2020 09:30',
  817: '15-05-2020 09:30',
  818: '15-05-2020 09:30',
  819: '15-05-2020 09:30',
  820: '15-05-2020 09:30',
  821: '15-05-2020 09:30',
  822: '15-05-2020 09:30',
  823: '15-05-2020 09:30',
  824: '15-05-2020 09:30',
  825: '15-05-2020 09:30',
  826: '15-05-2020 09:30',
  827: '15-05-2020 09:30',
  828: '15-05-2020 09:30',
  829: '15-05-2020 09:30',
  830: '15-05-2020 09:30',
  831: '15-05-2020 09:30',
  832: '15-05-2020 09:30',
  833: '15-05-2020 09:30',
  834: '15-05-2020 09:30',
  835: '15-05-2020 09:30',
  836: '15-05-2020 09:30',
  837: '15-05-2020 09:30',
  838: '15-05-2020 09:45',
  839: '15-05-2020 09:45',
  840: '15-05-2020 09:45',
  841: '15-05-2020 09:45',
  842: '15-05-2020 09:45',
  843: '15-05-2020 09:45',
  844: '15-05-2020 09:45',
  845: '15-05-2020 09:45',
  846: '15-05-2020 09:45',
  847: '15-05-2020 09:45',
  848: '15-05-2020 09:45',
  849: '15-05-2020 09:45',
  850: '15-05-2020 09:45',
  851: '15-05-2020 09:45',
  852: '15-05-2020 09:45',
  853: '15-05-2020 09:45',
  854: '15-05-2020 09:45',
  855: '15-05-2020 09:45',
  856: '15-05-2020 09:45',
  857: '15-05-2020 09:45',
  858: '15-05-2020 09:45',
  859: '15-05-2020 09:45',
  860: '15-05-2020 10:00',
  861: '15-05-2020 10:00',
  862: '15-05-2020 10:00',
  863: '15-05-2020 10:00',
  864: '15-05-2020 10:00',
  865: '15-05-2020 10:00',
  866: '15-05-2020 10:00',
  867: '15-05-2020 10:00',
  868: '15-05-2020 10:00',
  869: '15-05-2020 10:00',
  870: '15-05-2020 10:00',
  871: '15-05-2020 10:00',
  872: '15-05-2020 10:00',
  873: '15-05-2020 10:00',
  874: '15-05-2020 10:00',
  875: '15-05-2020 10:00',
  876: '15-05-2020 10:00',
  877: '15-05-2020 10:00',
  878: '15-05-2020 10:00',
  879: '15-05-2020 10:00',
  880: '15-05-2020 10:00',
  881: '15-05-2020 10:00',
  882: '15-05-2020 10:15',
  883: '15-05-2020 10:15',
  884: '15-05-2020 10:15',
  885: '15-05-2020 10:15',
  886: '15-05-2020 10:15',
  887: '15-05-2020 10:15',
  888: '15-05-2020 10:15',
  889: '15-05-2020 10:15',
  890: '15-05-2020 10:15',
  891: '15-05-2020 10:15',
  892: '15-05-2020 10:15',
  893: '15-05-2020 10:15',
  894: '15-05-2020 10:15',
  895: '15-05-2020 10:15',
  896: '15-05-2020 10:15',
  897: '15-05-2020 10:15',
  898: '15-05-2020 10:15',
  899: '15-05-2020 10:15',
  900: '15-05-2020 10:15',
  901: '15-05-2020 10:15',
  902: '15-05-2020 10:15',
  903: '15-05-2020 10:15',
  904: '15-05-2020 10:30',
  905: '15-05-2020 10:30',
  906: '15-05-2020 10:30',
  907: '15-05-2020 10:30',
  908: '15-05-2020 10:30',
  909: '15-05-2020 10:30',
  910: '15-05-2020 10:30',
  911: '15-05-2020 10:30',
  912: '15-05-2020 10:30',
  913: '15-05-2020 10:30',
  914: '15-05-2020 10:30',
  915: '15-05-2020 10:30',
  916: '15-05-2020 10:30',
  917: '15-05-2020 10:30',
  918: '15-05-2020 10:30',
  919: '15-05-2020 10:30',
  920: '15-05-2020 10:30',
  921: '15-05-2020 10:30',
  922: '15-05-2020 10:30',
  923: '15-05-2020 10:30',
  924: '15-05-2020 10:30',
  925: '15-05-2020 10:30',
  926: '15-05-2020 10:45',
  927: '15-05-2020 10:45',
  928: '15-05-2020 10:45',
  929: '15-05-2020 10:45',
  930: '15-05-2020 10:45',
  931: '15-05-2020 10:45',
  932: '15-05-2020 10:45',
  933: '15-05-2020 10:45',
  934: '15-05-2020 10:45',
  935: '15-05-2020 10:45',
  936: '15-05-2020 10:45',
  937: '15-05-2020 10:45',
  938: '15-05-2020 10:45',
  939: '15-05-2020 10:45',
  940: '15-05-2020 10:45',
  941: '15-05-2020 10:45',
  942: '15-05-2020 10:45',
  943: '15-05-2020 10:45',
  944: '15-05-2020 10:45',
  945: '15-05-2020 10:45',
  946: '15-05-2020 10:45',
  947: '15-05-2020 10:45',
  948: '15-05-2020 11:00',
  949: '15-05-2020 11:00',
  950: '15-05-2020 11:00',
  951: '15-05-2020 11:00',
  952: '15-05-2020 11:00',
  953: '15-05-2020 11:00',
  954: '15-05-2020 11:00',
  955: '15-05-2020 11:00',
  956: '15-05-2020 11:00',
  957: '15-05-2020 11:00',
  958: '15-05-2020 11:00',
  959: '15-05-2020 11:00',
  960: '15-05-2020 11:00',
  961: '15-05-2020 11:00',
  962: '15-05-2020 11:00',
  963: '15-05-2020 11:00',
  964: '15-05-2020 11:00',
  965: '15-05-2020 11:00',
  966: '15-05-2020 11:00',
  967: '15-05-2020 11:00',
  968: '15-05-2020 11:00',
  969: '15-05-2020 11:00',
  970: '15-05-2020 11:15',
  971: '15-05-2020 11:15',
  972: '15-05-2020 11:15',
  973: '15-05-2020 11:15',
  974: '15-05-2020 11:15',
  975: '15-05-2020 11:15',
  976: '15-05-2020 11:15',
  977: '15-05-2020 11:15',
  978: '15-05-2020 11:15',
  979: '15-05-2020 11:15',
  980: '15-05-2020 11:15',
  981: '15-05-2020 11:15',
  982: '15-05-2020 11:15',
  983: '15-05-2020 11:15',
  984: '15-05-2020 11:15',
  985: '15-05-2020 11:15',
  986: '15-05-2020 11:15',
  987: '15-05-2020 11:15',
  988: '15-05-2020 11:15',
  989: '15-05-2020 11:15',
  990: '15-05-2020 11:15',
  991: '15-05-2020 11:15',
  992: '15-05-2020 11:30',
  993: '15-05-2020 11:30',
  994: '15-05-2020 11:30',
  995: '15-05-2020 11:30',
  996: '15-05-2020 11:30',
  997: '15-05-2020 11:30',
  998: '15-05-2020 11:30',
  999: '15-05-2020 11:30',
  ...},
 'PLANT_ID': {0: 4135001,
  1: 4135001,
  2: 4135001,
  3: 4135001,
  4: 4135001,
  5: 4135001,
  6: 4135001,
  7: 4135001,
  8: 4135001,
  9: 4135001,
  10: 4135001,
  11: 4135001,
  12: 4135001,
  13: 4135001,
  14: 4135001,
  15: 4135001,
  16: 4135001,
  17: 4135001,
  18: 4135001,
  19: 4135001,
  20: 4135001,
  21: 4135001,
  22: 4135001,
  23: 4135001,
  24: 4135001,
  25: 4135001,
  26: 4135001,
  27: 4135001,
  28: 4135001,
  29: 4135001,
  30: 4135001,
  31: 4135001,
  32: 4135001,
  33: 4135001,
  34: 4135001,
  35: 4135001,
  36: 4135001,
  37: 4135001,
  38: 4135001,
  39: 4135001,
  40: 4135001,
  41: 4135001,
  42: 4135001,
  43: 4135001,
  44: 4135001,
  45: 4135001,
  46: 4135001,
  47: 4135001,
  48: 4135001,
  49: 4135001,
  50: 4135001,
  51: 4135001,
  52: 4135001,
  53: 4135001,
  54: 4135001,
  55: 4135001,
  56: 4135001,
  57: 4135001,
  58: 4135001,
  59: 4135001,
  60: 4135001,
  61: 4135001,
  62: 4135001,
  63: 4135001,
  64: 4135001,
  65: 4135001,
  66: 4135001,
  67: 4135001,
  68: 4135001,
  69: 4135001,
  70: 4135001,
  71: 4135001,
  72: 4135001,
  73: 4135001,
  74: 4135001,
  75: 4135001,
  76: 4135001,
  77: 4135001,
  78: 4135001,
  79: 4135001,
  80: 4135001,
  81: 4135001,
  82: 4135001,
  83: 4135001,
  84: 4135001,
  85: 4135001,
  86: 4135001,
  87: 4135001,
  88: 4135001,
  89: 4135001,
  90: 4135001,
  91: 4135001,
  92: 4135001,
  93: 4135001,
  94: 4135001,
  95: 4135001,
  96: 4135001,
  97: 4135001,
  98: 4135001,
  99: 4135001,
  100: 4135001,
  101: 4135001,
  102: 4135001,
  103: 4135001,
  104: 4135001,
  105: 4135001,
  106: 4135001,
  107: 4135001,
  108: 4135001,
  109: 4135001,
  110: 4135001,
  111: 4135001,
  112: 4135001,
  113: 4135001,
  114: 4135001,
  115: 4135001,
  116: 4135001,
  117: 4135001,
  118: 4135001,
  119: 4135001,
  120: 4135001,
  121: 4135001,
  122: 4135001,
  123: 4135001,
  124: 4135001,
  125: 4135001,
  126: 4135001,
  127: 4135001,
  128: 4135001,
  129: 4135001,
  130: 4135001,
  131: 4135001,
  132: 4135001,
  133: 4135001,
  134: 4135001,
  135: 4135001,
  136: 4135001,
  137: 4135001,
  138: 4135001,
  139: 4135001,
  140: 4135001,
  141: 4135001,
  142: 4135001,
  143: 4135001,
  144: 4135001,
  145: 4135001,
  146: 4135001,
  147: 4135001,
  148: 4135001,
  149: 4135001,
  150: 4135001,
  151: 4135001,
  152: 4135001,
  153: 4135001,
  154: 4135001,
  155: 4135001,
  156: 4135001,
  157: 4135001,
  158: 4135001,
  159: 4135001,
  160: 4135001,
  161: 4135001,
  162: 4135001,
  163: 4135001,
  164: 4135001,
  165: 4135001,
  166: 4135001,
  167: 4135001,
  168: 4135001,
  169: 4135001,
  170: 4135001,
  171: 4135001,
  172: 4135001,
  173: 4135001,
  174: 4135001,
  175: 4135001,
  176: 4135001,
  177: 4135001,
  178: 4135001,
  179: 4135001,
  180: 4135001,
  181: 4135001,
  182: 4135001,
  183: 4135001,
  184: 4135001,
  185: 4135001,
  186: 4135001,
  187: 4135001,
  188: 4135001,
  189: 4135001,
  190: 4135001,
  191: 4135001,
  192: 4135001,
  193: 4135001,
  194: 4135001,
  195: 4135001,
  196: 4135001,
  197: 4135001,
  198: 4135001,
  199: 4135001,
  200: 4135001,
  201: 4135001,
  202: 4135001,
  203: 4135001,
  204: 4135001,
  205: 4135001,
  206: 4135001,
  207: 4135001,
  208: 4135001,
  209: 4135001,
  210: 4135001,
  211: 4135001,
  212: 4135001,
  213: 4135001,
  214: 4135001,
  215: 4135001,
  216: 4135001,
  217: 4135001,
  218: 4135001,
  219: 4135001,
  220: 4135001,
  221: 4135001,
  222: 4135001,
  223: 4135001,
  224: 4135001,
  225: 4135001,
  226: 4135001,
  227: 4135001,
  228: 4135001,
  229: 4135001,
  230: 4135001,
  231: 4135001,
  232: 4135001,
  233: 4135001,
  234: 4135001,
  235: 4135001,
  236: 4135001,
  237: 4135001,
  238: 4135001,
  239: 4135001,
  240: 4135001,
  241: 4135001,
  242: 4135001,
  243: 4135001,
  244: 4135001,
  245: 4135001,
  246: 4135001,
  247: 4135001,
  248: 4135001,
  249: 4135001,
  250: 4135001,
  251: 4135001,
  252: 4135001,
  253: 4135001,
  254: 4135001,
  255: 4135001,
  256: 4135001,
  257: 4135001,
  258: 4135001,
  259: 4135001,
  260: 4135001,
  261: 4135001,
  262: 4135001,
  263: 4135001,
  264: 4135001,
  265: 4135001,
  266: 4135001,
  267: 4135001,
  268: 4135001,
  269: 4135001,
  270: 4135001,
  271: 4135001,
  272: 4135001,
  273: 4135001,
  274: 4135001,
  275: 4135001,
  276: 4135001,
  277: 4135001,
  278: 4135001,
  279: 4135001,
  280: 4135001,
  281: 4135001,
  282: 4135001,
  283: 4135001,
  284: 4135001,
  285: 4135001,
  286: 4135001,
  287: 4135001,
  288: 4135001,
  289: 4135001,
  290: 4135001,
  291: 4135001,
  292: 4135001,
  293: 4135001,
  294: 4135001,
  295: 4135001,
  296: 4135001,
  297: 4135001,
  298: 4135001,
  299: 4135001,
  300: 4135001,
  301: 4135001,
  302: 4135001,
  303: 4135001,
  304: 4135001,
  305: 4135001,
  306: 4135001,
  307: 4135001,
  308: 4135001,
  309: 4135001,
  310: 4135001,
  311: 4135001,
  312: 4135001,
  313: 4135001,
  314: 4135001,
  315: 4135001,
  316: 4135001,
  317: 4135001,
  318: 4135001,
  319: 4135001,
  320: 4135001,
  321: 4135001,
  322: 4135001,
  323: 4135001,
  324: 4135001,
  325: 4135001,
  326: 4135001,
  327: 4135001,
  328: 4135001,
  329: 4135001,
  330: 4135001,
  331: 4135001,
  332: 4135001,
  333: 4135001,
  334: 4135001,
  335: 4135001,
  336: 4135001,
  337: 4135001,
  338: 4135001,
  339: 4135001,
  340: 4135001,
  341: 4135001,
  342: 4135001,
  343: 4135001,
  344: 4135001,
  345: 4135001,
  346: 4135001,
  347: 4135001,
  348: 4135001,
  349: 4135001,
  350: 4135001,
  351: 4135001,
  352: 4135001,
  353: 4135001,
  354: 4135001,
  355: 4135001,
  356: 4135001,
  357: 4135001,
  358: 4135001,
  359: 4135001,
  360: 4135001,
  361: 4135001,
  362: 4135001,
  363: 4135001,
  364: 4135001,
  365: 4135001,
  366: 4135001,
  367: 4135001,
  368: 4135001,
  369: 4135001,
  370: 4135001,
  371: 4135001,
  372: 4135001,
  373: 4135001,
  374: 4135001,
  375: 4135001,
  376: 4135001,
  377: 4135001,
  378: 4135001,
  379: 4135001,
  380: 4135001,
  381: 4135001,
  382: 4135001,
  383: 4135001,
  384: 4135001,
  385: 4135001,
  386: 4135001,
  387: 4135001,
  388: 4135001,
  389: 4135001,
  390: 4135001,
  391: 4135001,
  392: 4135001,
  393: 4135001,
  394: 4135001,
  395: 4135001,
  396: 4135001,
  397: 4135001,
  398: 4135001,
  399: 4135001,
  400: 4135001,
  401: 4135001,
  402: 4135001,
  403: 4135001,
  404: 4135001,
  405: 4135001,
  406: 4135001,
  407: 4135001,
  408: 4135001,
  409: 4135001,
  410: 4135001,
  411: 4135001,
  412: 4135001,
  413: 4135001,
  414: 4135001,
  415: 4135001,
  416: 4135001,
  417: 4135001,
  418: 4135001,
  419: 4135001,
  420: 4135001,
  421: 4135001,
  422: 4135001,
  423: 4135001,
  424: 4135001,
  425: 4135001,
  426: 4135001,
  427: 4135001,
  428: 4135001,
  429: 4135001,
  430: 4135001,
  431: 4135001,
  432: 4135001,
  433: 4135001,
  434: 4135001,
  435: 4135001,
  436: 4135001,
  437: 4135001,
  438: 4135001,
  439: 4135001,
  440: 4135001,
  441: 4135001,
  442: 4135001,
  443: 4135001,
  444: 4135001,
  445: 4135001,
  446: 4135001,
  447: 4135001,
  448: 4135001,
  449: 4135001,
  450: 4135001,
  451: 4135001,
  452: 4135001,
  453: 4135001,
  454: 4135001,
  455: 4135001,
  456: 4135001,
  457: 4135001,
  458: 4135001,
  459: 4135001,
  460: 4135001,
  461: 4135001,
  462: 4135001,
  463: 4135001,
  464: 4135001,
  465: 4135001,
  466: 4135001,
  467: 4135001,
  468: 4135001,
  469: 4135001,
  470: 4135001,
  471: 4135001,
  472: 4135001,
  473: 4135001,
  474: 4135001,
  475: 4135001,
  476: 4135001,
  477: 4135001,
  478: 4135001,
  479: 4135001,
  480: 4135001,
  481: 4135001,
  482: 4135001,
  483: 4135001,
  484: 4135001,
  485: 4135001,
  486: 4135001,
  487: 4135001,
  488: 4135001,
  489: 4135001,
  490: 4135001,
  491: 4135001,
  492: 4135001,
  493: 4135001,
  494: 4135001,
  495: 4135001,
  496: 4135001,
  497: 4135001,
  498: 4135001,
  499: 4135001,
  500: 4135001,
  501: 4135001,
  502: 4135001,
  503: 4135001,
  504: 4135001,
  505: 4135001,
  506: 4135001,
  507: 4135001,
  508: 4135001,
  509: 4135001,
  510: 4135001,
  511: 4135001,
  512: 4135001,
  513: 4135001,
  514: 4135001,
  515: 4135001,
  516: 4135001,
  517: 4135001,
  518: 4135001,
  519: 4135001,
  520: 4135001,
  521: 4135001,
  522: 4135001,
  523: 4135001,
  524: 4135001,
  525: 4135001,
  526: 4135001,
  527: 4135001,
  528: 4135001,
  529: 4135001,
  530: 4135001,
  531: 4135001,
  532: 4135001,
  533: 4135001,
  534: 4135001,
  535: 4135001,
  536: 4135001,
  537: 4135001,
  538: 4135001,
  539: 4135001,
  540: 4135001,
  541: 4135001,
  542: 4135001,
  543: 4135001,
  544: 4135001,
  545: 4135001,
  546: 4135001,
  547: 4135001,
  548: 4135001,
  549: 4135001,
  550: 4135001,
  551: 4135001,
  552: 4135001,
  553: 4135001,
  554: 4135001,
  555: 4135001,
  556: 4135001,
  557: 4135001,
  558: 4135001,
  559: 4135001,
  560: 4135001,
  561: 4135001,
  562: 4135001,
  563: 4135001,
  564: 4135001,
  565: 4135001,
  566: 4135001,
  567: 4135001,
  568: 4135001,
  569: 4135001,
  570: 4135001,
  571: 4135001,
  572: 4135001,
  573: 4135001,
  574: 4135001,
  575: 4135001,
  576: 4135001,
  577: 4135001,
  578: 4135001,
  579: 4135001,
  580: 4135001,
  581: 4135001,
  582: 4135001,
  583: 4135001,
  584: 4135001,
  585: 4135001,
  586: 4135001,
  587: 4135001,
  588: 4135001,
  589: 4135001,
  590: 4135001,
  591: 4135001,
  592: 4135001,
  593: 4135001,
  594: 4135001,
  595: 4135001,
  596: 4135001,
  597: 4135001,
  598: 4135001,
  599: 4135001,
  600: 4135001,
  601: 4135001,
  602: 4135001,
  603: 4135001,
  604: 4135001,
  605: 4135001,
  606: 4135001,
  607: 4135001,
  608: 4135001,
  609: 4135001,
  610: 4135001,
  611: 4135001,
  612: 4135001,
  613: 4135001,
  614: 4135001,
  615: 4135001,
  616: 4135001,
  617: 4135001,
  618: 4135001,
  619: 4135001,
  620: 4135001,
  621: 4135001,
  622: 4135001,
  623: 4135001,
  624: 4135001,
  625: 4135001,
  626: 4135001,
  627: 4135001,
  628: 4135001,
  629: 4135001,
  630: 4135001,
  631: 4135001,
  632: 4135001,
  633: 4135001,
  634: 4135001,
  635: 4135001,
  636: 4135001,
  637: 4135001,
  638: 4135001,
  639: 4135001,
  640: 4135001,
  641: 4135001,
  642: 4135001,
  643: 4135001,
  644: 4135001,
  645: 4135001,
  646: 4135001,
  647: 4135001,
  648: 4135001,
  649: 4135001,
  650: 4135001,
  651: 4135001,
  652: 4135001,
  653: 4135001,
  654: 4135001,
  655: 4135001,
  656: 4135001,
  657: 4135001,
  658: 4135001,
  659: 4135001,
  660: 4135001,
  661: 4135001,
  662: 4135001,
  663: 4135001,
  664: 4135001,
  665: 4135001,
  666: 4135001,
  667: 4135001,
  668: 4135001,
  669: 4135001,
  670: 4135001,
  671: 4135001,
  672: 4135001,
  673: 4135001,
  674: 4135001,
  675: 4135001,
  676: 4135001,
  677: 4135001,
  678: 4135001,
  679: 4135001,
  680: 4135001,
  681: 4135001,
  682: 4135001,
  683: 4135001,
  684: 4135001,
  685: 4135001,
  686: 4135001,
  687: 4135001,
  688: 4135001,
  689: 4135001,
  690: 4135001,
  691: 4135001,
  692: 4135001,
  693: 4135001,
  694: 4135001,
  695: 4135001,
  696: 4135001,
  697: 4135001,
  698: 4135001,
  699: 4135001,
  700: 4135001,
  701: 4135001,
  702: 4135001,
  703: 4135001,
  704: 4135001,
  705: 4135001,
  706: 4135001,
  707: 4135001,
  708: 4135001,
  709: 4135001,
  710: 4135001,
  711: 4135001,
  712: 4135001,
  713: 4135001,
  714: 4135001,
  715: 4135001,
  716: 4135001,
  717: 4135001,
  718: 4135001,
  719: 4135001,
  720: 4135001,
  721: 4135001,
  722: 4135001,
  723: 4135001,
  724: 4135001,
  725: 4135001,
  726: 4135001,
  727: 4135001,
  728: 4135001,
  729: 4135001,
  730: 4135001,
  731: 4135001,
  732: 4135001,
  733: 4135001,
  734: 4135001,
  735: 4135001,
  736: 4135001,
  737: 4135001,
  738: 4135001,
  739: 4135001,
  740: 4135001,
  741: 4135001,
  742: 4135001,
  743: 4135001,
  744: 4135001,
  745: 4135001,
  746: 4135001,
  747: 4135001,
  748: 4135001,
  749: 4135001,
  750: 4135001,
  751: 4135001,
  752: 4135001,
  753: 4135001,
  754: 4135001,
  755: 4135001,
  756: 4135001,
  757: 4135001,
  758: 4135001,
  759: 4135001,
  760: 4135001,
  761: 4135001,
  762: 4135001,
  763: 4135001,
  764: 4135001,
  765: 4135001,
  766: 4135001,
  767: 4135001,
  768: 4135001,
  769: 4135001,
  770: 4135001,
  771: 4135001,
  772: 4135001,
  773: 4135001,
  774: 4135001,
  775: 4135001,
  776: 4135001,
  777: 4135001,
  778: 4135001,
  779: 4135001,
  780: 4135001,
  781: 4135001,
  782: 4135001,
  783: 4135001,
  784: 4135001,
  785: 4135001,
  786: 4135001,
  787: 4135001,
  788: 4135001,
  789: 4135001,
  790: 4135001,
  791: 4135001,
  792: 4135001,
  793: 4135001,
  794: 4135001,
  795: 4135001,
  796: 4135001,
  797: 4135001,
  798: 4135001,
  799: 4135001,
  800: 4135001,
  801: 4135001,
  802: 4135001,
  803: 4135001,
  804: 4135001,
  805: 4135001,
  806: 4135001,
  807: 4135001,
  808: 4135001,
  809: 4135001,
  810: 4135001,
  811: 4135001,
  812: 4135001,
  813: 4135001,
  814: 4135001,
  815: 4135001,
  816: 4135001,
  817: 4135001,
  818: 4135001,
  819: 4135001,
  820: 4135001,
  821: 4135001,
  822: 4135001,
  823: 4135001,
  824: 4135001,
  825: 4135001,
  826: 4135001,
  827: 4135001,
  828: 4135001,
  829: 4135001,
  830: 4135001,
  831: 4135001,
  832: 4135001,
  833: 4135001,
  834: 4135001,
  835: 4135001,
  836: 4135001,
  837: 4135001,
  838: 4135001,
  839: 4135001,
  840: 4135001,
  841: 4135001,
  842: 4135001,
  843: 4135001,
  844: 4135001,
  845: 4135001,
  846: 4135001,
  847: 4135001,
  848: 4135001,
  849: 4135001,
  850: 4135001,
  851: 4135001,
  852: 4135001,
  853: 4135001,
  854: 4135001,
  855: 4135001,
  856: 4135001,
  857: 4135001,
  858: 4135001,
  859: 4135001,
  860: 4135001,
  861: 4135001,
  862: 4135001,
  863: 4135001,
  864: 4135001,
  865: 4135001,
  866: 4135001,
  867: 4135001,
  868: 4135001,
  869: 4135001,
  870: 4135001,
  871: 4135001,
  872: 4135001,
  873: 4135001,
  874: 4135001,
  875: 4135001,
  876: 4135001,
  877: 4135001,
  878: 4135001,
  879: 4135001,
  880: 4135001,
  881: 4135001,
  882: 4135001,
  883: 4135001,
  884: 4135001,
  885: 4135001,
  886: 4135001,
  887: 4135001,
  888: 4135001,
  889: 4135001,
  890: 4135001,
  891: 4135001,
  892: 4135001,
  893: 4135001,
  894: 4135001,
  895: 4135001,
  896: 4135001,
  897: 4135001,
  898: 4135001,
  899: 4135001,
  900: 4135001,
  901: 4135001,
  902: 4135001,
  903: 4135001,
  904: 4135001,
  905: 4135001,
  906: 4135001,
  907: 4135001,
  908: 4135001,
  909: 4135001,
  910: 4135001,
  911: 4135001,
  912: 4135001,
  913: 4135001,
  914: 4135001,
  915: 4135001,
  916: 4135001,
  917: 4135001,
  918: 4135001,
  919: 4135001,
  920: 4135001,
  921: 4135001,
  922: 4135001,
  923: 4135001,
  924: 4135001,
  925: 4135001,
  926: 4135001,
  927: 4135001,
  928: 4135001,
  929: 4135001,
  930: 4135001,
  931: 4135001,
  932: 4135001,
  933: 4135001,
  934: 4135001,
  935: 4135001,
  936: 4135001,
  937: 4135001,
  938: 4135001,
  939: 4135001,
  940: 4135001,
  941: 4135001,
  942: 4135001,
  943: 4135001,
  944: 4135001,
  945: 4135001,
  946: 4135001,
  947: 4135001,
  948: 4135001,
  949: 4135001,
  950: 4135001,
  951: 4135001,
  952: 4135001,
  953: 4135001,
  954: 4135001,
  955: 4135001,
  956: 4135001,
  957: 4135001,
  958: 4135001,
  959: 4135001,
  960: 4135001,
  961: 4135001,
  962: 4135001,
  963: 4135001,
  964: 4135001,
  965: 4135001,
  966: 4135001,
  967: 4135001,
  968: 4135001,
  969: 4135001,
  970: 4135001,
  971: 4135001,
  972: 4135001,
  973: 4135001,
  974: 4135001,
  975: 4135001,
  976: 4135001,
  977: 4135001,
  978: 4135001,
  979: 4135001,
  980: 4135001,
  981: 4135001,
  982: 4135001,
  983: 4135001,
  984: 4135001,
  985: 4135001,
  986: 4135001,
  987: 4135001,
  988: 4135001,
  989: 4135001,
  990: 4135001,
  991: 4135001,
  992: 4135001,
  993: 4135001,
  994: 4135001,
  995: 4135001,
  996: 4135001,
  997: 4135001,
  998: 4135001,
  999: 4135001,
  ...},
 'SOURCE_KEY': {0: '1BY6WEcLGh8j5v7',
  1: '1IF53ai7Xc0U56Y',
  2: '3PZuoBAID5Wc2HD',
  3: '7JYdWkrLSPkdwr4',
  4: 'McdE0feGgRqW7Ca',
  5: 'VHMLBKoKgIrUVDU',
  6: 'WRmjgnKYAwPKWDb',
  7: 'ZnxXDlPa8U1GXgE',
  8: 'ZoEaEvLYb1n2sOq',
  9: 'adLQvlD726eNBSB',
  10: 'bvBOhCH3iADSZry',
  11: 'iCRJl6heRkivqQ3',
  12: 'ih0vzX44oOqAx2f',
  13: 'pkci93gMrogZuBj',
  14: 'rGa61gmuvPhdLxV',
  15: 'sjndEbLyjtCKgGv',
  16: 'uHbuxQJl8lW7ozc',
  17: 'wCURE6d3bPkepu2',
  18: 'z9Y9gH1T5YWrNuG',
  19: 'zBIq5rxdHJRwDNY',
  20: 'zVJPv84UY57bAof',
  21: '1BY6WEcLGh8j5v7',
  22: '1IF53ai7Xc0U56Y',
  23: '3PZuoBAID5Wc2HD',
  24: '7JYdWkrLSPkdwr4',
  25: 'McdE0feGgRqW7Ca',
  26: 'VHMLBKoKgIrUVDU',
  27: 'WRmjgnKYAwPKWDb',
  28: 'ZnxXDlPa8U1GXgE',
  29: 'ZoEaEvLYb1n2sOq',
  30: 'adLQvlD726eNBSB',
  31: 'bvBOhCH3iADSZry',
  32: 'iCRJl6heRkivqQ3',
  33: 'ih0vzX44oOqAx2f',
  34: 'pkci93gMrogZuBj',
  35: 'rGa61gmuvPhdLxV',
  36: 'sjndEbLyjtCKgGv',
  37: 'uHbuxQJl8lW7ozc',
  38: 'wCURE6d3bPkepu2',
  39: 'z9Y9gH1T5YWrNuG',
  40: 'zBIq5rxdHJRwDNY',
  41: 'zVJPv84UY57bAof',
  42: '1BY6WEcLGh8j5v7',
  43: '1IF53ai7Xc0U56Y',
  44: '3PZuoBAID5Wc2HD',
  45: '7JYdWkrLSPkdwr4',
  46: 'McdE0feGgRqW7Ca',
  47: 'VHMLBKoKgIrUVDU',
  48: 'WRmjgnKYAwPKWDb',
  49: 'ZnxXDlPa8U1GXgE',
  50: 'ZoEaEvLYb1n2sOq',
  51: 'adLQvlD726eNBSB',
  52: 'bvBOhCH3iADSZry',
  53: 'iCRJl6heRkivqQ3',
  54: 'ih0vzX44oOqAx2f',
  55: 'pkci93gMrogZuBj',
  56: 'rGa61gmuvPhdLxV',
  57: 'sjndEbLyjtCKgGv',
  58: 'uHbuxQJl8lW7ozc',
  59: 'wCURE6d3bPkepu2',
  60: 'z9Y9gH1T5YWrNuG',
  61: 'zBIq5rxdHJRwDNY',
  62: 'zVJPv84UY57bAof',
  63: '1BY6WEcLGh8j5v7',
  64: '1IF53ai7Xc0U56Y',
  65: '3PZuoBAID5Wc2HD',
  66: '7JYdWkrLSPkdwr4',
  67: 'McdE0feGgRqW7Ca',
  68: 'VHMLBKoKgIrUVDU',
  69: 'WRmjgnKYAwPKWDb',
  70: 'ZnxXDlPa8U1GXgE',
  71: 'ZoEaEvLYb1n2sOq',
  72: 'adLQvlD726eNBSB',
  73: 'bvBOhCH3iADSZry',
  74: 'iCRJl6heRkivqQ3',
  75: 'ih0vzX44oOqAx2f',
  76: 'pkci93gMrogZuBj',
  77: 'rGa61gmuvPhdLxV',
  78: 'sjndEbLyjtCKgGv',
  79: 'uHbuxQJl8lW7ozc',
  80: 'wCURE6d3bPkepu2',
  81: 'z9Y9gH1T5YWrNuG',
  82: 'zBIq5rxdHJRwDNY',
  83: 'zVJPv84UY57bAof',
  84: '1BY6WEcLGh8j5v7',
  85: '1IF53ai7Xc0U56Y',
  86: '3PZuoBAID5Wc2HD',
  87: '7JYdWkrLSPkdwr4',
  88: 'McdE0feGgRqW7Ca',
  89: 'VHMLBKoKgIrUVDU',
  90: 'WRmjgnKYAwPKWDb',
  91: 'YxYtjZvoooNbGkE',
  92: 'ZnxXDlPa8U1GXgE',
  93: 'ZoEaEvLYb1n2sOq',
  94: 'adLQvlD726eNBSB',
  95: 'bvBOhCH3iADSZry',
  96: 'iCRJl6heRkivqQ3',
  97: 'ih0vzX44oOqAx2f',
  98: 'pkci93gMrogZuBj',
  99: 'rGa61gmuvPhdLxV',
  100: 'sjndEbLyjtCKgGv',
  101: 'uHbuxQJl8lW7ozc',
  102: 'wCURE6d3bPkepu2',
  103: 'z9Y9gH1T5YWrNuG',
  104: 'zBIq5rxdHJRwDNY',
  105: 'zVJPv84UY57bAof',
  106: '1BY6WEcLGh8j5v7',
  107: '1IF53ai7Xc0U56Y',
  108: '3PZuoBAID5Wc2HD',
  109: '7JYdWkrLSPkdwr4',
  110: 'McdE0feGgRqW7Ca',
  111: 'VHMLBKoKgIrUVDU',
  112: 'WRmjgnKYAwPKWDb',
  113: 'ZnxXDlPa8U1GXgE',
  114: 'ZoEaEvLYb1n2sOq',
  115: 'adLQvlD726eNBSB',
  116: 'bvBOhCH3iADSZry',
  117: 'iCRJl6heRkivqQ3',
  118: 'ih0vzX44oOqAx2f',
  119: 'pkci93gMrogZuBj',
  120: 'rGa61gmuvPhdLxV',
  121: 'sjndEbLyjtCKgGv',
  122: 'uHbuxQJl8lW7ozc',
  123: 'wCURE6d3bPkepu2',
  124: 'z9Y9gH1T5YWrNuG',
  125: 'zBIq5rxdHJRwDNY',
  126: 'zVJPv84UY57bAof',
  127: '1BY6WEcLGh8j5v7',
  128: '1IF53ai7Xc0U56Y',
  129: '3PZuoBAID5Wc2HD',
  130: '7JYdWkrLSPkdwr4',
  131: 'McdE0feGgRqW7Ca',
  132: 'VHMLBKoKgIrUVDU',
  133: 'WRmjgnKYAwPKWDb',
  134: 'ZnxXDlPa8U1GXgE',
  135: 'ZoEaEvLYb1n2sOq',
  136: 'adLQvlD726eNBSB',
  137: 'bvBOhCH3iADSZry',
  138: 'iCRJl6heRkivqQ3',
  139: 'ih0vzX44oOqAx2f',
  140: 'pkci93gMrogZuBj',
  141: 'rGa61gmuvPhdLxV',
  142: 'sjndEbLyjtCKgGv',
  143: 'uHbuxQJl8lW7ozc',
  144: 'wCURE6d3bPkepu2',
  145: 'z9Y9gH1T5YWrNuG',
  146: 'zBIq5rxdHJRwDNY',
  147: 'zVJPv84UY57bAof',
  148: '1BY6WEcLGh8j5v7',
  149: '1IF53ai7Xc0U56Y',
  150: '3PZuoBAID5Wc2HD',
  151: '7JYdWkrLSPkdwr4',
  152: 'McdE0feGgRqW7Ca',
  153: 'VHMLBKoKgIrUVDU',
  154: 'WRmjgnKYAwPKWDb',
  155: 'YxYtjZvoooNbGkE',
  156: 'ZnxXDlPa8U1GXgE',
  157: 'ZoEaEvLYb1n2sOq',
  158: 'adLQvlD726eNBSB',
  159: 'bvBOhCH3iADSZry',
  160: 'iCRJl6heRkivqQ3',
  161: 'ih0vzX44oOqAx2f',
  162: 'pkci93gMrogZuBj',
  163: 'rGa61gmuvPhdLxV',
  164: 'sjndEbLyjtCKgGv',
  165: 'uHbuxQJl8lW7ozc',
  166: 'wCURE6d3bPkepu2',
  167: 'z9Y9gH1T5YWrNuG',
  168: 'zBIq5rxdHJRwDNY',
  169: 'zVJPv84UY57bAof',
  170: '1BY6WEcLGh8j5v7',
  171: '1IF53ai7Xc0U56Y',
  172: '3PZuoBAID5Wc2HD',
  173: '7JYdWkrLSPkdwr4',
  174: 'McdE0feGgRqW7Ca',
  175: 'VHMLBKoKgIrUVDU',
  176: 'WRmjgnKYAwPKWDb',
  177: 'ZnxXDlPa8U1GXgE',
  178: 'ZoEaEvLYb1n2sOq',
  179: 'adLQvlD726eNBSB',
  180: 'bvBOhCH3iADSZry',
  181: 'iCRJl6heRkivqQ3',
  182: 'ih0vzX44oOqAx2f',
  183: 'pkci93gMrogZuBj',
  184: 'rGa61gmuvPhdLxV',
  185: 'sjndEbLyjtCKgGv',
  186: 'uHbuxQJl8lW7ozc',
  187: 'wCURE6d3bPkepu2',
  188: 'z9Y9gH1T5YWrNuG',
  189: 'zBIq5rxdHJRwDNY',
  190: 'zVJPv84UY57bAof',
  191: '1BY6WEcLGh8j5v7',
  192: '1IF53ai7Xc0U56Y',
  193: '3PZuoBAID5Wc2HD',
  194: '7JYdWkrLSPkdwr4',
  195: 'McdE0feGgRqW7Ca',
  196: 'VHMLBKoKgIrUVDU',
  197: 'WRmjgnKYAwPKWDb',
  198: 'ZnxXDlPa8U1GXgE',
  199: 'ZoEaEvLYb1n2sOq',
  200: 'adLQvlD726eNBSB',
  201: 'bvBOhCH3iADSZry',
  202: 'iCRJl6heRkivqQ3',
  203: 'ih0vzX44oOqAx2f',
  204: 'pkci93gMrogZuBj',
  205: 'rGa61gmuvPhdLxV',
  206: 'sjndEbLyjtCKgGv',
  207: 'uHbuxQJl8lW7ozc',
  208: 'wCURE6d3bPkepu2',
  209: 'z9Y9gH1T5YWrNuG',
  210: 'zBIq5rxdHJRwDNY',
  211: 'zVJPv84UY57bAof',
  212: '1BY6WEcLGh8j5v7',
  213: '1IF53ai7Xc0U56Y',
  214: '3PZuoBAID5Wc2HD',
  215: '7JYdWkrLSPkdwr4',
  216: 'McdE0feGgRqW7Ca',
  217: 'VHMLBKoKgIrUVDU',
  218: 'WRmjgnKYAwPKWDb',
  219: 'ZnxXDlPa8U1GXgE',
  220: 'ZoEaEvLYb1n2sOq',
  221: 'adLQvlD726eNBSB',
  222: 'bvBOhCH3iADSZry',
  223: 'iCRJl6heRkivqQ3',
  224: 'ih0vzX44oOqAx2f',
  225: 'pkci93gMrogZuBj',
  226: 'rGa61gmuvPhdLxV',
  227: 'sjndEbLyjtCKgGv',
  228: 'uHbuxQJl8lW7ozc',
  229: 'wCURE6d3bPkepu2',
  230: 'z9Y9gH1T5YWrNuG',
  231: 'zBIq5rxdHJRwDNY',
  232: 'zVJPv84UY57bAof',
  233: '1BY6WEcLGh8j5v7',
  234: '1IF53ai7Xc0U56Y',
  235: '3PZuoBAID5Wc2HD',
  236: '7JYdWkrLSPkdwr4',
  237: 'McdE0feGgRqW7Ca',
  238: 'VHMLBKoKgIrUVDU',
  239: 'WRmjgnKYAwPKWDb',
  240: 'ZnxXDlPa8U1GXgE',
  241: 'ZoEaEvLYb1n2sOq',
  242: 'adLQvlD726eNBSB',
  243: 'bvBOhCH3iADSZry',
  244: 'iCRJl6heRkivqQ3',
  245: 'ih0vzX44oOqAx2f',
  246: 'pkci93gMrogZuBj',
  247: 'rGa61gmuvPhdLxV',
  248: 'sjndEbLyjtCKgGv',
  249: 'uHbuxQJl8lW7ozc',
  250: 'wCURE6d3bPkepu2',
  251: 'z9Y9gH1T5YWrNuG',
  252: 'zBIq5rxdHJRwDNY',
  253: 'zVJPv84UY57bAof',
  254: '1BY6WEcLGh8j5v7',
  255: '1IF53ai7Xc0U56Y',
  256: '3PZuoBAID5Wc2HD',
  257: '7JYdWkrLSPkdwr4',
  258: 'McdE0feGgRqW7Ca',
  259: 'VHMLBKoKgIrUVDU',
  260: 'WRmjgnKYAwPKWDb',
  261: 'ZnxXDlPa8U1GXgE',
  262: 'ZoEaEvLYb1n2sOq',
  263: 'adLQvlD726eNBSB',
  264: 'bvBOhCH3iADSZry',
  265: 'iCRJl6heRkivqQ3',
  266: 'ih0vzX44oOqAx2f',
  267: 'pkci93gMrogZuBj',
  268: 'rGa61gmuvPhdLxV',
  269: 'sjndEbLyjtCKgGv',
  270: 'uHbuxQJl8lW7ozc',
  271: 'wCURE6d3bPkepu2',
  272: 'z9Y9gH1T5YWrNuG',
  273: 'zBIq5rxdHJRwDNY',
  274: 'zVJPv84UY57bAof',
  275: '1BY6WEcLGh8j5v7',
  276: '1IF53ai7Xc0U56Y',
  277: '3PZuoBAID5Wc2HD',
  278: '7JYdWkrLSPkdwr4',
  279: 'McdE0feGgRqW7Ca',
  280: 'VHMLBKoKgIrUVDU',
  281: 'WRmjgnKYAwPKWDb',
  282: 'ZnxXDlPa8U1GXgE',
  283: 'ZoEaEvLYb1n2sOq',
  284: 'adLQvlD726eNBSB',
  285: 'bvBOhCH3iADSZry',
  286: 'iCRJl6heRkivqQ3',
  287: 'ih0vzX44oOqAx2f',
  288: 'pkci93gMrogZuBj',
  289: 'rGa61gmuvPhdLxV',
  290: 'sjndEbLyjtCKgGv',
  291: 'uHbuxQJl8lW7ozc',
  292: 'wCURE6d3bPkepu2',
  293: 'z9Y9gH1T5YWrNuG',
  294: 'zBIq5rxdHJRwDNY',
  295: 'zVJPv84UY57bAof',
  296: '1BY6WEcLGh8j5v7',
  297: '1IF53ai7Xc0U56Y',
  298: '3PZuoBAID5Wc2HD',
  299: '7JYdWkrLSPkdwr4',
  300: 'McdE0feGgRqW7Ca',
  301: 'VHMLBKoKgIrUVDU',
  302: 'WRmjgnKYAwPKWDb',
  303: 'ZnxXDlPa8U1GXgE',
  304: 'ZoEaEvLYb1n2sOq',
  305: 'adLQvlD726eNBSB',
  306: 'bvBOhCH3iADSZry',
  307: 'iCRJl6heRkivqQ3',
  308: 'ih0vzX44oOqAx2f',
  309: 'pkci93gMrogZuBj',
  310: 'rGa61gmuvPhdLxV',
  311: 'sjndEbLyjtCKgGv',
  312: 'uHbuxQJl8lW7ozc',
  313: 'wCURE6d3bPkepu2',
  314: 'z9Y9gH1T5YWrNuG',
  315: 'zBIq5rxdHJRwDNY',
  316: 'zVJPv84UY57bAof',
  317: '1BY6WEcLGh8j5v7',
  318: '1IF53ai7Xc0U56Y',
  319: '3PZuoBAID5Wc2HD',
  320: '7JYdWkrLSPkdwr4',
  321: 'McdE0feGgRqW7Ca',
  322: 'VHMLBKoKgIrUVDU',
  323: 'WRmjgnKYAwPKWDb',
  324: 'ZnxXDlPa8U1GXgE',
  325: 'ZoEaEvLYb1n2sOq',
  326: 'adLQvlD726eNBSB',
  327: 'bvBOhCH3iADSZry',
  328: 'iCRJl6heRkivqQ3',
  329: 'ih0vzX44oOqAx2f',
  330: 'pkci93gMrogZuBj',
  331: 'rGa61gmuvPhdLxV',
  332: 'sjndEbLyjtCKgGv',
  333: 'uHbuxQJl8lW7ozc',
  334: 'wCURE6d3bPkepu2',
  335: 'z9Y9gH1T5YWrNuG',
  336: 'zBIq5rxdHJRwDNY',
  337: 'zVJPv84UY57bAof',
  338: '1BY6WEcLGh8j5v7',
  339: '1IF53ai7Xc0U56Y',
  340: '3PZuoBAID5Wc2HD',
  341: '7JYdWkrLSPkdwr4',
  342: 'McdE0feGgRqW7Ca',
  343: 'VHMLBKoKgIrUVDU',
  344: 'WRmjgnKYAwPKWDb',
  345: 'ZnxXDlPa8U1GXgE',
  346: 'ZoEaEvLYb1n2sOq',
  347: 'adLQvlD726eNBSB',
  348: 'bvBOhCH3iADSZry',
  349: 'iCRJl6heRkivqQ3',
  350: 'ih0vzX44oOqAx2f',
  351: 'pkci93gMrogZuBj',
  352: 'rGa61gmuvPhdLxV',
  353: 'sjndEbLyjtCKgGv',
  354: 'uHbuxQJl8lW7ozc',
  355: 'wCURE6d3bPkepu2',
  356: 'z9Y9gH1T5YWrNuG',
  357: 'zBIq5rxdHJRwDNY',
  358: 'zVJPv84UY57bAof',
  359: '1BY6WEcLGh8j5v7',
  360: '1IF53ai7Xc0U56Y',
  361: '3PZuoBAID5Wc2HD',
  362: '7JYdWkrLSPkdwr4',
  363: 'McdE0feGgRqW7Ca',
  364: 'VHMLBKoKgIrUVDU',
  365: 'WRmjgnKYAwPKWDb',
  366: 'YxYtjZvoooNbGkE',
  367: 'ZnxXDlPa8U1GXgE',
  368: 'ZoEaEvLYb1n2sOq',
  369: 'adLQvlD726eNBSB',
  370: 'bvBOhCH3iADSZry',
  371: 'iCRJl6heRkivqQ3',
  372: 'ih0vzX44oOqAx2f',
  373: 'pkci93gMrogZuBj',
  374: 'rGa61gmuvPhdLxV',
  375: 'sjndEbLyjtCKgGv',
  376: 'uHbuxQJl8lW7ozc',
  377: 'wCURE6d3bPkepu2',
  378: 'z9Y9gH1T5YWrNuG',
  379: 'zBIq5rxdHJRwDNY',
  380: 'zVJPv84UY57bAof',
  381: '1BY6WEcLGh8j5v7',
  382: '1IF53ai7Xc0U56Y',
  383: '3PZuoBAID5Wc2HD',
  384: '7JYdWkrLSPkdwr4',
  385: 'McdE0feGgRqW7Ca',
  386: 'VHMLBKoKgIrUVDU',
  387: 'WRmjgnKYAwPKWDb',
  388: 'YxYtjZvoooNbGkE',
  389: 'ZnxXDlPa8U1GXgE',
  390: 'ZoEaEvLYb1n2sOq',
  391: 'adLQvlD726eNBSB',
  392: 'bvBOhCH3iADSZry',
  393: 'iCRJl6heRkivqQ3',
  394: 'ih0vzX44oOqAx2f',
  395: 'pkci93gMrogZuBj',
  396: 'rGa61gmuvPhdLxV',
  397: 'sjndEbLyjtCKgGv',
  398: 'uHbuxQJl8lW7ozc',
  399: 'wCURE6d3bPkepu2',
  400: 'z9Y9gH1T5YWrNuG',
  401: 'zBIq5rxdHJRwDNY',
  402: 'zVJPv84UY57bAof',
  403: '1BY6WEcLGh8j5v7',
  404: '1IF53ai7Xc0U56Y',
  405: '3PZuoBAID5Wc2HD',
  406: '7JYdWkrLSPkdwr4',
  407: 'McdE0feGgRqW7Ca',
  408: 'VHMLBKoKgIrUVDU',
  409: 'WRmjgnKYAwPKWDb',
  410: 'YxYtjZvoooNbGkE',
  411: 'ZnxXDlPa8U1GXgE',
  412: 'ZoEaEvLYb1n2sOq',
  413: 'adLQvlD726eNBSB',
  414: 'bvBOhCH3iADSZry',
  415: 'iCRJl6heRkivqQ3',
  416: 'ih0vzX44oOqAx2f',
  417: 'pkci93gMrogZuBj',
  418: 'rGa61gmuvPhdLxV',
  419: 'sjndEbLyjtCKgGv',
  420: 'uHbuxQJl8lW7ozc',
  421: 'wCURE6d3bPkepu2',
  422: 'z9Y9gH1T5YWrNuG',
  423: 'zBIq5rxdHJRwDNY',
  424: 'zVJPv84UY57bAof',
  425: '1BY6WEcLGh8j5v7',
  426: '1IF53ai7Xc0U56Y',
  427: '3PZuoBAID5Wc2HD',
  428: '7JYdWkrLSPkdwr4',
  429: 'McdE0feGgRqW7Ca',
  430: 'VHMLBKoKgIrUVDU',
  431: 'WRmjgnKYAwPKWDb',
  432: 'YxYtjZvoooNbGkE',
  433: 'ZnxXDlPa8U1GXgE',
  434: 'ZoEaEvLYb1n2sOq',
  435: 'adLQvlD726eNBSB',
  436: 'bvBOhCH3iADSZry',
  437: 'iCRJl6heRkivqQ3',
  438: 'ih0vzX44oOqAx2f',
  439: 'pkci93gMrogZuBj',
  440: 'rGa61gmuvPhdLxV',
  441: 'sjndEbLyjtCKgGv',
  442: 'uHbuxQJl8lW7ozc',
  443: 'wCURE6d3bPkepu2',
  444: 'z9Y9gH1T5YWrNuG',
  445: 'zBIq5rxdHJRwDNY',
  446: 'zVJPv84UY57bAof',
  447: '1BY6WEcLGh8j5v7',
  448: '1IF53ai7Xc0U56Y',
  449: '3PZuoBAID5Wc2HD',
  450: '7JYdWkrLSPkdwr4',
  451: 'McdE0feGgRqW7Ca',
  452: 'VHMLBKoKgIrUVDU',
  453: 'WRmjgnKYAwPKWDb',
  454: 'ZnxXDlPa8U1GXgE',
  455: 'ZoEaEvLYb1n2sOq',
  456: 'adLQvlD726eNBSB',
  457: 'bvBOhCH3iADSZry',
  458: 'iCRJl6heRkivqQ3',
  459: 'ih0vzX44oOqAx2f',
  460: 'pkci93gMrogZuBj',
  461: 'rGa61gmuvPhdLxV',
  462: 'sjndEbLyjtCKgGv',
  463: 'uHbuxQJl8lW7ozc',
  464: 'wCURE6d3bPkepu2',
  465: 'z9Y9gH1T5YWrNuG',
  466: 'zBIq5rxdHJRwDNY',
  467: 'zVJPv84UY57bAof',
  468: '1BY6WEcLGh8j5v7',
  469: '1IF53ai7Xc0U56Y',
  470: '3PZuoBAID5Wc2HD',
  471: '7JYdWkrLSPkdwr4',
  472: 'McdE0feGgRqW7Ca',
  473: 'VHMLBKoKgIrUVDU',
  474: 'WRmjgnKYAwPKWDb',
  475: 'ZnxXDlPa8U1GXgE',
  476: 'ZoEaEvLYb1n2sOq',
  477: 'adLQvlD726eNBSB',
  478: 'bvBOhCH3iADSZry',
  479: 'iCRJl6heRkivqQ3',
  480: 'ih0vzX44oOqAx2f',
  481: 'pkci93gMrogZuBj',
  482: 'rGa61gmuvPhdLxV',
  483: 'sjndEbLyjtCKgGv',
  484: 'uHbuxQJl8lW7ozc',
  485: 'wCURE6d3bPkepu2',
  486: 'z9Y9gH1T5YWrNuG',
  487: 'zBIq5rxdHJRwDNY',
  488: 'zVJPv84UY57bAof',
  489: '1BY6WEcLGh8j5v7',
  490: '1IF53ai7Xc0U56Y',
  491: '3PZuoBAID5Wc2HD',
  492: '7JYdWkrLSPkdwr4',
  493: 'McdE0feGgRqW7Ca',
  494: 'VHMLBKoKgIrUVDU',
  495: 'WRmjgnKYAwPKWDb',
  496: 'ZnxXDlPa8U1GXgE',
  497: 'ZoEaEvLYb1n2sOq',
  498: 'adLQvlD726eNBSB',
  499: 'bvBOhCH3iADSZry',
  500: 'iCRJl6heRkivqQ3',
  501: 'ih0vzX44oOqAx2f',
  502: 'pkci93gMrogZuBj',
  503: 'rGa61gmuvPhdLxV',
  504: 'sjndEbLyjtCKgGv',
  505: 'uHbuxQJl8lW7ozc',
  506: 'wCURE6d3bPkepu2',
  507: 'z9Y9gH1T5YWrNuG',
  508: 'zBIq5rxdHJRwDNY',
  509: 'zVJPv84UY57bAof',
  510: '1BY6WEcLGh8j5v7',
  511: '1IF53ai7Xc0U56Y',
  512: '3PZuoBAID5Wc2HD',
  513: '7JYdWkrLSPkdwr4',
  514: 'McdE0feGgRqW7Ca',
  515: 'VHMLBKoKgIrUVDU',
  516: 'WRmjgnKYAwPKWDb',
  517: 'ZnxXDlPa8U1GXgE',
  518: 'ZoEaEvLYb1n2sOq',
  519: 'adLQvlD726eNBSB',
  520: 'bvBOhCH3iADSZry',
  521: 'iCRJl6heRkivqQ3',
  522: 'ih0vzX44oOqAx2f',
  523: 'pkci93gMrogZuBj',
  524: 'rGa61gmuvPhdLxV',
  525: 'sjndEbLyjtCKgGv',
  526: 'uHbuxQJl8lW7ozc',
  527: 'wCURE6d3bPkepu2',
  528: 'z9Y9gH1T5YWrNuG',
  529: 'zBIq5rxdHJRwDNY',
  530: 'zVJPv84UY57bAof',
  531: '1BY6WEcLGh8j5v7',
  532: '1IF53ai7Xc0U56Y',
  533: '3PZuoBAID5Wc2HD',
  534: '7JYdWkrLSPkdwr4',
  535: 'McdE0feGgRqW7Ca',
  536: 'VHMLBKoKgIrUVDU',
  537: 'WRmjgnKYAwPKWDb',
  538: 'ZnxXDlPa8U1GXgE',
  539: 'ZoEaEvLYb1n2sOq',
  540: 'adLQvlD726eNBSB',
  541: 'bvBOhCH3iADSZry',
  542: 'iCRJl6heRkivqQ3',
  543: 'ih0vzX44oOqAx2f',
  544: 'pkci93gMrogZuBj',
  545: 'rGa61gmuvPhdLxV',
  546: 'sjndEbLyjtCKgGv',
  547: 'uHbuxQJl8lW7ozc',
  548: 'wCURE6d3bPkepu2',
  549: 'z9Y9gH1T5YWrNuG',
  550: 'zBIq5rxdHJRwDNY',
  551: 'zVJPv84UY57bAof',
  552: '1BY6WEcLGh8j5v7',
  553: '1IF53ai7Xc0U56Y',
  554: '3PZuoBAID5Wc2HD',
  555: '7JYdWkrLSPkdwr4',
  556: 'McdE0feGgRqW7Ca',
  557: 'VHMLBKoKgIrUVDU',
  558: 'WRmjgnKYAwPKWDb',
  559: 'YxYtjZvoooNbGkE',
  560: 'ZnxXDlPa8U1GXgE',
  561: 'ZoEaEvLYb1n2sOq',
  562: 'adLQvlD726eNBSB',
  563: 'bvBOhCH3iADSZry',
  564: 'iCRJl6heRkivqQ3',
  565: 'ih0vzX44oOqAx2f',
  566: 'pkci93gMrogZuBj',
  567: 'rGa61gmuvPhdLxV',
  568: 'sjndEbLyjtCKgGv',
  569: 'uHbuxQJl8lW7ozc',
  570: 'wCURE6d3bPkepu2',
  571: 'z9Y9gH1T5YWrNuG',
  572: 'zBIq5rxdHJRwDNY',
  573: 'zVJPv84UY57bAof',
  574: '1BY6WEcLGh8j5v7',
  575: '1IF53ai7Xc0U56Y',
  576: '3PZuoBAID5Wc2HD',
  577: '7JYdWkrLSPkdwr4',
  578: 'McdE0feGgRqW7Ca',
  579: 'VHMLBKoKgIrUVDU',
  580: 'WRmjgnKYAwPKWDb',
  581: 'YxYtjZvoooNbGkE',
  582: 'ZnxXDlPa8U1GXgE',
  583: 'ZoEaEvLYb1n2sOq',
  584: 'adLQvlD726eNBSB',
  585: 'bvBOhCH3iADSZry',
  586: 'iCRJl6heRkivqQ3',
  587: 'ih0vzX44oOqAx2f',
  588: 'pkci93gMrogZuBj',
  589: 'rGa61gmuvPhdLxV',
  590: 'sjndEbLyjtCKgGv',
  591: 'uHbuxQJl8lW7ozc',
  592: 'wCURE6d3bPkepu2',
  593: 'z9Y9gH1T5YWrNuG',
  594: 'zBIq5rxdHJRwDNY',
  595: 'zVJPv84UY57bAof',
  596: '1BY6WEcLGh8j5v7',
  597: '1IF53ai7Xc0U56Y',
  598: '3PZuoBAID5Wc2HD',
  599: '7JYdWkrLSPkdwr4',
  600: 'McdE0feGgRqW7Ca',
  601: 'VHMLBKoKgIrUVDU',
  602: 'WRmjgnKYAwPKWDb',
  603: 'YxYtjZvoooNbGkE',
  604: 'ZnxXDlPa8U1GXgE',
  605: 'ZoEaEvLYb1n2sOq',
  606: 'adLQvlD726eNBSB',
  607: 'bvBOhCH3iADSZry',
  608: 'iCRJl6heRkivqQ3',
  609: 'ih0vzX44oOqAx2f',
  610: 'pkci93gMrogZuBj',
  611: 'rGa61gmuvPhdLxV',
  612: 'sjndEbLyjtCKgGv',
  613: 'uHbuxQJl8lW7ozc',
  614: 'wCURE6d3bPkepu2',
  615: 'z9Y9gH1T5YWrNuG',
  616: 'zBIq5rxdHJRwDNY',
  617: 'zVJPv84UY57bAof',
  618: '1BY6WEcLGh8j5v7',
  619: '1IF53ai7Xc0U56Y',
  620: '3PZuoBAID5Wc2HD',
  621: '7JYdWkrLSPkdwr4',
  622: 'McdE0feGgRqW7Ca',
  623: 'VHMLBKoKgIrUVDU',
  624: 'WRmjgnKYAwPKWDb',
  625: 'YxYtjZvoooNbGkE',
  626: 'ZnxXDlPa8U1GXgE',
  627: 'ZoEaEvLYb1n2sOq',
  628: 'adLQvlD726eNBSB',
  629: 'bvBOhCH3iADSZry',
  630: 'iCRJl6heRkivqQ3',
  631: 'ih0vzX44oOqAx2f',
  632: 'pkci93gMrogZuBj',
  633: 'rGa61gmuvPhdLxV',
  634: 'sjndEbLyjtCKgGv',
  635: 'uHbuxQJl8lW7ozc',
  636: 'wCURE6d3bPkepu2',
  637: 'z9Y9gH1T5YWrNuG',
  638: 'zBIq5rxdHJRwDNY',
  639: 'zVJPv84UY57bAof',
  640: '1BY6WEcLGh8j5v7',
  641: '1IF53ai7Xc0U56Y',
  642: '3PZuoBAID5Wc2HD',
  643: '7JYdWkrLSPkdwr4',
  644: 'McdE0feGgRqW7Ca',
  645: 'VHMLBKoKgIrUVDU',
  646: 'WRmjgnKYAwPKWDb',
  647: 'YxYtjZvoooNbGkE',
  648: 'ZnxXDlPa8U1GXgE',
  649: 'ZoEaEvLYb1n2sOq',
  650: 'adLQvlD726eNBSB',
  651: 'bvBOhCH3iADSZry',
  652: 'iCRJl6heRkivqQ3',
  653: 'ih0vzX44oOqAx2f',
  654: 'pkci93gMrogZuBj',
  655: 'rGa61gmuvPhdLxV',
  656: 'sjndEbLyjtCKgGv',
  657: 'uHbuxQJl8lW7ozc',
  658: 'wCURE6d3bPkepu2',
  659: 'z9Y9gH1T5YWrNuG',
  660: 'zBIq5rxdHJRwDNY',
  661: 'zVJPv84UY57bAof',
  662: '1BY6WEcLGh8j5v7',
  663: '1IF53ai7Xc0U56Y',
  664: '3PZuoBAID5Wc2HD',
  665: '7JYdWkrLSPkdwr4',
  666: 'McdE0feGgRqW7Ca',
  667: 'VHMLBKoKgIrUVDU',
  668: 'WRmjgnKYAwPKWDb',
  669: 'YxYtjZvoooNbGkE',
  670: 'ZnxXDlPa8U1GXgE',
  671: 'ZoEaEvLYb1n2sOq',
  672: 'adLQvlD726eNBSB',
  673: 'bvBOhCH3iADSZry',
  674: 'iCRJl6heRkivqQ3',
  675: 'ih0vzX44oOqAx2f',
  676: 'pkci93gMrogZuBj',
  677: 'rGa61gmuvPhdLxV',
  678: 'sjndEbLyjtCKgGv',
  679: 'uHbuxQJl8lW7ozc',
  680: 'wCURE6d3bPkepu2',
  681: 'z9Y9gH1T5YWrNuG',
  682: 'zBIq5rxdHJRwDNY',
  683: 'zVJPv84UY57bAof',
  684: '1BY6WEcLGh8j5v7',
  685: '1IF53ai7Xc0U56Y',
  686: '3PZuoBAID5Wc2HD',
  687: '7JYdWkrLSPkdwr4',
  688: 'McdE0feGgRqW7Ca',
  689: 'VHMLBKoKgIrUVDU',
  690: 'WRmjgnKYAwPKWDb',
  691: 'YxYtjZvoooNbGkE',
  692: 'ZnxXDlPa8U1GXgE',
  693: 'ZoEaEvLYb1n2sOq',
  694: 'adLQvlD726eNBSB',
  695: 'bvBOhCH3iADSZry',
  696: 'iCRJl6heRkivqQ3',
  697: 'ih0vzX44oOqAx2f',
  698: 'pkci93gMrogZuBj',
  699: 'rGa61gmuvPhdLxV',
  700: 'sjndEbLyjtCKgGv',
  701: 'uHbuxQJl8lW7ozc',
  702: 'wCURE6d3bPkepu2',
  703: 'z9Y9gH1T5YWrNuG',
  704: 'zBIq5rxdHJRwDNY',
  705: 'zVJPv84UY57bAof',
  706: '1BY6WEcLGh8j5v7',
  707: '1IF53ai7Xc0U56Y',
  708: '3PZuoBAID5Wc2HD',
  709: '7JYdWkrLSPkdwr4',
  710: 'McdE0feGgRqW7Ca',
  711: 'VHMLBKoKgIrUVDU',
  712: 'WRmjgnKYAwPKWDb',
  713: 'YxYtjZvoooNbGkE',
  714: 'ZnxXDlPa8U1GXgE',
  715: 'ZoEaEvLYb1n2sOq',
  716: 'adLQvlD726eNBSB',
  717: 'bvBOhCH3iADSZry',
  718: 'iCRJl6heRkivqQ3',
  719: 'ih0vzX44oOqAx2f',
  720: 'pkci93gMrogZuBj',
  721: 'rGa61gmuvPhdLxV',
  722: 'sjndEbLyjtCKgGv',
  723: 'uHbuxQJl8lW7ozc',
  724: 'wCURE6d3bPkepu2',
  725: 'z9Y9gH1T5YWrNuG',
  726: 'zBIq5rxdHJRwDNY',
  727: 'zVJPv84UY57bAof',
  728: '1BY6WEcLGh8j5v7',
  729: '1IF53ai7Xc0U56Y',
  730: '3PZuoBAID5Wc2HD',
  731: '7JYdWkrLSPkdwr4',
  732: 'McdE0feGgRqW7Ca',
  733: 'VHMLBKoKgIrUVDU',
  734: 'WRmjgnKYAwPKWDb',
  735: 'YxYtjZvoooNbGkE',
  736: 'ZnxXDlPa8U1GXgE',
  737: 'ZoEaEvLYb1n2sOq',
  738: 'adLQvlD726eNBSB',
  739: 'bvBOhCH3iADSZry',
  740: 'iCRJl6heRkivqQ3',
  741: 'ih0vzX44oOqAx2f',
  742: 'pkci93gMrogZuBj',
  743: 'rGa61gmuvPhdLxV',
  744: 'sjndEbLyjtCKgGv',
  745: 'uHbuxQJl8lW7ozc',
  746: 'wCURE6d3bPkepu2',
  747: 'z9Y9gH1T5YWrNuG',
  748: 'zBIq5rxdHJRwDNY',
  749: 'zVJPv84UY57bAof',
  750: '1BY6WEcLGh8j5v7',
  751: '1IF53ai7Xc0U56Y',
  752: '3PZuoBAID5Wc2HD',
  753: '7JYdWkrLSPkdwr4',
  754: 'McdE0feGgRqW7Ca',
  755: 'VHMLBKoKgIrUVDU',
  756: 'WRmjgnKYAwPKWDb',
  757: 'YxYtjZvoooNbGkE',
  758: 'ZnxXDlPa8U1GXgE',
  759: 'ZoEaEvLYb1n2sOq',
  760: 'adLQvlD726eNBSB',
  761: 'bvBOhCH3iADSZry',
  762: 'iCRJl6heRkivqQ3',
  763: 'ih0vzX44oOqAx2f',
  764: 'pkci93gMrogZuBj',
  765: 'rGa61gmuvPhdLxV',
  766: 'sjndEbLyjtCKgGv',
  767: 'uHbuxQJl8lW7ozc',
  768: 'wCURE6d3bPkepu2',
  769: 'z9Y9gH1T5YWrNuG',
  770: 'zBIq5rxdHJRwDNY',
  771: 'zVJPv84UY57bAof',
  772: '1BY6WEcLGh8j5v7',
  773: '1IF53ai7Xc0U56Y',
  774: '3PZuoBAID5Wc2HD',
  775: '7JYdWkrLSPkdwr4',
  776: 'McdE0feGgRqW7Ca',
  777: 'VHMLBKoKgIrUVDU',
  778: 'WRmjgnKYAwPKWDb',
  779: 'YxYtjZvoooNbGkE',
  780: 'ZnxXDlPa8U1GXgE',
  781: 'ZoEaEvLYb1n2sOq',
  782: 'adLQvlD726eNBSB',
  783: 'bvBOhCH3iADSZry',
  784: 'iCRJl6heRkivqQ3',
  785: 'ih0vzX44oOqAx2f',
  786: 'pkci93gMrogZuBj',
  787: 'rGa61gmuvPhdLxV',
  788: 'sjndEbLyjtCKgGv',
  789: 'uHbuxQJl8lW7ozc',
  790: 'wCURE6d3bPkepu2',
  791: 'z9Y9gH1T5YWrNuG',
  792: 'zBIq5rxdHJRwDNY',
  793: 'zVJPv84UY57bAof',
  794: '1BY6WEcLGh8j5v7',
  795: '1IF53ai7Xc0U56Y',
  796: '3PZuoBAID5Wc2HD',
  797: '7JYdWkrLSPkdwr4',
  798: 'McdE0feGgRqW7Ca',
  799: 'VHMLBKoKgIrUVDU',
  800: 'WRmjgnKYAwPKWDb',
  801: 'YxYtjZvoooNbGkE',
  802: 'ZnxXDlPa8U1GXgE',
  803: 'ZoEaEvLYb1n2sOq',
  804: 'adLQvlD726eNBSB',
  805: 'bvBOhCH3iADSZry',
  806: 'iCRJl6heRkivqQ3',
  807: 'ih0vzX44oOqAx2f',
  808: 'pkci93gMrogZuBj',
  809: 'rGa61gmuvPhdLxV',
  810: 'sjndEbLyjtCKgGv',
  811: 'uHbuxQJl8lW7ozc',
  812: 'wCURE6d3bPkepu2',
  813: 'z9Y9gH1T5YWrNuG',
  814: 'zBIq5rxdHJRwDNY',
  815: 'zVJPv84UY57bAof',
  816: '1BY6WEcLGh8j5v7',
  817: '1IF53ai7Xc0U56Y',
  818: '3PZuoBAID5Wc2HD',
  819: '7JYdWkrLSPkdwr4',
  820: 'McdE0feGgRqW7Ca',
  821: 'VHMLBKoKgIrUVDU',
  822: 'WRmjgnKYAwPKWDb',
  823: 'YxYtjZvoooNbGkE',
  824: 'ZnxXDlPa8U1GXgE',
  825: 'ZoEaEvLYb1n2sOq',
  826: 'adLQvlD726eNBSB',
  827: 'bvBOhCH3iADSZry',
  828: 'iCRJl6heRkivqQ3',
  829: 'ih0vzX44oOqAx2f',
  830: 'pkci93gMrogZuBj',
  831: 'rGa61gmuvPhdLxV',
  832: 'sjndEbLyjtCKgGv',
  833: 'uHbuxQJl8lW7ozc',
  834: 'wCURE6d3bPkepu2',
  835: 'z9Y9gH1T5YWrNuG',
  836: 'zBIq5rxdHJRwDNY',
  837: 'zVJPv84UY57bAof',
  838: '1BY6WEcLGh8j5v7',
  839: '1IF53ai7Xc0U56Y',
  840: '3PZuoBAID5Wc2HD',
  841: '7JYdWkrLSPkdwr4',
  842: 'McdE0feGgRqW7Ca',
  843: 'VHMLBKoKgIrUVDU',
  844: 'WRmjgnKYAwPKWDb',
  845: 'YxYtjZvoooNbGkE',
  846: 'ZnxXDlPa8U1GXgE',
  847: 'ZoEaEvLYb1n2sOq',
  848: 'adLQvlD726eNBSB',
  849: 'bvBOhCH3iADSZry',
  850: 'iCRJl6heRkivqQ3',
  851: 'ih0vzX44oOqAx2f',
  852: 'pkci93gMrogZuBj',
  853: 'rGa61gmuvPhdLxV',
  854: 'sjndEbLyjtCKgGv',
  855: 'uHbuxQJl8lW7ozc',
  856: 'wCURE6d3bPkepu2',
  857: 'z9Y9gH1T5YWrNuG',
  858: 'zBIq5rxdHJRwDNY',
  859: 'zVJPv84UY57bAof',
  860: '1BY6WEcLGh8j5v7',
  861: '1IF53ai7Xc0U56Y',
  862: '3PZuoBAID5Wc2HD',
  863: '7JYdWkrLSPkdwr4',
  864: 'McdE0feGgRqW7Ca',
  865: 'VHMLBKoKgIrUVDU',
  866: 'WRmjgnKYAwPKWDb',
  867: 'YxYtjZvoooNbGkE',
  868: 'ZnxXDlPa8U1GXgE',
  869: 'ZoEaEvLYb1n2sOq',
  870: 'adLQvlD726eNBSB',
  871: 'bvBOhCH3iADSZry',
  872: 'iCRJl6heRkivqQ3',
  873: 'ih0vzX44oOqAx2f',
  874: 'pkci93gMrogZuBj',
  875: 'rGa61gmuvPhdLxV',
  876: 'sjndEbLyjtCKgGv',
  877: 'uHbuxQJl8lW7ozc',
  878: 'wCURE6d3bPkepu2',
  879: 'z9Y9gH1T5YWrNuG',
  880: 'zBIq5rxdHJRwDNY',
  881: 'zVJPv84UY57bAof',
  882: '1BY6WEcLGh8j5v7',
  883: '1IF53ai7Xc0U56Y',
  884: '3PZuoBAID5Wc2HD',
  885: '7JYdWkrLSPkdwr4',
  886: 'McdE0feGgRqW7Ca',
  887: 'VHMLBKoKgIrUVDU',
  888: 'WRmjgnKYAwPKWDb',
  889: 'YxYtjZvoooNbGkE',
  890: 'ZnxXDlPa8U1GXgE',
  891: 'ZoEaEvLYb1n2sOq',
  892: 'adLQvlD726eNBSB',
  893: 'bvBOhCH3iADSZry',
  894: 'iCRJl6heRkivqQ3',
  895: 'ih0vzX44oOqAx2f',
  896: 'pkci93gMrogZuBj',
  897: 'rGa61gmuvPhdLxV',
  898: 'sjndEbLyjtCKgGv',
  899: 'uHbuxQJl8lW7ozc',
  900: 'wCURE6d3bPkepu2',
  901: 'z9Y9gH1T5YWrNuG',
  902: 'zBIq5rxdHJRwDNY',
  903: 'zVJPv84UY57bAof',
  904: '1BY6WEcLGh8j5v7',
  905: '1IF53ai7Xc0U56Y',
  906: '3PZuoBAID5Wc2HD',
  907: '7JYdWkrLSPkdwr4',
  908: 'McdE0feGgRqW7Ca',
  909: 'VHMLBKoKgIrUVDU',
  910: 'WRmjgnKYAwPKWDb',
  911: 'YxYtjZvoooNbGkE',
  912: 'ZnxXDlPa8U1GXgE',
  913: 'ZoEaEvLYb1n2sOq',
  914: 'adLQvlD726eNBSB',
  915: 'bvBOhCH3iADSZry',
  916: 'iCRJl6heRkivqQ3',
  917: 'ih0vzX44oOqAx2f',
  918: 'pkci93gMrogZuBj',
  919: 'rGa61gmuvPhdLxV',
  920: 'sjndEbLyjtCKgGv',
  921: 'uHbuxQJl8lW7ozc',
  922: 'wCURE6d3bPkepu2',
  923: 'z9Y9gH1T5YWrNuG',
  924: 'zBIq5rxdHJRwDNY',
  925: 'zVJPv84UY57bAof',
  926: '1BY6WEcLGh8j5v7',
  927: '1IF53ai7Xc0U56Y',
  928: '3PZuoBAID5Wc2HD',
  929: '7JYdWkrLSPkdwr4',
  930: 'McdE0feGgRqW7Ca',
  931: 'VHMLBKoKgIrUVDU',
  932: 'WRmjgnKYAwPKWDb',
  933: 'YxYtjZvoooNbGkE',
  934: 'ZnxXDlPa8U1GXgE',
  935: 'ZoEaEvLYb1n2sOq',
  936: 'adLQvlD726eNBSB',
  937: 'bvBOhCH3iADSZry',
  938: 'iCRJl6heRkivqQ3',
  939: 'ih0vzX44oOqAx2f',
  940: 'pkci93gMrogZuBj',
  941: 'rGa61gmuvPhdLxV',
  942: 'sjndEbLyjtCKgGv',
  943: 'uHbuxQJl8lW7ozc',
  944: 'wCURE6d3bPkepu2',
  945: 'z9Y9gH1T5YWrNuG',
  946: 'zBIq5rxdHJRwDNY',
  947: 'zVJPv84UY57bAof',
  948: '1BY6WEcLGh8j5v7',
  949: '1IF53ai7Xc0U56Y',
  950: '3PZuoBAID5Wc2HD',
  951: '7JYdWkrLSPkdwr4',
  952: 'McdE0feGgRqW7Ca',
  953: 'VHMLBKoKgIrUVDU',
  954: 'WRmjgnKYAwPKWDb',
  955: 'YxYtjZvoooNbGkE',
  956: 'ZnxXDlPa8U1GXgE',
  957: 'ZoEaEvLYb1n2sOq',
  958: 'adLQvlD726eNBSB',
  959: 'bvBOhCH3iADSZry',
  960: 'iCRJl6heRkivqQ3',
  961: 'ih0vzX44oOqAx2f',
  962: 'pkci93gMrogZuBj',
  963: 'rGa61gmuvPhdLxV',
  964: 'sjndEbLyjtCKgGv',
  965: 'uHbuxQJl8lW7ozc',
  966: 'wCURE6d3bPkepu2',
  967: 'z9Y9gH1T5YWrNuG',
  968: 'zBIq5rxdHJRwDNY',
  969: 'zVJPv84UY57bAof',
  970: '1BY6WEcLGh8j5v7',
  971: '1IF53ai7Xc0U56Y',
  972: '3PZuoBAID5Wc2HD',
  973: '7JYdWkrLSPkdwr4',
  974: 'McdE0feGgRqW7Ca',
  975: 'VHMLBKoKgIrUVDU',
  976: 'WRmjgnKYAwPKWDb',
  977: 'YxYtjZvoooNbGkE',
  978: 'ZnxXDlPa8U1GXgE',
  979: 'ZoEaEvLYb1n2sOq',
  980: 'adLQvlD726eNBSB',
  981: 'bvBOhCH3iADSZry',
  982: 'iCRJl6heRkivqQ3',
  983: 'ih0vzX44oOqAx2f',
  984: 'pkci93gMrogZuBj',
  985: 'rGa61gmuvPhdLxV',
  986: 'sjndEbLyjtCKgGv',
  987: 'uHbuxQJl8lW7ozc',
  988: 'wCURE6d3bPkepu2',
  989: 'z9Y9gH1T5YWrNuG',
  990: 'zBIq5rxdHJRwDNY',
  991: 'zVJPv84UY57bAof',
  992: '1BY6WEcLGh8j5v7',
  993: '1IF53ai7Xc0U56Y',
  994: '3PZuoBAID5Wc2HD',
  995: '7JYdWkrLSPkdwr4',
  996: 'McdE0feGgRqW7Ca',
  997: 'VHMLBKoKgIrUVDU',
  998: 'WRmjgnKYAwPKWDb',
  999: 'YxYtjZvoooNbGkE',
  ...},
 'DC_POWER': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: 0.0,
  20: 0.0,
  21: 0.0,
  22: 0.0,
  23: 0.0,
  24: 0.0,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: 0.0,
  35: 0.0,
  36: 0.0,
  37: 0.0,
  38: 0.0,
  39: 0.0,
  40: 0.0,
  41: 0.0,
  42: 0.0,
  43: 0.0,
  44: 0.0,
  45: 0.0,
  46: 0.0,
  47: 0.0,
  48: 0.0,
  49: 0.0,
  50: 0.0,
  51: 0.0,
  52: 0.0,
  53: 0.0,
  54: 0.0,
  55: 0.0,
  56: 0.0,
  57: 0.0,
  58: 0.0,
  59: 0.0,
  60: 0.0,
  61: 0.0,
  62: 0.0,
  63: 0.0,
  64: 0.0,
  65: 0.0,
  66: 0.0,
  67: 0.0,
  68: 0.0,
  69: 0.0,
  70: 0.0,
  71: 0.0,
  72: 0.0,
  73: 0.0,
  74: 0.0,
  75: 0.0,
  76: 0.0,
  77: 0.0,
  78: 0.0,
  79: 0.0,
  80: 0.0,
  81: 0.0,
  82: 0.0,
  83: 0.0,
  84: 0.0,
  85: 0.0,
  86: 0.0,
  87: 0.0,
  88: 0.0,
  89: 0.0,
  90: 0.0,
  91: 0.0,
  92: 0.0,
  93: 0.0,
  94: 0.0,
  95: 0.0,
  96: 0.0,
  97: 0.0,
  98: 0.0,
  99: 0.0,
  100: 0.0,
  101: 0.0,
  102: 0.0,
  103: 0.0,
  104: 0.0,
  105: 0.0,
  106: 0.0,
  107: 0.0,
  108: 0.0,
  109: 0.0,
  110: 0.0,
  111: 0.0,
  112: 0.0,
  113: 0.0,
  114: 0.0,
  115: 0.0,
  116: 0.0,
  117: 0.0,
  118: 0.0,
  119: 0.0,
  120: 0.0,
  121: 0.0,
  122: 0.0,
  123: 0.0,
  124: 0.0,
  125: 0.0,
  126: 0.0,
  127: 0.0,
  128: 0.0,
  129: 0.0,
  130: 0.0,
  131: 0.0,
  132: 0.0,
  133: 0.0,
  134: 0.0,
  135: 0.0,
  136: 0.0,
  137: 0.0,
  138: 0.0,
  139: 0.0,
  140: 0.0,
  141: 0.0,
  142: 0.0,
  143: 0.0,
  144: 0.0,
  145: 0.0,
  146: 0.0,
  147: 0.0,
  148: 0.0,
  149: 0.0,
  150: 0.0,
  151: 0.0,
  152: 0.0,
  153: 0.0,
  154: 0.0,
  155: 0.0,
  156: 0.0,
  157: 0.0,
  158: 0.0,
  159: 0.0,
  160: 0.0,
  161: 0.0,
  162: 0.0,
  163: 0.0,
  164: 0.0,
  165: 0.0,
  166: 0.0,
  167: 0.0,
  168: 0.0,
  169: 0.0,
  170: 0.0,
  171: 0.0,
  172: 0.0,
  173: 0.0,
  174: 0.0,
  175: 0.0,
  176: 0.0,
  177: 0.0,
  178: 0.0,
  179: 0.0,
  180: 0.0,
  181: 0.0,
  182: 0.0,
  183: 0.0,
  184: 0.0,
  185: 0.0,
  186: 0.0,
  187: 0.0,
  188: 0.0,
  189: 0.0,
  190: 0.0,
  191: 0.0,
  192: 0.0,
  193: 0.0,
  194: 0.0,
  195: 0.0,
  196: 0.0,
  197: 0.0,
  198: 0.0,
  199: 0.0,
  200: 0.0,
  201: 0.0,
  202: 0.0,
  203: 0.0,
  204: 0.0,
  205: 0.0,
  206: 0.0,
  207: 0.0,
  208: 0.0,
  209: 0.0,
  210: 0.0,
  211: 0.0,
  212: 0.0,
  213: 0.0,
  214: 0.0,
  215: 0.0,
  216: 0.0,
  217: 0.0,
  218: 0.0,
  219: 0.0,
  220: 0.0,
  221: 0.0,
  222: 0.0,
  223: 0.0,
  224: 0.0,
  225: 0.0,
  226: 0.0,
  227: 0.0,
  228: 0.0,
  229: 0.0,
  230: 0.0,
  231: 0.0,
  232: 0.0,
  233: 0.0,
  234: 0.0,
  235: 0.0,
  236: 0.0,
  237: 0.0,
  238: 0.0,
  239: 0.0,
  240: 0.0,
  241: 0.0,
  242: 0.0,
  243: 0.0,
  244: 0.0,
  245: 0.0,
  246: 0.0,
  247: 0.0,
  248: 0.0,
  249: 0.0,
  250: 0.0,
  251: 0.0,
  252: 0.0,
  253: 0.0,
  254: 0.0,
  255: 0.0,
  256: 0.0,
  257: 0.0,
  258: 0.0,
  259: 0.0,
  260: 0.0,
  261: 0.0,
  262: 0.0,
  263: 0.0,
  264: 0.0,
  265: 0.0,
  266: 0.0,
  267: 0.0,
  268: 0.0,
  269: 0.0,
  270: 0.0,
  271: 0.0,
  272: 0.0,
  273: 0.0,
  274: 0.0,
  275: 0.0,
  276: 0.0,
  277: 0.0,
  278: 0.0,
  279: 0.0,
  280: 0.0,
  281: 0.0,
  282: 0.0,
  283: 0.0,
  284: 0.0,
  285: 0.0,
  286: 0.0,
  287: 0.0,
  288: 0.0,
  289: 0.0,
  290: 0.0,
  291: 0.0,
  292: 0.0,
  293: 0.0,
  294: 0.0,
  295: 0.0,
  296: 0.0,
  297: 0.0,
  298: 0.0,
  299: 0.0,
  300: 0.0,
  301: 0.0,
  302: 0.0,
  303: 0.0,
  304: 0.0,
  305: 0.0,
  306: 0.0,
  307: 0.0,
  308: 0.0,
  309: 0.0,
  310: 0.0,
  311: 0.0,
  312: 0.0,
  313: 0.0,
  314: 0.0,
  315: 0.0,
  316: 0.0,
  317: 0.0,
  318: 0.0,
  319: 0.0,
  320: 0.0,
  321: 0.0,
  322: 0.0,
  323: 0.0,
  324: 0.0,
  325: 0.0,
  326: 0.0,
  327: 0.0,
  328: 0.0,
  329: 0.0,
  330: 0.0,
  331: 0.0,
  332: 0.0,
  333: 0.0,
  334: 0.0,
  335: 0.0,
  336: 0.0,
  337: 0.0,
  338: 0.0,
  339: 0.0,
  340: 0.0,
  341: 0.0,
  342: 0.0,
  343: 0.0,
  344: 0.0,
  345: 0.0,
  346: 0.0,
  347: 0.0,
  348: 0.0,
  349: 0.0,
  350: 0.0,
  351: 0.0,
  352: 0.0,
  353: 0.0,
  354: 0.0,
  355: 0.0,
  356: 0.0,
  357: 0.0,
  358: 0.0,
  359: 0.0,
  360: 0.0,
  361: 0.0,
  362: 0.0,
  363: 0.0,
  364: 0.0,
  365: 0.0,
  366: 0.0,
  367: 0.0,
  368: 0.0,
  369: 0.0,
  370: 0.0,
  371: 0.0,
  372: 0.0,
  373: 0.0,
  374: 0.0,
  375: 0.0,
  376: 0.0,
  377: 0.0,
  378: 0.0,
  379: 0.0,
  380: 0.0,
  381: 0.0,
  382: 0.0,
  383: 0.0,
  384: 0.0,
  385: 0.0,
  386: 0.0,
  387: 0.0,
  388: 0.0,
  389: 0.0,
  390: 0.0,
  391: 0.0,
  392: 0.0,
  393: 0.0,
  394: 0.0,
  395: 0.0,
  396: 0.0,
  397: 0.0,
  398: 0.0,
  399: 0.0,
  400: 0.0,
  401: 0.0,
  402: 0.0,
  403: 0.0,
  404: 0.0,
  405: 0.0,
  406: 0.0,
  407: 0.0,
  408: 0.0,
  409: 0.0,
  410: 0.0,
  411: 0.0,
  412: 0.0,
  413: 0.0,
  414: 0.0,
  415: 0.0,
  416: 0.0,
  417: 0.0,
  418: 0.0,
  419: 0.0,
  420: 0.0,
  421: 0.0,
  422: 0.0,
  423: 0.0,
  424: 0.0,
  425: 0.0,
  426: 0.0,
  427: 0.0,
  428: 0.0,
  429: 0.0,
  430: 0.0,
  431: 0.0,
  432: 0.0,
  433: 0.0,
  434: 0.0,
  435: 0.0,
  436: 0.0,
  437: 0.0,
  438: 0.0,
  439: 0.0,
  440: 0.0,
  441: 0.0,
  442: 0.0,
  443: 0.0,
  444: 0.0,
  445: 0.0,
  446: 0.0,
  447: 0.0,
  448: 0.0,
  449: 0.0,
  450: 0.0,
  451: 0.0,
  452: 0.0,
  453: 0.0,
  454: 0.0,
  455: 0.0,
  456: 0.0,
  457: 0.0,
  458: 0.0,
  459: 0.0,
  460: 0.0,
  461: 0.0,
  462: 0.0,
  463: 0.0,
  464: 0.0,
  465: 0.0,
  466: 0.0,
  467: 0.0,
  468: 0.0,
  469: 0.0,
  470: 0.0,
  471: 0.0,
  472: 0.0,
  473: 0.0,
  474: 0.0,
  475: 0.0,
  476: 0.0,
  477: 0.0,
  478: 0.0,
  479: 0.0,
  480: 0.0,
  481: 0.0,
  482: 0.0,
  483: 0.0,
  484: 0.0,
  485: 0.0,
  486: 0.0,
  487: 0.0,
  488: 0.0,
  489: 0.0,
  490: 0.0,
  491: 0.0,
  492: 0.0,
  493: 0.0,
  494: 0.0,
  495: 0.0,
  496: 0.0,
  497: 0.0,
  498: 0.0,
  499: 0.0,
  500: 0.0,
  501: 0.0,
  502: 0.0,
  503: 0.0,
  504: 0.0,
  505: 0.0,
  506: 0.0,
  507: 0.0,
  508: 0.0,
  509: 0.0,
  510: 37.14285714,
  511: 53.5,
  512: 58.0,
  513: 58.42857143,
  514: 54.375,
  515: 0.0,
  516: 43.85714286,
  517: 58.57142857,
  518: 24.5,
  519: 56.125,
  520: 37.0,
  521: 41.85714286,
  522: 50.16666667,
  523: 37.375,
  524: 38.875,
  525: 52.625,
  526: 38.625,
  527: 54.625,
  528: 54.625,
  529: 57.0,
  530: 34.875,
  531: 278.0,
  532: 310.5714286,
  533: 318.625,
  534: 316.25,
  535: 311.4285714,
  536: 61.125,
  537: 315.0,
  538: 303.4285714,
  539: 296.3333333,
  540: 315.1428571,
  541: 279.625,
  542: 298.7142857,
  543: 300.0,
  544: 307.1428571,
  545: 315.4285714,
  546: 303.0,
  547: 311.5714286,
  548: 309.0,
  549: 312.7142857,
  550: 301.2857143,
  551: 277.7142857,
  552: 626.5714286,
  553: 713.5,
  554: 720.0,
  555: 704.2857143,
  556: 692.75,
  557: 716.1428571,
  558: 712.1428571,
  559: 873.0,
  560: 697.75,
  561: 674.3333333,
  562: 719.25,
  563: 627.4285714,
  564: 677.0,
  565: 680.5,
  566: 687.0,
  567: 702.875,
  568: 683.25,
  569: 696.125,
  570: 682.25,
  571: 688.875,
  572: 681.25,
  573: 614.875,
  574: 1224.75,
  575: 1334.285714,
  576: 1306.714286,
  577: 1338.125,
  578: 1331.857143,
  579: 1373.875,
  580: 1298.857143,
  581: 1292.125,
  582: 1380.571429,
  583: 1290.0,
  584: 1348.142857,
  585: 1210.0,
  586: 1271.428571,
  587: 1400.0,
  588: 1264.142857,
  589: 1293.571429,
  590: 1296.714286,
  591: 1289.285714,
  592: 1317.714286,
  593: 1332.857143,
  594: 1264.714286,
  595: 1166.857143,
  596: 1734.571429,
  597: 1990.375,
  598: 1963.375,
  599: 1904.428571,
  600: 1920.25,
  601: 1942.142857,
  602: 1937.375,
  603: 1862.571429,
  604: 1904.571429,
  605: 1870.857143,
  606: 2009.625,
  607: 1739.571429,
  608: 1875.25,
  609: 1881.2,
  610: 1861.625,
  611: 1865.375,
  612: 1877.125,
  613: 1895.75,
  614: 1873.375,
  615: 1897.125,
  616: 1877.875,
  617: 1661.5,
  618: 1856.375,
  619: 2114.285714,
  620: 2090.285714,
  621: 2035.428571,
  622: 2098.428571,
  623: 2088.714286,
  624: 2024.285714,
  625: 2022.0,
  626: 2076.285714,
  627: 1995.5,
  628: 2143.714286,
  629: 1873.5,
  630: 2036.428571,
  631: 2034.833333,
  632: 2004.428571,
  633: 2012.428571,
  634: 2046.428571,
  635: 2062.0,
  636: 2045.857143,
  637: 2072.857143,
  638: 2015.285714,
  639: 1814.857143,
  640: 1842.285714,
  641: 2078.0,
  642: 2071.25,
  643: 2002.0,
  644: 2023.375,
  645: 2069.125,
  646: 2052.5,
  647: 1970.285714,
  648: 2036.285714,
  649: 1952.428571,
  650: 2109.875,
  651: 1865.142857,
  652: 1974.0,
  653: 1974.666667,
  654: 1958.25,
  655: 1951.0,
  656: 1969.25,
  657: 1998.75,
  658: 1987.714286,
  659: 2025.857143,
  660: 1967.125,
  661: 1740.125,
  662: 1877.875,
  663: 2142.714286,
  664: 2122.857143,
  665: 2048.142857,
  666: 2121.0,
  667: 2099.428571,
  668: 2072.0,
  669: 2041.375,
  670: 2067.285714,
  671: 2016.75,
  672: 2176.857143,
  673: 1890.125,
  674: 2000.142857,
  675: 2026.857143,
  676: 1897.142857,
  677: 1961.428571,
  678: 2050.571429,
  679: 1902.857143,
  680: 2043.5,
  681: 2060.125,
  682: 1993.714286,
  683: 1783.142857,
  684: 3246.0,
  685: 2805.625,
  686: 2736.125,
  687: 2741.5,
  688: 3208.875,
  689: 2879.625,
  690: 2779.5,
  691: 2891.714286,
  692: 3132.625,
  693: 2785.285714,
  694: 2816.25,
  695: 2921.571429,
  696: 2916.375,
  697: 3172.142857,
  698: 2989.375,
  699: 2833.714286,
  700: 2996.875,
  701: 3193.0,
  702: 3142.857143,
  703: 3180.142857,
  704: 2844.625,
  705: 2621.0,
  706: 3917.5,
  707: 4560.428571,
  708: 4523.714286,
  709: 4146.285714,
  710: 5073.0,
  711: 4215.285714,
  712: 4968.0,
  713: 4347.25,
  714: 4023.428571,
  715: 4332.875,
  716: 4617.142857,
  717: 3702.75,
  718: 5294.0,
  719: 4205.8,
  720: 5152.571429,
  721: 5105.625,
  722: 5169.0,
  723: 5404.714286,
  724: 4648.125,
  725: 5082.875,
  726: 4961.285714,
  727: 4485.5,
  728: 4322.0,
  729: 4983.625,
  730: 4991.5,
  731: 4695.0,
  732: 5605.625,
  733: 4783.75,
  734: 5060.25,
  735: 5405.285714,
  736: 5053.285714,
  737: 5234.428571,
  738: 4948.375,
  739: 4529.428571,
  740: 5338.875,
  741: 4959.333333,
  742: 5177.625,
  743: 5408.428571,
  744: 5472.5,
  745: 5063.125,
  746: 5436.428571,
  747: 5280.285714,
  748: 5279.125,
  749: 5101.285714,
  750: 4257.125,
  751: 5141.857143,
  752: 5567.428571,
  753: 4845.428571,
  754: 3473.142857,
  755: 4865.142857,
  756: 5739.714286,
  757: 4101.5,
  758: 3797.571429,
  759: 4670.625,
  760: 5279.428571,
  761: 4415.625,
  762: 3607.285714,
  763: 3442.571429,
  764: 4021.714286,
  765: 4131.5,
  766: 3493.857143,
  767: 3478.142857,
  768: 3475.5,
  769: 3504.125,
  770: 4217.428571,
  771: 2978.75,
  772: 5706.714286,
  773: 8229.25,
  774: 7681.875,
  775: 8048.0,
  776: 7378.428571,
  777: 7936.25,
  778: 6262.375,
  779: 8235.857143,
  780: 8110.0,
  781: 8211.142857,
  782: 8274.875,
  783: 6829.428571,
  784: 7948.0,
  785: 7889.571429,
  786: 8200.0,
  787: 8391.0,
  788: 7380.571429,
  789: 6981.875,
  790: 7251.714286,
  791: 7161.857143,
  792: 8404.25,
  793: 5085.857143,
  794: 4015.5,
  795: 3541.0,
  796: 3170.142857,
  797: 3592.285714,
  798: 5938.0,
  799: 3690.428571,
  800: 2986.0,
  801: 4983.714286,
  802: 4756.0,
  803: 4134.857143,
  804: 3395.714286,
  805: 3834.25,
  806: 5278.857143,
  807: 5475.571429,
  808: 6005.428571,
  809: 5290.125,
  810: 5795.0,
  811: 7028.0,
  812: 5680.0,
  813: 5932.0,
  814: 4303.142857,
  815: 0.0,
  816: 3219.285714,
  817: 4325.5,
  818: 3819.375,
  819: 3608.625,
  820: 3564.571429,
  821: 3707.25,
  822: 3864.625,
  823: 3504.125,
  824: 3475.285714,
  825: 3568.625,
  826: 3996.875,
  827: 3299.428571,
  828: 3721.375,
  829: 3395.857143,
  830: 3706.142857,
  831: 3719.142857,
  832: 3561.571429,
  833: 3776.285714,
  834: 3443.0,
  835: 3503.428571,
  836: 3741.875,
  837: 0.0,
  838: 6454.428571,
  839: 7680.714286,
  840: 7804.0,
  841: 7452.285714,
  842: 7631.625,
  843: 7630.571429,
  844: 7901.285714,
  845: 7631.857143,
  846: 7184.714286,
  847: 7462.714286,
  848: 8003.857143,
  849: 6281.0,
  850: 8049.0,
  851: 7024.5,
  852: 7194.75,
  853: 7639.5,
  854: 7483.375,
  855: 6500.0,
  856: 7844.0,
  857: 7065.5,
  858: 8163.857143,
  859: 5286.5,
  860: 6008.375,
  861: 8567.0,
  862: 7384.625,
  863: 7301.375,
  864: 10005.42857,
  865: 6739.5,
  866: 6619.25,
  867: 8841.375,
  868: 6873.0,
  869: 8507.125,
  870: 8481.142857,
  871: 5785.125,
  872: 9506.714286,
  873: 7627.75,
  874: 9441.428571,
  875: 9031.857143,
  876: 9505.285714,
  877: 9905.857143,
  878: 8500.142857,
  879: 9893.285714,
  880: 8889.142857,
  881: 9611.428571,
  882: 6637.285714,
  883: 6741.25,
  884: 5928.857143,
  885: 6788.857143,
  886: 7233.25,
  887: 7024.857143,
  888: 6390.0,
  889: 8140.0,
  890: 7869.857143,
  891: 7596.428571,
  892: 6276.875,
  893: 6173.285714,
  894: 8047.5,
  895: 8989.0,
  896: 6130.125,
  897: 7055.5,
  898: 8947.125,
  899: 7079.0,
  900: 7232.75,
  901: 6152.5,
  902: 7940.0,
  903: 8342.25,
  904: 4399.75,
  905: 5592.714286,
  906: 5319.857143,
  907: 4893.0,
  908: 6013.285714,
  909: 4944.0,
  910: 5174.285714,
  911: 5977.25,
  912: 4820.428571,
  913: 6197.875,
  914: 5257.571429,
  915: 4439.875,
  916: 7315.714286,
  917: 5035.6,
  918: 7577.428571,
  919: 7707.571429,
  920: 6573.285714,
  921: 6439.0,
  922: 5780.857143,
  923: 6128.714286,
  924: 7030.857143,
  925: 6484.714286,
  926: 4360.571429,
  927: 4895.125,
  928: 4999.75,
  929: 4605.428571,
  930: 5362.5,
  931: 4794.714286,
  932: 4875.0,
  933: 5081.142857,
  934: 4868.142857,
  935: 4825.714286,
  936: 4962.0,
  937: 4342.428571,
  938: 5251.375,
  939: 4621.0,
  940: 6211.875,
  941: 5240.375,
  942: 4994.625,
  943: 6817.5,
  944: 5415.125,
  945: 6141.75,
  946: 5068.625,
  947: 5388.375,
  948: 6829.5,
  949: 8230.428571,
  950: 7337.428571,
  951: 7736.571429,
  952: 10257.85714,
  953: 7767.0,
  954: 7080.428571,
  955: 8608.75,
  956: 8656.428571,
  957: 7970.875,
  958: 7679.0,
  959: 7113.625,
  960: 10353.0,
  961: 8766.0,
  962: 10114.57143,
  963: 10648.85714,
  964: 9636.285714,
  965: 10481.14286,
  966: 8641.428571,
  967: 9592.571429,
  968: 9982.0,
  969: 10284.28571,
  970: 5969.0,
  971: 6082.375,
  972: 5851.75,
  973: 6103.25,
  974: 8710.25,
  975: 6141.75,
  976: 5567.375,
  977: 7360.714286,
  978: 6437.857143,
  979: 6429.142857,
  980: 5966.875,
  981: 5902.428571,
  982: 8474.625,
  983: 6663.333333,
  984: 7711.375,
  985: 8062.625,
  986: 7676.25,
  987: 9303.75,
  988: 8375.375,
  989: 10653.625,
  990: 8240.0,
  991: 8617.5,
  992: 6226.125,
  993: 7405.285714,
  994: 8226.857143,
  995: 7328.0,
  996: 6471.285714,
  997: 7761.142857,
  998: 8600.571429,
  999: 6340.5,
  ...},
 'AC_POWER': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: 0.0,
  20: 0.0,
  21: 0.0,
  22: 0.0,
  23: 0.0,
  24: 0.0,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: 0.0,
  35: 0.0,
  36: 0.0,
  37: 0.0,
  38: 0.0,
  39: 0.0,
  40: 0.0,
  41: 0.0,
  42: 0.0,
  43: 0.0,
  44: 0.0,
  45: 0.0,
  46: 0.0,
  47: 0.0,
  48: 0.0,
  49: 0.0,
  50: 0.0,
  51: 0.0,
  52: 0.0,
  53: 0.0,
  54: 0.0,
  55: 0.0,
  56: 0.0,
  57: 0.0,
  58: 0.0,
  59: 0.0,
  60: 0.0,
  61: 0.0,
  62: 0.0,
  63: 0.0,
  64: 0.0,
  65: 0.0,
  66: 0.0,
  67: 0.0,
  68: 0.0,
  69: 0.0,
  70: 0.0,
  71: 0.0,
  72: 0.0,
  73: 0.0,
  74: 0.0,
  75: 0.0,
  76: 0.0,
  77: 0.0,
  78: 0.0,
  79: 0.0,
  80: 0.0,
  81: 0.0,
  82: 0.0,
  83: 0.0,
  84: 0.0,
  85: 0.0,
  86: 0.0,
  87: 0.0,
  88: 0.0,
  89: 0.0,
  90: 0.0,
  91: 0.0,
  92: 0.0,
  93: 0.0,
  94: 0.0,
  95: 0.0,
  96: 0.0,
  97: 0.0,
  98: 0.0,
  99: 0.0,
  100: 0.0,
  101: 0.0,
  102: 0.0,
  103: 0.0,
  104: 0.0,
  105: 0.0,
  106: 0.0,
  107: 0.0,
  108: 0.0,
  109: 0.0,
  110: 0.0,
  111: 0.0,
  112: 0.0,
  113: 0.0,
  114: 0.0,
  115: 0.0,
  116: 0.0,
  117: 0.0,
  118: 0.0,
  119: 0.0,
  120: 0.0,
  121: 0.0,
  122: 0.0,
  123: 0.0,
  124: 0.0,
  125: 0.0,
  126: 0.0,
  127: 0.0,
  128: 0.0,
  129: 0.0,
  130: 0.0,
  131: 0.0,
  132: 0.0,
  133: 0.0,
  134: 0.0,
  135: 0.0,
  136: 0.0,
  137: 0.0,
  138: 0.0,
  139: 0.0,
  140: 0.0,
  141: 0.0,
  142: 0.0,
  143: 0.0,
  144: 0.0,
  145: 0.0,
  146: 0.0,
  147: 0.0,
  148: 0.0,
  149: 0.0,
  150: 0.0,
  151: 0.0,
  152: 0.0,
  153: 0.0,
  154: 0.0,
  155: 0.0,
  156: 0.0,
  157: 0.0,
  158: 0.0,
  159: 0.0,
  160: 0.0,
  161: 0.0,
  162: 0.0,
  163: 0.0,
  164: 0.0,
  165: 0.0,
  166: 0.0,
  167: 0.0,
  168: 0.0,
  169: 0.0,
  170: 0.0,
  171: 0.0,
  172: 0.0,
  173: 0.0,
  174: 0.0,
  175: 0.0,
  176: 0.0,
  177: 0.0,
  178: 0.0,
  179: 0.0,
  180: 0.0,
  181: 0.0,
  182: 0.0,
  183: 0.0,
  184: 0.0,
  185: 0.0,
  186: 0.0,
  187: 0.0,
  188: 0.0,
  189: 0.0,
  190: 0.0,
  191: 0.0,
  192: 0.0,
  193: 0.0,
  194: 0.0,
  195: 0.0,
  196: 0.0,
  197: 0.0,
  198: 0.0,
  199: 0.0,
  200: 0.0,
  201: 0.0,
  202: 0.0,
  203: 0.0,
  204: 0.0,
  205: 0.0,
  206: 0.0,
  207: 0.0,
  208: 0.0,
  209: 0.0,
  210: 0.0,
  211: 0.0,
  212: 0.0,
  213: 0.0,
  214: 0.0,
  215: 0.0,
  216: 0.0,
  217: 0.0,
  218: 0.0,
  219: 0.0,
  220: 0.0,
  221: 0.0,
  222: 0.0,
  223: 0.0,
  224: 0.0,
  225: 0.0,
  226: 0.0,
  227: 0.0,
  228: 0.0,
  229: 0.0,
  230: 0.0,
  231: 0.0,
  232: 0.0,
  233: 0.0,
  234: 0.0,
  235: 0.0,
  236: 0.0,
  237: 0.0,
  238: 0.0,
  239: 0.0,
  240: 0.0,
  241: 0.0,
  242: 0.0,
  243: 0.0,
  244: 0.0,
  245: 0.0,
  246: 0.0,
  247: 0.0,
  248: 0.0,
  249: 0.0,
  250: 0.0,
  251: 0.0,
  252: 0.0,
  253: 0.0,
  254: 0.0,
  255: 0.0,
  256: 0.0,
  257: 0.0,
  258: 0.0,
  259: 0.0,
  260: 0.0,
  261: 0.0,
  262: 0.0,
  263: 0.0,
  264: 0.0,
  265: 0.0,
  266: 0.0,
  267: 0.0,
  268: 0.0,
  269: 0.0,
  270: 0.0,
  271: 0.0,
  272: 0.0,
  273: 0.0,
  274: 0.0,
  275: 0.0,
  276: 0.0,
  277: 0.0,
  278: 0.0,
  279: 0.0,
  280: 0.0,
  281: 0.0,
  282: 0.0,
  283: 0.0,
  284: 0.0,
  285: 0.0,
  286: 0.0,
  287: 0.0,
  288: 0.0,
  289: 0.0,
  290: 0.0,
  291: 0.0,
  292: 0.0,
  293: 0.0,
  294: 0.0,
  295: 0.0,
  296: 0.0,
  297: 0.0,
  298: 0.0,
  299: 0.0,
  300: 0.0,
  301: 0.0,
  302: 0.0,
  303: 0.0,
  304: 0.0,
  305: 0.0,
  306: 0.0,
  307: 0.0,
  308: 0.0,
  309: 0.0,
  310: 0.0,
  311: 0.0,
  312: 0.0,
  313: 0.0,
  314: 0.0,
  315: 0.0,
  316: 0.0,
  317: 0.0,
  318: 0.0,
  319: 0.0,
  320: 0.0,
  321: 0.0,
  322: 0.0,
  323: 0.0,
  324: 0.0,
  325: 0.0,
  326: 0.0,
  327: 0.0,
  328: 0.0,
  329: 0.0,
  330: 0.0,
  331: 0.0,
  332: 0.0,
  333: 0.0,
  334: 0.0,
  335: 0.0,
  336: 0.0,
  337: 0.0,
  338: 0.0,
  339: 0.0,
  340: 0.0,
  341: 0.0,
  342: 0.0,
  343: 0.0,
  344: 0.0,
  345: 0.0,
  346: 0.0,
  347: 0.0,
  348: 0.0,
  349: 0.0,
  350: 0.0,
  351: 0.0,
  352: 0.0,
  353: 0.0,
  354: 0.0,
  355: 0.0,
  356: 0.0,
  357: 0.0,
  358: 0.0,
  359: 0.0,
  360: 0.0,
  361: 0.0,
  362: 0.0,
  363: 0.0,
  364: 0.0,
  365: 0.0,
  366: 0.0,
  367: 0.0,
  368: 0.0,
  369: 0.0,
  370: 0.0,
  371: 0.0,
  372: 0.0,
  373: 0.0,
  374: 0.0,
  375: 0.0,
  376: 0.0,
  377: 0.0,
  378: 0.0,
  379: 0.0,
  380: 0.0,
  381: 0.0,
  382: 0.0,
  383: 0.0,
  384: 0.0,
  385: 0.0,
  386: 0.0,
  387: 0.0,
  388: 0.0,
  389: 0.0,
  390: 0.0,
  391: 0.0,
  392: 0.0,
  393: 0.0,
  394: 0.0,
  395: 0.0,
  396: 0.0,
  397: 0.0,
  398: 0.0,
  399: 0.0,
  400: 0.0,
  401: 0.0,
  402: 0.0,
  403: 0.0,
  404: 0.0,
  405: 0.0,
  406: 0.0,
  407: 0.0,
  408: 0.0,
  409: 0.0,
  410: 0.0,
  411: 0.0,
  412: 0.0,
  413: 0.0,
  414: 0.0,
  415: 0.0,
  416: 0.0,
  417: 0.0,
  418: 0.0,
  419: 0.0,
  420: 0.0,
  421: 0.0,
  422: 0.0,
  423: 0.0,
  424: 0.0,
  425: 0.0,
  426: 0.0,
  427: 0.0,
  428: 0.0,
  429: 0.0,
  430: 0.0,
  431: 0.0,
  432: 0.0,
  433: 0.0,
  434: 0.0,
  435: 0.0,
  436: 0.0,
  437: 0.0,
  438: 0.0,
  439: 0.0,
  440: 0.0,
  441: 0.0,
  442: 0.0,
  443: 0.0,
  444: 0.0,
  445: 0.0,
  446: 0.0,
  447: 0.0,
  448: 0.0,
  449: 0.0,
  450: 0.0,
  451: 0.0,
  452: 0.0,
  453: 0.0,
  454: 0.0,
  455: 0.0,
  456: 0.0,
  457: 0.0,
  458: 0.0,
  459: 0.0,
  460: 0.0,
  461: 0.0,
  462: 0.0,
  463: 0.0,
  464: 0.0,
  465: 0.0,
  466: 0.0,
  467: 0.0,
  468: 0.0,
  469: 0.0,
  470: 0.0,
  471: 0.0,
  472: 0.0,
  473: 0.0,
  474: 0.0,
  475: 0.0,
  476: 0.0,
  477: 0.0,
  478: 0.0,
  479: 0.0,
  480: 0.0,
  481: 0.0,
  482: 0.0,
  483: 0.0,
  484: 0.0,
  485: 0.0,
  486: 0.0,
  487: 0.0,
  488: 0.0,
  489: 0.0,
  490: 0.0,
  491: 0.0,
  492: 0.0,
  493: 0.0,
  494: 0.0,
  495: 0.0,
  496: 0.0,
  497: 0.0,
  498: 0.0,
  499: 0.0,
  500: 0.0,
  501: 0.0,
  502: 0.0,
  503: 0.0,
  504: 0.0,
  505: 0.0,
  506: 0.0,
  507: 0.0,
  508: 0.0,
  509: 0.0,
  510: 3.585714286,
  511: 5.1625,
  512: 5.585714286,
  513: 5.628571429,
  514: 5.25,
  515: 0.0,
  516: 4.228571429,
  517: 5.671428571,
  518: 2.366666667,
  519: 5.425,
  520: 3.571428571,
  521: 4.028571429,
  522: 4.85,
  523: 3.6,
  524: 3.75,
  525: 5.075,
  526: 3.7375,
  527: 5.275,
  528: 5.275,
  529: 5.5,
  530: 3.3625,
  531: 26.8625,
  532: 30.01428571,
  533: 30.775,
  534: 30.5625,
  535: 30.1,
  536: 5.9125,
  537: 30.4375,
  538: 29.32857143,
  539: 28.63333333,
  540: 30.44285714,
  541: 27.0125,
  542: 28.85714286,
  543: 28.95714286,
  544: 29.67142857,
  545: 30.47142857,
  546: 29.25714286,
  547: 30.08571429,
  548: 29.84285714,
  549: 30.18571429,
  550: 29.08571429,
  551: 26.84285714,
  552: 60.68571429,
  553: 69.2,
  554: 69.8,
  555: 68.24285714,
  556: 67.1375,
  557: 69.42857143,
  558: 69.02857143,
  559: 84.8,
  560: 67.625,
  561: 65.35,
  562: 69.7375,
  563: 60.77142857,
  564: 65.5875,
  565: 65.95,
  566: 66.575,
  567: 68.1375,
  568: 66.2125,
  569: 67.475,
  570: 66.1125,
  571: 66.775,
  572: 66.0125,
  573: 59.55,
  574: 119.275,
  575: 130.1428571,
  576: 127.4142857,
  577: 130.4875,
  578: 129.8857143,
  579: 134.075,
  580: 126.6714286,
  581: 125.9625,
  582: 134.6428571,
  583: 125.75,
  584: 131.5,
  585: 117.8125,
  586: 123.8285714,
  587: 136.6,
  588: 123.1142857,
  589: 126.0857143,
  590: 126.3714286,
  591: 125.6857143,
  592: 128.5142857,
  593: 130.0142857,
  594: 123.1571429,
  595: 113.4571429,
  596: 170.0142857,
  597: 195.1,
  598: 192.45,
  599: 186.7,
  600: 188.25,
  601: 190.3714286,
  602: 189.9125,
  603: 182.5857143,
  604: 186.7,
  605: 183.3857143,
  606: 197.0,
  607: 170.5,
  608: 183.8375,
  609: 184.38,
  610: 182.475,
  611: 182.8375,
  612: 184.0125,
  613: 185.8125,
  614: 183.625,
  615: 185.9625,
  616: 184.075,
  617: 162.675,
  618: 181.9375,
  619: 207.2714286,
  620: 204.9285714,
  621: 199.5285714,
  622: 205.7285714,
  623: 204.7428571,
  624: 198.4714286,
  625: 198.225,
  626: 203.5571429,
  627: 195.6125,
  628: 210.1714286,
  629: 183.65,
  630: 199.6285714,
  631: 199.4833333,
  632: 196.5142857,
  633: 197.2571429,
  634: 200.6142857,
  635: 202.1714286,
  636: 200.5571429,
  637: 203.2285714,
  638: 197.5571429,
  639: 177.8857143,
  640: 180.5857143,
  641: 203.7125,
  642: 203.05,
  643: 196.275,
  644: 198.3625,
  645: 202.85,
  646: 201.2,
  647: 193.1285714,
  648: 199.6285714,
  649: 191.4142857,
  650: 206.85,
  651: 182.8,
  652: 193.5125,
  653: 193.5833333,
  654: 191.9375,
  655: 191.2625,
  656: 193.0375,
  657: 195.925,
  658: 194.8571429,
  659: 198.6,
  660: 192.825,
  661: 170.5375,
  662: 184.0625,
  663: 210.0714286,
  664: 208.1142857,
  665: 200.7857143,
  666: 207.9571429,
  667: 205.8285714,
  668: 203.1142857,
  669: 200.125,
  670: 202.6428571,
  671: 197.6875,
  672: 213.4285714,
  673: 185.3,
  674: 196.0857143,
  675: 198.7285714,
  676: 185.9714286,
  677: 192.2714286,
  678: 201.0285714,
  679: 186.5428571,
  680: 200.325,
  681: 201.9375,
  682: 195.4428571,
  683: 174.6428571,
  684: 318.6714286,
  685: 275.4625,
  686: 268.625,
  687: 269.15,
  688: 315.05,
  689: 282.675,
  690: 272.8375,
  691: 283.9428571,
  692: 307.5875,
  693: 273.4428571,
  694: 276.4625,
  695: 286.8142857,
  696: 286.3,
  697: 311.5142857,
  698: 293.575,
  699: 278.1857143,
  700: 294.3125,
  701: 313.5375,
  702: 308.6428571,
  703: 312.3,
  704: 279.2625,
  705: 257.2571429,
  706: 384.8375,
  707: 447.5,
  708: 443.9571429,
  709: 407.1,
  710: 497.4142857,
  711: 413.8571429,
  712: 487.2428571,
  713: 426.7,
  714: 395.1857143,
  715: 425.3625,
  716: 453.1,
  717: 363.9125,
  718: 518.9857143,
  719: 413.02,
  720: 505.2571429,
  721: 500.6625,
  722: 506.8,
  723: 529.9285714,
  724: 456.0125,
  725: 498.5375,
  726: 486.6285714,
  727: 440.0625,
  728: 424.1285714,
  729: 488.75,
  730: 489.5875,
  731: 460.675,
  732: 549.4125,
  733: 469.2375,
  734: 496.25,
  735: 529.9142857,
  736: 495.4285714,
  737: 513.1571429,
  738: 485.35,
  739: 444.4,
  740: 523.4125,
  741: 486.4333333,
  742: 507.6625,
  743: 530.2571429,
  744: 536.425,
  745: 496.4625,
  746: 532.9,
  747: 517.8428571,
  748: 517.5875,
  749: 500.3285714,
  750: 417.7875,
  751: 504.3428571,
  752: 545.7285714,
  753: 475.3285714,
  754: 341.2571429,
  755: 477.3428571,
  756: 562.6428571,
  757: 402.7125,
  758: 373.0714286,
  759: 458.25,
  760: 517.7142857,
  761: 433.2,
  762: 354.4571429,
  763: 338.2,
  764: 394.9285714,
  765: 405.675,
  766: 343.2714286,
  767: 341.8857143,
  768: 341.5,
  769: 344.2625,
  770: 414.0714286,
  771: 292.4375,
  772: 559.2285714,
  773: 805.325,
  774: 752.0375,
  775: 787.6625,
  776: 722.3,
  777: 776.7625,
  778: 613.55,
  779: 806.0428571,
  780: 793.775,
  781: 803.6285714,
  782: 809.8,
  783: 668.8571429,
  784: 777.9375,
  785: 772.2857143,
  786: 802.55,
  787: 821.1285714,
  788: 722.6,
  789: 683.7,
  790: 710.0,
  791: 701.2571429,
  792: 822.45,
  793: 498.4714286,
  794: 394.0125,
  795: 347.9857143,
  796: 311.3857143,
  797: 352.8571429,
  798: 581.825,
  799: 362.4571429,
  800: 293.2285714,
  801: 488.8714286,
  802: 466.3142857,
  803: 406.0571429,
  804: 333.6428571,
  805: 376.325,
  806: 517.6,
  807: 536.5857143,
  808: 588.5142857,
  809: 518.7625,
  810: 567.8125,
  811: 688.1857143,
  812: 556.675,
  813: 581.2625,
  814: 422.4714286,
  815: 0.0,
  816: 315.8857143,
  817: 424.1375,
  818: 374.6375,
  819: 353.975,
  820: 350.1428571,
  821: 363.65,
  822: 378.975,
  823: 343.95,
  824: 341.0142857,
  825: 350.3125,
  826: 391.9375,
  827: 323.7,
  828: 365.4625,
  829: 333.2714286,
  830: 364.2142857,
  831: 365.3571429,
  832: 349.8714286,
  833: 371.1142857,
  834: 338.0714286,
  835: 344.2571429,
  836: 367.4625,
  837: 0.0,
  838: 632.3428571,
  839: 751.2714286,
  840: 763.2857143,
  841: 729.1857143,
  842: 746.9625,
  843: 746.7714286,
  844: 772.8,
  845: 746.8714286,
  846: 703.3428571,
  847: 730.2285714,
  848: 782.8,
  849: 615.3428571,
  850: 787.5428571,
  851: 687.85,
  852: 704.3875,
  853: 747.5375,
  854: 732.5,
  855: 636.6875,
  856: 767.7375,
  857: 691.8,
  858: 798.8142857,
  859: 517.05,
  860: 588.775,
  861: 837.2,
  862: 722.0,
  863: 714.275,
  864: 977.3142857,
  865: 659.9875,
  866: 648.275,
  867: 864.175,
  868: 673.025,
  869: 831.6875,
  870: 828.9428571,
  871: 567.0625,
  872: 928.4,
  873: 746.55,
  874: 922.8,
  875: 882.7714286,
  876: 928.9285714,
  877: 968.2428571,
  878: 831.5285714,
  879: 966.7428571,
  880: 868.6571429,
  881: 939.4571429,
  882: 650.0285714,
  883: 660.1625,
  884: 581.1,
  885: 664.5,
  886: 708.125,
  887: 687.4571429,
  888: 625.9142857,
  889: 796.1142857,
  890: 769.8428571,
  891: 742.7571429,
  892: 615.05,
  893: 604.3857143,
  894: 786.9375,
  895: 878.55,
  896: 600.675,
  897: 690.925,
  898: 874.1625,
  899: 692.7375,
  900: 707.6625,
  901: 602.65,
  902: 776.375,
  903: 815.8375,
  904: 431.875,
  905: 548.2285714,
  906: 521.6,
  907: 479.8875,
  908: 589.2571429,
  909: 484.875,
  910: 507.4,
  911: 585.625,
  912: 472.7571429,
  913: 606.7875,
  914: 515.5714286,
  915: 435.7875,
  916: 715.6571429,
  917: 493.74,
  918: 741.6857143,
  919: 753.6714286,
  920: 643.7571429,
  921: 630.7142857,
  922: 566.4571429,
  923: 600.5,
  924: 688.1428571,
  925: 634.6142857,
  926: 428.2285714,
  927: 480.05,
  928: 490.3875,
  929: 452.0142857,
  930: 525.6125,
  931: 470.4428571,
  932: 478.3375,
  933: 498.2714286,
  934: 477.3857143,
  935: 473.4,
  936: 486.55,
  937: 426.4285714,
  938: 514.7625,
  939: 453.32,
  940: 608.1125,
  941: 513.8,
  942: 489.6625,
  943: 667.2,
  944: 530.8375,
  945: 601.6125,
  946: 497.05,
  947: 528.1625,
  948: 668.725,
  949: 804.9,
  950: 717.8714286,
  951: 756.7714286,
  952: 1001.385714,
  953: 759.7857143,
  954: 692.8,
  955: 841.525,
  956: 846.3142857,
  957: 779.3375,
  958: 751.1714286,
  959: 696.1375,
  960: 1010.628571,
  961: 857.02,
  962: 987.3714286,
  963: 1039.528571,
  964: 941.3857143,
  965: 1023.128571,
  966: 844.6428571,
  967: 937.3428571,
  968: 974.4714286,
  969: 1004.314286,
  970: 584.7857143,
  971: 595.925,
  972: 573.425,
  973: 597.9,
  974: 851.3375,
  975: 601.6875,
  976: 545.65,
  977: 720.1142857,
  978: 630.5428571,
  979: 629.4428571,
  980: 584.65,
  981: 578.2571429,
  982: 828.3625,
  983: 652.5333333,
  984: 754.05,
  985: 788.2375,
  986: 750.875,
  987: 909.3125,
  988: 819.0375,
  989: 1039.7625,
  990: 805.55,
  991: 842.275,
  992: 609.8625,
  993: 724.4428571,
  994: 803.6714286,
  995: 716.9285714,
  996: 633.8571429,
  997: 759.2142857,
  998: 840.0857143,
  999: 621.0125,
  ...},
 'DAILY_YIELD': {0: 0.0,
  1: 0.0,
  2: 0.0,
  3: 0.0,
  4: 0.0,
  5: 0.0,
  6: 0.0,
  7: 0.0,
  8: 0.0,
  9: 0.0,
  10: 0.0,
  11: 0.0,
  12: 0.0,
  13: 0.0,
  14: 0.0,
  15: 0.0,
  16: 0.0,
  17: 0.0,
  18: 0.0,
  19: 0.0,
  20: 0.0,
  21: 0.0,
  22: 0.0,
  23: 0.0,
  24: 0.0,
  25: 0.0,
  26: 0.0,
  27: 0.0,
  28: 0.0,
  29: 0.0,
  30: 0.0,
  31: 0.0,
  32: 0.0,
  33: 0.0,
  34: 0.0,
  35: 0.0,
  36: 0.0,
  37: 0.0,
  38: 0.0,
  39: 0.0,
  40: 0.0,
  41: 0.0,
  42: 0.0,
  43: 0.0,
  44: 0.0,
  45: 0.0,
  46: 0.0,
  47: 0.0,
  48: 0.0,
  49: 0.0,
  50: 0.0,
  51: 0.0,
  52: 0.0,
  53: 0.0,
  54: 0.0,
  55: 0.0,
  56: 0.0,
  57: 0.0,
  58: 0.0,
  59: 0.0,
  60: 0.0,
  61: 0.0,
  62: 0.0,
  63: 0.0,
  64: 0.0,
  65: 0.0,
  66: 0.0,
  67: 0.0,
  68: 0.0,
  69: 0.0,
  70: 0.0,
  71: 0.0,
  72: 0.0,
  73: 0.0,
  74: 0.0,
  75: 0.0,
  76: 0.0,
  77: 0.0,
  78: 0.0,
  79: 0.0,
  80: 0.0,
  81: 0.0,
  82: 0.0,
  83: 0.0,
  84: 0.0,
  85: 0.0,
  86: 0.0,
  87: 0.0,
  88: 0.0,
  89: 0.0,
  90: 0.0,
  91: 0.0,
  92: 0.0,
  93: 0.0,
  94: 0.0,
  95: 0.0,
  96: 0.0,
  97: 0.0,
  98: 0.0,
  99: 0.0,
  100: 0.0,
  101: 0.0,
  102: 0.0,
  103: 0.0,
  104: 0.0,
  105: 0.0,
  106: 0.0,
  107: 0.0,
  108: 0.0,
  109: 0.0,
  110: 0.0,
  111: 0.0,
  112: 0.0,
  113: 0.0,
  114: 0.0,
  115: 0.0,
  116: 0.0,
  117: 0.0,
  118: 0.0,
  119: 0.0,
  120: 0.0,
  121: 0.0,
  122: 0.0,
  123: 0.0,
  124: 0.0,
  125: 0.0,
  126: 0.0,
  127: 0.0,
  128: 0.0,
  129: 0.0,
  130: 0.0,
  131: 0.0,
  132: 0.0,
  133: 0.0,
  134: 0.0,
  135: 0.0,
  136: 0.0,
  137: 0.0,
  138: 0.0,
  139: 0.0,
  140: 0.0,
  141: 0.0,
  142: 0.0,
  143: 0.0,
  144: 0.0,
  145: 0.0,
  146: 0.0,
  147: 0.0,
  148: 0.0,
  149: 0.0,
  150: 0.0,
  151: 0.0,
  152: 0.0,
  153: 0.0,
  154: 0.0,
  155: 0.0,
  156: 0.0,
  157: 0.0,
  158: 0.0,
  159: 0.0,
  160: 0.0,
  161: 0.0,
  162: 0.0,
  163: 0.0,
  164: 0.0,
  165: 0.0,
  166: 0.0,
  167: 0.0,
  168: 0.0,
  169: 0.0,
  170: 0.0,
  171: 0.0,
  172: 0.0,
  173: 0.0,
  174: 0.0,
  175: 0.0,
  176: 0.0,
  177: 0.0,
  178: 0.0,
  179: 0.0,
  180: 0.0,
  181: 0.0,
  182: 0.0,
  183: 0.0,
  184: 0.0,
  185: 0.0,
  186: 0.0,
  187: 0.0,
  188: 0.0,
  189: 0.0,
  190: 0.0,
  191: 0.0,
  192: 0.0,
  193: 0.0,
  194: 0.0,
  195: 0.0,
  196: 0.0,
  197: 0.0,
  198: 0.0,
  199: 0.0,
  200: 0.0,
  201: 0.0,
  202: 0.0,
  203: 0.0,
  204: 0.0,
  205: 0.0,
  206: 0.0,
  207: 0.0,
  208: 0.0,
  209: 0.0,
  210: 0.0,
  211: 0.0,
  212: 0.0,
  213: 0.0,
  214: 0.0,
  215: 0.0,
  216: 0.0,
  217: 0.0,
  218: 0.0,
  219: 0.0,
  220: 0.0,
  221: 0.0,
  222: 0.0,
  223: 0.0,
  224: 0.0,
  225: 0.0,
  226: 0.0,
  227: 0.0,
  228: 0.0,
  229: 0.0,
  230: 0.0,
  231: 0.0,
  232: 0.0,
  233: 0.0,
  234: 0.0,
  235: 0.0,
  236: 0.0,
  237: 0.0,
  238: 0.0,
  239: 0.0,
  240: 0.0,
  241: 0.0,
  242: 0.0,
  243: 0.0,
  244: 0.0,
  245: 0.0,
  246: 0.0,
  247: 0.0,
  248: 0.0,
  249: 0.0,
  250: 0.0,
  251: 0.0,
  252: 0.0,
  253: 0.0,
  254: 0.0,
  255: 0.0,
  256: 0.0,
  257: 0.0,
  258: 0.0,
  259: 0.0,
  260: 0.0,
  261: 0.0,
  262: 0.0,
  263: 0.0,
  264: 0.0,
  265: 0.0,
  266: 0.0,
  267: 0.0,
  268: 0.0,
  269: 0.0,
  270: 0.0,
  271: 0.0,
  272: 0.0,
  273: 0.0,
  274: 0.0,
  275: 0.0,
  276: 0.0,
  277: 0.0,
  278: 0.0,
  279: 0.0,
  280: 0.0,
  281: 0.0,
  282: 0.0,
  283: 0.0,
  284: 0.0,
  285: 0.0,
  286: 0.0,
  287: 0.0,
  288: 0.0,
  289: 0.0,
  290: 0.0,
  291: 0.0,
  292: 0.0,
  293: 0.0,
  294: 0.0,
  295: 0.0,
  296: 0.0,
  297: 0.0,
  298: 0.0,
  299: 0.0,
  300: 0.0,
  301: 0.0,
  302: 0.0,
  303: 0.0,
  304: 0.0,
  305: 0.0,
  306: 0.0,
  307: 0.0,
  308: 0.0,
  309: 0.0,
  310: 0.0,
  311: 0.0,
  312: 0.0,
  313: 0.0,
  314: 0.0,
  315: 0.0,
  316: 0.0,
  317: 0.0,
  318: 0.0,
  319: 0.0,
  320: 0.0,
  321: 0.0,
  322: 0.0,
  323: 0.0,
  324: 0.0,
  325: 0.0,
  326: 0.0,
  327: 0.0,
  328: 0.0,
  329: 0.0,
  330: 0.0,
  331: 0.0,
  332: 0.0,
  333: 0.0,
  334: 0.0,
  335: 0.0,
  336: 0.0,
  337: 0.0,
  338: 0.0,
  339: 0.0,
  340: 0.0,
  341: 0.0,
  342: 0.0,
  343: 0.0,
  344: 0.0,
  345: 0.0,
  346: 0.0,
  347: 0.0,
  348: 0.0,
  349: 0.0,
  350: 0.0,
  351: 0.0,
  352: 0.0,
  353: 0.0,
  354: 0.0,
  355: 0.0,
  356: 0.0,
  357: 0.0,
  358: 0.0,
  359: 0.0,
  360: 0.0,
  361: 0.0,
  362: 0.0,
  363: 0.0,
  364: 0.0,
  365: 0.0,
  366: 0.0,
  367: 0.0,
  368: 0.0,
  369: 0.0,
  370: 0.0,
  371: 0.0,
  372: 0.0,
  373: 0.0,
  374: 0.0,
  375: 0.0,
  376: 0.0,
  377: 0.0,
  378: 0.0,
  379: 0.0,
  380: 0.0,
  381: 0.0,
  382: 0.0,
  383: 0.0,
  384: 0.0,
  385: 0.0,
  386: 0.0,
  387: 0.0,
  388: 0.0,
  389: 0.0,
  390: 0.0,
  391: 0.0,
  392: 0.0,
  393: 0.0,
  394: 0.0,
  395: 0.0,
  396: 0.0,
  397: 0.0,
  398: 0.0,
  399: 0.0,
  400: 0.0,
  401: 0.0,
  402: 0.0,
  403: 0.0,
  404: 0.0,
  405: 0.0,
  406: 0.0,
  407: 0.0,
  408: 0.0,
  409: 0.0,
  410: 0.0,
  411: 0.0,
  412: 0.0,
  413: 0.0,
  414: 0.0,
  415: 0.0,
  416: 0.0,
  417: 0.0,
  418: 0.0,
  419: 0.0,
  420: 0.0,
  421: 0.0,
  422: 0.0,
  423: 0.0,
  424: 0.0,
  425: 0.0,
  426: 0.0,
  427: 0.0,
  428: 0.0,
  429: 0.0,
  430: 0.0,
  431: 0.0,
  432: 0.0,
  433: 0.0,
  434: 0.0,
  435: 0.0,
  436: 0.0,
  437: 0.0,
  438: 0.0,
  439: 0.0,
  440: 0.0,
  441: 0.0,
  442: 0.0,
  443: 0.0,
  444: 0.0,
  445: 0.0,
  446: 0.0,
  447: 0.0,
  448: 0.0,
  449: 0.0,
  450: 0.0,
  451: 0.0,
  452: 0.0,
  453: 0.0,
  454: 0.0,
  455: 0.0,
  456: 0.0,
  457: 0.0,
  458: 0.0,
  459: 0.0,
  460: 0.0,
  461: 0.0,
  462: 0.0,
  463: 0.0,
  464: 0.0,
  465: 0.0,
  466: 0.0,
  467: 0.0,
  468: 0.0,
  469: 0.0,
  470: 0.0,
  471: 0.0,
  472: 0.0,
  473: 0.0,
  474: 0.0,
  475: 0.0,
  476: 0.0,
  477: 0.0,
  478: 0.0,
  479: 0.0,
  480: 0.0,
  481: 0.0,
  482: 0.0,
  483: 0.0,
  484: 0.0,
  485: 0.0,
  486: 0.0,
  487: 0.0,
  488: 0.0,
  489: 0.0,
  490: 0.0,
  491: 0.0,
  492: 0.0,
  493: 0.0,
  494: 0.0,
  495: 0.0,
  496: 0.0,
  497: 0.0,
  498: 0.0,
  499: 0.0,
  500: 0.0,
  501: 0.0,
  502: 0.0,
  503: 0.0,
  504: 0.0,
  505: 0.0,
  506: 0.0,
  507: 0.0,
  508: 0.0,
  509: 0.0,
  510: 0.0,
  511: 0.0,
  512: 0.0,
  513: 0.0,
  514: 0.0,
  515: 0.0,
  516: 0.0,
  517: 0.0,
  518: 0.0,
  519: 0.0,
  520: 0.0,
  521: 0.0,
  522: 0.0,
  523: 0.0,
  524: 0.0,
  525: 0.0,
  526: 0.0,
  527: 0.0,
  528: 0.0,
  529: 0.0,
  530: 0.0,
  531: 2.625,
  532: 3.0,
  533: 3.5,
  534: 3.5,
  535: 3.142857143,
  536: 0.0,
  537: 3.375,
  538: 3.142857143,
  539: 2.833333333,
  540: 3.428571429,
  541: 3.0,
  542: 2.571428571,
  543: 3.142857143,
  544: 2.571428571,
  545: 3.0,
  546: 3.0,
  547: 3.142857143,
  548: 3.428571429,
  549: 3.428571429,
  550: 2.857142857,
  551: 2.571428571,
  552: 13.28571429,
  553: 15.5,
  554: 15.14285714,
  555: 15.0,
  556: 15.375,
  557: 7.142857143,
  558: 15.14285714,
  559: 22.0,
  560: 15.5,
  561: 14.33333333,
  562: 15.5,
  563: 13.85714286,
  564: 14.5,
  565: 15.25,
  566: 14.25,
  567: 14.875,
  568: 14.75,
  569: 15.25,
  570: 15.25,
  571: 15.25,
  572: 14.375,
  573: 13.0,
  574: 34.875,
  575: 39.14285714,
  576: 37.0,
  577: 39.0,
  578: 38.71428571,
  579: 32.25,
  580: 36.85714286,
  581: 37.25,
  582: 40.0,
  583: 37.0,
  584: 39.0,
  585: 34.75,
  586: 36.42857143,
  587: 40.66666667,
  588: 37.14285714,
  589: 38.14285714,
  590: 37.42857143,
  591: 37.71428571,
  592: 37.71428571,
  593: 38.0,
  594: 36.57142857,
  595: 33.85714286,
  596: 72.71428571,
  597: 82.375,
  598: 79.5,
  599: 80.0,
  600: 80.25,
  601: 75.14285714,
  602: 79.0,
  603: 77.57142857,
  604: 81.0,
  605: 78.0,
  606: 82.375,
  607: 72.57142857,
  608: 77.0,
  609: 81.0,
  610: 77.875,
  611: 78.5,
  612: 78.125,
  613: 78.625,
  614: 79.0,
  615: 80.25,
  616: 76.875,
  617: 70.0,
  618: 116.25,
  619: 131.7142857,
  620: 129.7142857,
  621: 124.7142857,
  622: 129.4285714,
  623: 120.2857143,
  624: 126.7142857,
  625: 125.25,
  626: 127.8571429,
  627: 125.75,
  628: 134.1428571,
  629: 116.25,
  630: 124.4285714,
  631: 121.3333333,
  632: 125.0,
  633: 126.1428571,
  634: 125.8571429,
  635: 126.8571429,
  636: 127.0,
  637: 129.0,
  638: 125.5714286,
  639: 112.7142857,
  640: 162.4285714,
  641: 184.625,
  642: 181.875,
  643: 175.375,
  644: 181.125,
  645: 172.375,
  646: 177.625,
  647: 175.0,
  648: 178.2857143,
  649: 174.8571429,
  650: 187.5,
  651: 162.8571429,
  652: 174.375,
  653: 173.5,
  654: 174.5,
  655: 176.75,
  656: 175.875,
  657: 177.625,
  658: 174.8571429,
  659: 176.7142857,
  660: 175.25,
  661: 157.0,
  662: 206.375,
  663: 233.7142857,
  664: 231.2857143,
  665: 223.8571429,
  666: 229.5714286,
  667: 221.8571429,
  668: 227.1428571,
  669: 223.125,
  670: 224.7142857,
  671: 221.625,
  672: 237.7142857,
  673: 207.625,
  674: 222.7142857,
  675: 219.5714286,
  676: 219.4285714,
  677: 222.2857143,
  678: 224.0,
  679: 225.0,
  680: 222.5,
  681: 224.75,
  682: 221.8571429,
  683: 197.7142857,
  684: 263.5714286,
  685: 292.5,
  686: 287.75,
  687: 281.0,
  688: 291.0,
  689: 279.125,
  690: 284.0,
  691: 281.5714286,
  692: 287.125,
  693: 277.7142857,
  694: 296.375,
  695: 262.5714286,
  696: 280.875,
  697: 285.2857143,
  698: 276.375,
  699: 275.2857143,
  700: 284.0,
  701: 284.0,
  702: 281.2857143,
  703: 284.7142857,
  704: 279.125,
  705: 245.5714286,
  706: 359.875,
  707: 380.0,
  708: 376.0,
  709: 364.5714286,
  710: 395.4285714,
  711: 365.8571429,
  712: 384.0,
  713: 370.125,
  714: 380.0,
  715: 364.375,
  716: 385.7142857,
  717: 347.625,
  718: 379.8571429,
  719: 371.6,
  720: 374.4285714,
  721: 371.25,
  722: 383.8571429,
  723: 389.0,
  724: 382.875,
  725: 395.5,
  726: 372.1428571,
  727: 335.125,
  728: 464.0,
  729: 509.0,
  730: 501.5,
  731: 481.5,
  732: 532.75,
  733: 484.125,
  734: 513.25,
  735: 497.5714286,
  736: 490.7142857,
  737: 490.0,
  738: 512.375,
  739: 453.0,
  740: 517.5,
  741: 472.6666667,
  742: 508.5,
  743: 508.8571429,
  744: 520.375,
  745: 524.375,
  746: 504.7142857,
  747: 519.4285714,
  748: 506.5,
  749: 457.2857143,
  750: 556.125,
  751: 617.7142857,
  752: 613.2857143,
  753: 584.0,
  754: 641.4285714,
  755: 592.0,
  756: 627.2857143,
  757: 600.625,
  758: 592.1428571,
  759: 592.5,
  760: 621.8571429,
  761: 547.875,
  762: 621.5714286,
  763: 584.4285714,
  764: 614.2857143,
  765: 612.25,
  766: 625.8571429,
  767: 625.4285714,
  768: 610.875,
  769: 623.25,
  770: 610.8571429,
  771: 553.875,
  772: 688.8571429,
  773: 804.125,
  774: 798.0,
  775: 759.5,
  776: 757.7142857,
  777: 767.0,
  778: 794.625,
  779: 755.5714286,
  780: 748.75,
  781: 761.0,
  782: 809.125,
  783: 699.1428571,
  784: 762.75,
  785: 738.2857143,
  786: 771.0,
  787: 769.2857143,
  788: 745.1428571,
  789: 750.125,
  790: 732.7142857,
  791: 748.4285714,
  792: 778.75,
  793: 653.2857143,
  794: 811.625,
  795: 946.5714286,
  796: 929.2857143,
  797: 903.2857143,
  798: 939.25,
  799: 912.0,
  800: 909.7142857,
  801: 917.1428571,
  802: 919.8571429,
  803: 906.4285714,
  804: 954.7142857,
  805: 830.5,
  806: 941.0,
  807: 909.2857143,
  808: 956.7142857,
  809: 946.5,
  810: 925.25,
  811: 934.7142857,
  812: 909.625,
  813: 921.375,
  814: 945.1428571,
  815: 711.0,
  816: 886.5714286,
  817: 1029.125,
  818: 1004.375,
  819: 980.75,
  820: 1044.857143,
  821: 990.0,
  822: 979.375,
  823: 1008.125,
  824: 1005.714286,
  825: 993.25,
  826: 1034.5,
  827: 903.1428571,
  828: 1040.75,
  829: 1001.857143,
  830: 1061.857143,
  831: 1048.285714,
  832: 1029.285714,
  833: 1058.142857,
  834: 1014.285714,
  835: 1029.142857,
  836: 1034.5,
  837: 711.0,
  838: 991.4285714,
  839: 1166.571429,
  840: 1138.0,
  841: 1105.428571,
  842: 1167.0,
  843: 1117.857143,
  844: 1117.0,
  845: 1131.714286,
  846: 1122.0,
  847: 1121.285714,
  848: 1170.428571,
  849: 1006.857143,
  850: 1169.428571,
  851: 1126.166667,
  852: 1180.375,
  853: 1177.0,
  854: 1154.125,
  855: 1169.375,
  856: 1142.25,
  857: 1144.875,
  858: 1166.714286,
  859: 747.5,
  860: 1161.25,
  861: 1390.714286,
  862: 1366.375,
  863: 1327.0,
  864: 1405.571429,
  865: 1324.625,
  866: 1329.75,
  867: 1367.0,
  868: 1325.875,
  869: 1350.125,
  870: 1393.857143,
  871: 1173.25,
  872: 1397.142857,
  873: 1342.625,
  874: 1411.571429,
  875: 1413.714286,
  876: 1395.0,
  877: 1394.428571,
  878: 1357.142857,
  879: 1357.714286,
  880: 1391.571429,
  881: 970.4285714,
  882: 1307.571429,
  883: 1542.625,
  884: 1509.0,
  885: 1471.142857,
  886: 1596.125,
  887: 1471.285714,
  888: 1466.428571,
  889: 1541.571429,
  890: 1494.857143,
  891: 1503.285714,
  892: 1542.0,
  893: 1311.142857,
  894: 1572.75,
  895: 1529.5,
  896: 1580.125,
  897: 1572.0,
  898: 1574.75,
  899: 1584.375,
  900: 1535.25,
  901: 1558.0,
  902: 1551.75,
  903: 1158.0,
  904: 1440.0,
  905: 1723.142857,
  906: 1661.857143,
  907: 1634.125,
  908: 1782.428571,
  909: 1636.0,
  910: 1606.285714,
  911: 1740.0,
  912: 1657.571429,
  913: 1692.625,
  914: 1712.428571,
  915: 1454.625,
  916: 1782.285714,
  917: 1695.8,
  918: 1765.571429,
  919: 1766.0,
  920: 1774.285714,
  921: 1757.857143,
  922: 1713.428571,
  923: 1718.285714,
  924: 1756.857143,
  925: 1351.714286,
  926: 1549.0,
  927: 1851.75,
  928: 1791.75,
  929: 1752.571429,
  930: 1929.25,
  931: 1760.142857,
  932: 1729.125,
  933: 1866.428571,
  934: 1777.285714,
  935: 1817.714286,
  936: 1842.5,
  937: 1564.0,
  938: 1926.375,
  939: 1822.4,
  940: 1936.75,
  941: 1916.25,
  942: 1917.625,
  943: 1930.375,
  944: 1850.375,
  945: 1867.25,
  946: 1895.125,
  947: 1505.25,
  948: 1693.0,
  949: 2014.571429,
  950: 1949.571429,
  951: 1893.714286,
  952: 2107.571429,
  953: 1907.0,
  954: 1877.571429,
  955: 2030.25,
  956: 1937.0,
  957: 1983.375,
  958: 2006.285714,
  959: 1711.5,
  960: 2092.714286,
  961: 1947.4,
  962: 2115.857143,
  963: 2089.285714,
  964: 2088.714286,
  965: 2124.571429,
  966: 2018.857143,
  967: 2044.571429,
  968: 2067.285714,
  969: 1689.285714,
  970: 1843.714286,
  971: 2194.125,
  972: 2110.375,
  973: 2064.875,
  974: 2333.125,
  975: 2073.0,
  976: 2031.0,
  977: 2226.857143,
  978: 2119.285714,
  979: 2184.285714,
  980: 2176.75,
  981: 1865.285714,
  982: 2313.375,
  983: 2166.333333,
  984: 2341.75,
  985: 2300.5,
  986: 2305.375,
  987: 2371.0,
  988: 2234.875,
  989: 2303.25,
  990: 2271.375,
  991: 1922.25,
  992: 1994.5,
  993: 2377.571429,
  994: 2295.857143,
  995: 2242.0,
  996: 2518.714286,
  997: 2252.285714,
  998: 2217.0,
  999: 2394.875,
  ...},
 'TOTAL_YIELD': {0: 6259559.0,
  1: 6183645.0,
  2: 6987759.0,
  3: 7602960.0,
  4: 7158964.0,
  5: 7206408.0,
  6: 7028673.0,
  7: 6522172.0,
  8: 7098099.0,
  9: 6271355.0,
  10: 6316803.0,
  11: 7177992.0,
  12: 6185184.0,
  13: 7169102.0,
  14: 7111493.0,
  15: 7016832.0,
  16: 7038681.0,
  17: 6782598.0,
  18: 7007866.0,
  19: 6339380.0,
  20: 7116151.0,
  21: 6259559.0,
  22: 6183645.0,
  23: 6987759.0,
  24: 7602960.0,
  25: 7158964.0,
  26: 7206408.0,
  27: 7028673.0,
  28: 6522172.0,
  29: 7098099.0,
  30: 6271355.0,
  31: 6316803.0,
  32: 7177992.0,
  33: 6185184.0,
  34: 7169102.0,
  35: 7111493.0,
  36: 7016832.0,
  37: 7038681.0,
  38: 6782598.0,
  39: 7007866.0,
  40: 6339380.0,
  41: 7116151.0,
  42: 6259559.0,
  43: 6183645.0,
  44: 6987759.0,
  45: 7602960.0,
  46: 7158964.0,
  47: 7206408.0,
  48: 7028673.0,
  49: 6522172.0,
  50: 7098099.0,
  51: 6271355.0,
  52: 6316803.0,
  53: 7177992.0,
  54: 6185184.0,
  55: 7169102.0,
  56: 7111493.0,
  57: 7016832.0,
  58: 7038681.0,
  59: 6782598.0,
  60: 7007866.0,
  61: 6339380.0,
  62: 7116151.0,
  63: 6259559.0,
  64: 6183645.0,
  65: 6987759.0,
  66: 7602960.0,
  67: 7158964.0,
  68: 7206408.0,
  69: 7028673.0,
  70: 6522172.0,
  71: 7098099.0,
  72: 6271355.0,
  73: 6316803.0,
  74: 7177992.0,
  75: 6185184.0,
  76: 7169102.0,
  77: 7111493.0,
  78: 7016832.0,
  79: 7038681.0,
  80: 6782598.0,
  81: 7007866.0,
  82: 6339380.0,
  83: 7116151.0,
  84: 6259559.0,
  85: 6183645.0,
  86: 6987759.0,
  87: 7602960.0,
  88: 7158964.0,
  89: 7206408.0,
  90: 7028673.0,
  91: 7179966.0,
  92: 6522172.0,
  93: 7098099.0,
  94: 6271355.0,
  95: 6316803.0,
  96: 7177992.0,
  97: 6185184.0,
  98: 7169102.0,
  99: 7111493.0,
  100: 7016832.0,
  101: 7038681.0,
  102: 6782598.0,
  103: 7007866.0,
  104: 6339380.0,
  105: 7116151.0,
  106: 6259559.0,
  107: 6183645.0,
  108: 6987759.0,
  109: 7602960.0,
  110: 7158964.0,
  111: 7206408.0,
  112: 7028673.0,
  113: 6522172.0,
  114: 7098099.0,
  115: 6271355.0,
  116: 6316803.0,
  117: 7177992.0,
  118: 6185184.0,
  119: 7169102.0,
  120: 7111493.0,
  121: 7016832.0,
  122: 7038681.0,
  123: 6782598.0,
  124: 7007866.0,
  125: 6339380.0,
  126: 7116151.0,
  127: 6259559.0,
  128: 6183645.0,
  129: 6987759.0,
  130: 7602960.0,
  131: 7158964.0,
  132: 7206408.0,
  133: 7028673.0,
  134: 6522172.0,
  135: 7098099.0,
  136: 6271355.0,
  137: 6316803.0,
  138: 7177992.0,
  139: 6185184.0,
  140: 7169102.0,
  141: 7111493.0,
  142: 7016832.0,
  143: 7038681.0,
  144: 6782598.0,
  145: 7007866.0,
  146: 6339380.0,
  147: 7116151.0,
  148: 6259559.0,
  149: 6183645.0,
  150: 6987759.0,
  151: 7602960.0,
  152: 7158964.0,
  153: 7206408.0,
  154: 7028673.0,
  155: 7179966.0,
  156: 6522172.0,
  157: 7098099.0,
  158: 6271355.0,
  159: 6316803.0,
  160: 7177992.0,
  161: 6185184.0,
  162: 7169102.0,
  163: 7111493.0,
  164: 7016832.0,
  165: 7038681.0,
  166: 6782598.0,
  167: 7007866.0,
  168: 6339380.0,
  169: 7116151.0,
  170: 6259559.0,
  171: 6183645.0,
  172: 6987759.0,
  173: 7602960.0,
  174: 7158964.0,
  175: 7206408.0,
  176: 7028673.0,
  177: 6522172.0,
  178: 7098099.0,
  179: 6271355.0,
  180: 6316803.0,
  181: 7177992.0,
  182: 6185184.0,
  183: 7169102.0,
  184: 7111493.0,
  185: 7016832.0,
  186: 7038681.0,
  187: 6782598.0,
  188: 7007866.0,
  189: 6339380.0,
  190: 7116151.0,
  191: 6259559.0,
  192: 6183645.0,
  193: 6987759.0,
  194: 7602960.0,
  195: 7158964.0,
  196: 7206408.0,
  197: 7028673.0,
  198: 6522172.0,
  199: 7098099.0,
  200: 6271355.0,
  201: 6316803.0,
  202: 7177992.0,
  203: 6185184.0,
  204: 7169102.0,
  205: 7111493.0,
  206: 7016832.0,
  207: 7038681.0,
  208: 6782598.0,
  209: 7007866.0,
  210: 6339380.0,
  211: 7116151.0,
  212: 6259559.0,
  213: 6183645.0,
  214: 6987759.0,
  215: 7602960.0,
  216: 7158964.0,
  217: 7206408.0,
  218: 7028673.0,
  219: 6522172.0,
  220: 7098099.0,
  221: 6271355.0,
  222: 6316803.0,
  223: 7177992.0,
  224: 6185184.0,
  225: 7169102.0,
  226: 7111493.0,
  227: 7016832.0,
  228: 7038681.0,
  229: 6782598.0,
  230: 7007866.0,
  231: 6339380.0,
  232: 7116151.0,
  233: 6259559.0,
  234: 6183645.0,
  235: 6987759.0,
  236: 7602960.0,
  237: 7158964.0,
  238: 7206408.0,
  239: 7028673.0,
  240: 6522172.0,
  241: 7098099.0,
  242: 6271355.0,
  243: 6316803.0,
  244: 7177992.0,
  245: 6185184.0,
  246: 7169102.0,
  247: 7111493.0,
  248: 7016832.0,
  249: 7038681.0,
  250: 6782598.0,
  251: 7007866.0,
  252: 6339380.0,
  253: 7116151.0,
  254: 6259559.0,
  255: 6183645.0,
  256: 6987759.0,
  257: 7602960.0,
  258: 7158964.0,
  259: 7206408.0,
  260: 7028673.0,
  261: 6522172.0,
  262: 7098099.0,
  263: 6271355.0,
  264: 6316803.0,
  265: 7177992.0,
  266: 6185184.0,
  267: 7169102.0,
  268: 7111493.0,
  269: 7016832.0,
  270: 7038681.0,
  271: 6782598.0,
  272: 7007866.0,
  273: 6339380.0,
  274: 7116151.0,
  275: 6259559.0,
  276: 6183645.0,
  277: 6987759.0,
  278: 7602960.0,
  279: 7158964.0,
  280: 7206408.0,
  281: 7028673.0,
  282: 6522172.0,
  283: 7098099.0,
  284: 6271355.0,
  285: 6316803.0,
  286: 7177992.0,
  287: 6185184.0,
  288: 7169102.0,
  289: 7111493.0,
  290: 7016832.0,
  291: 7038681.0,
  292: 6782598.0,
  293: 7007866.0,
  294: 6339380.0,
  295: 7116151.0,
  296: 6259559.0,
  297: 6183645.0,
  298: 6987759.0,
  299: 7602960.0,
  300: 7158964.0,
  301: 7206408.0,
  302: 7028673.0,
  303: 6522172.0,
  304: 7098099.0,
  305: 6271355.0,
  306: 6316803.0,
  307: 7177992.0,
  308: 6185184.0,
  309: 7169102.0,
  310: 7111493.0,
  311: 7016832.0,
  312: 7038681.0,
  313: 6782598.0,
  314: 7007866.0,
  315: 6339380.0,
  316: 7116151.0,
  317: 6259559.0,
  318: 6183645.0,
  319: 6987759.0,
  320: 7602960.0,
  321: 7158964.0,
  322: 7206408.0,
  323: 7028673.0,
  324: 6522172.0,
  325: 7098099.0,
  326: 6271355.0,
  327: 6316803.0,
  328: 7177992.0,
  329: 6185184.0,
  330: 7169102.0,
  331: 7111493.0,
  332: 7016832.0,
  333: 7038681.0,
  334: 6782598.0,
  335: 7007866.0,
  336: 6339380.0,
  337: 7116151.0,
  338: 6259559.0,
  339: 6183645.0,
  340: 6987759.0,
  341: 7602960.0,
  342: 7158964.0,
  343: 7206408.0,
  344: 7028673.0,
  345: 6522172.0,
  346: 7098099.0,
  347: 6271355.0,
  348: 6316803.0,
  349: 7177992.0,
  350: 6185184.0,
  351: 7169102.0,
  352: 7111493.0,
  353: 7016832.0,
  354: 7038681.0,
  355: 6782598.0,
  356: 7007866.0,
  357: 6339380.0,
  358: 7116151.0,
  359: 6259559.0,
  360: 6183645.0,
  361: 6987759.0,
  362: 7602960.0,
  363: 7158964.0,
  364: 7206408.0,
  365: 7028673.0,
  366: 7179966.0,
  367: 6522172.0,
  368: 7098099.0,
  369: 6271355.0,
  370: 6316803.0,
  371: 7177992.0,
  372: 6185184.0,
  373: 7169102.0,
  374: 7111493.0,
  375: 7016832.0,
  376: 7038681.0,
  377: 6782598.0,
  378: 7007866.0,
  379: 6339380.0,
  380: 7116151.0,
  381: 6259559.0,
  382: 6183645.0,
  383: 6987759.0,
  384: 7602960.0,
  385: 7158964.0,
  386: 7206408.0,
  387: 7028673.0,
  388: 7179966.0,
  389: 6522172.0,
  390: 7098099.0,
  391: 6271355.0,
  392: 6316803.0,
  393: 7177992.0,
  394: 6185184.0,
  395: 7169102.0,
  396: 7111493.0,
  397: 7016832.0,
  398: 7038681.0,
  399: 6782598.0,
  400: 7007866.0,
  401: 6339380.0,
  402: 7116151.0,
  403: 6259559.0,
  404: 6183645.0,
  405: 6987759.0,
  406: 7602960.0,
  407: 7158964.0,
  408: 7206408.0,
  409: 7028673.0,
  410: 7179966.0,
  411: 6522172.0,
  412: 7098099.0,
  413: 6271355.0,
  414: 6316803.0,
  415: 7177992.0,
  416: 6185184.0,
  417: 7169102.0,
  418: 7111493.0,
  419: 7016832.0,
  420: 7038681.0,
  421: 6782598.0,
  422: 7007866.0,
  423: 6339380.0,
  424: 7116151.0,
  425: 6259559.0,
  426: 6183645.0,
  427: 6987759.0,
  428: 7602960.0,
  429: 7158964.0,
  430: 7206408.0,
  431: 7028673.0,
  432: 7179966.0,
  433: 6522172.0,
  434: 7098099.0,
  435: 6271355.0,
  436: 6316803.0,
  437: 7177992.0,
  438: 6185184.0,
  439: 7169102.0,
  440: 7111493.0,
  441: 7016832.0,
  442: 7038681.0,
  443: 6782598.0,
  444: 7007866.0,
  445: 6339380.0,
  446: 7116151.0,
  447: 6259559.0,
  448: 6183645.0,
  449: 6987759.0,
  450: 7602960.0,
  451: 7158964.0,
  452: 7206408.0,
  453: 7028673.0,
  454: 6522172.0,
  455: 7098099.0,
  456: 6271355.0,
  457: 6316803.0,
  458: 7177992.0,
  459: 6185184.0,
  460: 7169102.0,
  461: 7111493.0,
  462: 7016832.0,
  463: 7038681.0,
  464: 6782598.0,
  465: 7007866.0,
  466: 6339380.0,
  467: 7116151.0,
  468: 6259559.0,
  469: 6183645.0,
  470: 6987759.0,
  471: 7602960.0,
  472: 7158964.0,
  473: 7206408.0,
  474: 7028673.0,
  475: 6522172.0,
  476: 7098099.0,
  477: 6271355.0,
  478: 6316803.0,
  479: 7177992.0,
  480: 6185184.0,
  481: 7169102.0,
  482: 7111493.0,
  483: 7016832.0,
  484: 7038681.0,
  485: 6782598.0,
  486: 7007866.0,
  487: 6339380.0,
  488: 7116151.0,
  489: 6259559.0,
  490: 6183645.0,
  491: 6987759.0,
  492: 7602960.0,
  493: 7158964.0,
  494: 7206408.0,
  495: 7028673.0,
  496: 6522172.0,
  497: 7098099.0,
  498: 6271355.0,
  499: 6316803.0,
  500: 7177992.0,
  501: 6185184.0,
  502: 7169102.0,
  503: 7111493.0,
  504: 7016832.0,
  505: 7038681.0,
  506: 6782598.0,
  507: 7007866.0,
  508: 6339380.0,
  509: 7116151.0,
  510: 6259559.0,
  511: 6183645.0,
  512: 6987759.0,
  513: 7602960.0,
  514: 7158964.0,
  515: 7206408.0,
  516: 7028673.0,
  517: 6522172.0,
  518: 7098099.0,
  519: 6271355.0,
  520: 6316803.0,
  521: 7177992.0,
  522: 6185184.0,
  523: 7169102.0,
  524: 7111493.0,
  525: 7016832.0,
  526: 7038681.0,
  527: 6782598.0,
  528: 7007866.0,
  529: 6339380.0,
  530: 7116151.0,
  531: 6259561.625,
  532: 6183648.0,
  533: 6987762.5,
  534: 7602963.5,
  535: 7158967.143,
  536: 7206408.0,
  537: 7028676.375,
  538: 6522175.143,
  539: 7098101.833,
  540: 6271358.429,
  541: 6316806.0,
  542: 7177994.571,
  543: 6185187.667,
  544: 7169104.571,
  545: 7111496.0,
  546: 7016835.0,
  547: 7038684.143,
  548: 6782601.429,
  549: 7007869.429,
  550: 6339382.857,
  551: 7116153.571,
  552: 6259572.286,
  553: 6183660.5,
  554: 6987774.143,
  555: 7602975.0,
  556: 7158979.375,
  557: 7206415.143,
  558: 7028688.143,
  559: 7179988.0,
  560: 6522187.5,
  561: 7098113.333,
  562: 6271370.5,
  563: 6316816.857,
  564: 7178006.5,
  565: 6185198.0,
  566: 7169116.25,
  567: 7111507.875,
  568: 7016846.75,
  569: 7038696.25,
  570: 6782613.25,
  571: 7007881.25,
  572: 6339394.375,
  573: 7116164.0,
  574: 6259593.875,
  575: 6183684.143,
  576: 6987796.0,
  577: 7602999.0,
  578: 7159002.714,
  579: 7206440.25,
  580: 7028709.857,
  581: 7180003.25,
  582: 6522212.0,
  583: 7098136.0,
  584: 6271394.0,
  585: 6316837.75,
  586: 7178028.429,
  587: 6185222.2,
  588: 7169139.143,
  589: 7111531.143,
  590: 7016869.429,
  591: 7038718.714,
  592: 6782635.714,
  593: 7007904.0,
  594: 6339416.571,
  595: 7116184.857,
  596: 6259631.714,
  597: 6183727.375,
  598: 6987838.5,
  599: 7603040.0,
  600: 7159044.25,
  601: 7206483.143,
  602: 7028752.0,
  603: 7180043.571,
  604: 6522253.0,
  605: 7098177.0,
  606: 6271437.375,
  607: 6316875.571,
  608: 7178069.0,
  609: 6185262.429,
  610: 7169179.875,
  611: 7111571.5,
  612: 7016910.125,
  613: 7038759.625,
  614: 6782677.0,
  615: 7007946.25,
  616: 6339456.875,
  617: 7116221.0,
  618: 6259675.25,
  619: 6183776.714,
  620: 6987888.714,
  621: 7603084.714,
  622: 7159093.429,
  623: 7206528.286,
  624: 7028799.714,
  625: 7180091.25,
  626: 6522299.857,
  627: 7098224.75,
  628: 6271489.143,
  629: 6316919.25,
  630: 7178116.429,
  631: 6185316.2,
  632: 7169227.0,
  633: 7111619.143,
  634: 7016957.857,
  635: 7038807.857,
  636: 6782725.0,
  637: 7007995.0,
  638: 6339505.571,
  639: 7116263.714,
  640: 6259721.429,
  641: 6183829.625,
  642: 6987940.875,
  643: 7603135.375,
  644: 7159145.125,
  645: 7206580.375,
  646: 7028850.625,
  647: 7180141.0,
  648: 6522350.286,
  649: 7098273.857,
  650: 6271542.5,
  651: 6316965.857,
  652: 7178166.375,
  653: 6185357.571,
  654: 7169276.5,
  655: 7111669.75,
  656: 7017007.875,
  657: 7038858.625,
  658: 6782772.857,
  659: 7008042.714,
  660: 6339555.25,
  661: 7116308.0,
  662: 6259765.375,
  663: 6183878.714,
  664: 6987990.286,
  665: 7603183.857,
  666: 7159193.571,
  667: 7206629.857,
  668: 7028900.143,
  669: 7180189.125,
  670: 6522396.714,
  671: 7098320.625,
  672: 6271592.714,
  673: 6317010.625,
  674: 7178214.714,
  675: 6185406.0,
  676: 7169321.429,
  677: 7111715.286,
  678: 7017056.0,
  679: 7038906.0,
  680: 6782820.5,
  681: 7008090.75,
  682: 6339601.857,
  683: 7116348.714,
  684: 6259822.571,
  685: 6183937.5,
  686: 6988046.75,
  687: 7603241.0,
  688: 7159255.0,
  689: 7206687.125,
  690: 7028957.0,
  691: 7180247.571,
  692: 6522459.125,
  693: 7098376.714,
  694: 6271651.375,
  695: 6317065.571,
  696: 7178272.875,
  697: 6185465.0,
  698: 7169378.375,
  699: 7111768.286,
  700: 7017116.0,
  701: 7038965.0,
  702: 6782879.286,
  703: 7008150.714,
  704: 6339659.125,
  705: 7116396.571,
  706: 6259918.875,
  707: 6184025.0,
  708: 6988135.0,
  709: 7603324.571,
  710: 7159359.429,
  711: 7206773.857,
  712: 7029057.0,
  713: 7180336.125,
  714: 6522552.0,
  715: 7098463.375,
  716: 6271740.714,
  717: 6317150.625,
  718: 7178371.857,
  719: 6185555.143,
  720: 7169476.429,
  721: 7111864.25,
  722: 7017215.857,
  723: 7039070.0,
  724: 6782980.875,
  725: 7008261.5,
  726: 6339752.143,
  727: 7116486.125,
  728: 6260023.0,
  729: 6184154.0,
  730: 6988260.5,
  731: 7603441.5,
  732: 7159496.75,
  733: 7206892.125,
  734: 7029186.25,
  735: 7180463.571,
  736: 6522662.714,
  737: 7098589.0,
  738: 6271867.375,
  739: 6317256.0,
  740: 7178509.5,
  741: 6185657.6,
  742: 7169610.5,
  743: 7112001.857,
  744: 7017352.375,
  745: 7039205.375,
  746: 6783102.714,
  747: 7008385.429,
  748: 6339886.5,
  749: 7116608.286,
  750: 6260115.125,
  751: 6184262.714,
  752: 6988372.286,
  753: 7603544.0,
  754: 7159605.429,
  755: 7207000.0,
  756: 7029300.286,
  757: 7180566.625,
  758: 6522764.143,
  759: 7098691.5,
  760: 6271976.857,
  761: 6317350.875,
  762: 7178613.571,
  763: 6185764.5,
  764: 7169716.286,
  765: 7112105.25,
  766: 7017457.857,
  767: 7039306.429,
  768: 6783208.875,
  769: 7008489.25,
  770: 6339990.857,
  771: 7116704.875,
  772: 6260247.857,
  773: 6184449.125,
  774: 6988557.0,
  775: 7603719.5,
  776: 7159721.714,
  777: 7207175.0,
  778: 7029467.625,
  779: 7180721.571,
  780: 6522920.75,
  781: 7098860.0,
  782: 6272164.125,
  783: 6317502.143,
  784: 7178754.75,
  785: 6185910.286,
  786: 7169873.0,
  787: 7112262.286,
  788: 7017577.143,
  789: 7039431.125,
  790: 6783330.714,
  791: 7008614.429,
  792: 6340158.75,
  793: 7116804.286,
  794: 6260370.625,
  795: 6184591.571,
  796: 6988688.286,
  797: 7603863.286,
  798: 7159903.25,
  799: 7207320.0,
  800: 7029582.714,
  801: 7180883.143,
  802: 6523091.857,
  803: 7099005.429,
  804: 6272309.714,
  805: 6317633.5,
  806: 7178933.0,
  807: 6186090.4,
  808: 7170058.714,
  809: 7112439.5,
  810: 7017757.25,
  811: 7039615.714,
  812: 6783507.625,
  813: 7008787.375,
  814: 6340325.143,
  815: 7116862.0,
  816: 6260445.571,
  817: 6184674.125,
  818: 6988763.375,
  819: 7603940.75,
  820: 7160008.857,
  821: 7207398.0,
  822: 7029652.375,
  823: 7180974.125,
  824: 6523177.714,
  825: 7099092.25,
  826: 6272389.5,
  827: 6317706.143,
  828: 7179032.75,
  829: 6186196.75,
  830: 7170163.857,
  831: 7112541.286,
  832: 7017861.286,
  833: 7039739.143,
  834: 6783612.286,
  835: 7008895.143,
  836: 6340414.5,
  837: 7116862.0,
  838: 6260550.429,
  839: 6184811.571,
  840: 6988897.0,
  841: 7604065.429,
  842: 7160131.0,
  843: 7207525.857,
  844: 7029790.0,
  845: 7181097.714,
  846: 6523294.0,
  847: 7099220.286,
  848: 6272525.429,
  849: 6317809.857,
  850: 7179161.429,
  851: 6186305.571,
  852: 7170282.375,
  853: 7112670.0,
  854: 7017986.125,
  855: 7039850.375,
  856: 6783740.25,
  857: 7009010.875,
  858: 6340546.714,
  859: 7116898.5,
  860: 6260720.25,
  861: 6185035.714,
  862: 6989125.375,
  863: 7604287.0,
  864: 7160369.571,
  865: 7207732.625,
  866: 7030002.75,
  867: 7181333.0,
  868: 6523497.875,
  869: 7099449.125,
  870: 6272748.857,
  871: 6317976.25,
  872: 7179389.143,
  873: 6186536.857,
  874: 7170513.571,
  875: 7112906.714,
  876: 7018227.0,
  877: 7040075.429,
  878: 6783955.143,
  879: 7009223.714,
  880: 6340771.571,
  881: 7117121.429,
  882: 6260866.571,
  883: 6185187.625,
  884: 6989268.0,
  885: 7604431.143,
  886: 7160560.125,
  887: 7207879.286,
  888: 7030139.429,
  889: 7181507.571,
  890: 6523666.857,
  891: 7099602.286,
  892: 6272897.0,
  893: 6318114.143,
  894: 7179564.75,
  895: 6186695.25,
  896: 7170682.125,
  897: 7113065.0,
  898: 7018406.75,
  899: 7040265.375,
  900: 6784133.25,
  901: 7009424.0,
  902: 6340931.75,
  903: 7117309.0,
  904: 6260999.0,
  905: 6185368.143,
  906: 6989420.857,
  907: 7604594.125,
  908: 7160746.429,
  909: 7208044.0,
  910: 7030279.286,
  911: 7181706.0,
  912: 6523829.571,
  913: 7099791.625,
  914: 6273067.429,
  915: 6318257.625,
  916: 7179774.286,
  917: 6186877.0,
  918: 7170867.571,
  919: 7113259.0,
  920: 7018606.286,
  921: 7040438.857,
  922: 6784311.429,
  923: 7009584.286,
  924: 6341136.857,
  925: 7117502.714,
  926: 6261108.0,
  927: 6185496.75,
  928: 6989550.75,
  929: 7604712.571,
  930: 7160893.25,
  931: 7208168.143,
  932: 7030402.125,
  933: 7181832.429,
  934: 6523949.286,
  935: 7099916.714,
  936: 6273197.5,
  937: 6318367.0,
  938: 7179918.375,
  939: 6186996.429,
  940: 7171038.75,
  941: 7113409.25,
  942: 7018749.625,
  943: 7040611.375,
  944: 6784448.375,
  945: 7009733.25,
  946: 6341275.125,
  947: 7117656.25,
  948: 6261252.0,
  949: 6185659.571,
  950: 6989708.571,
  951: 7604853.714,
  952: 7161071.571,
  953: 7208315.0,
  954: 7030550.571,
  955: 7181996.25,
  956: 6524109.0,
  957: 7100082.375,
  958: 6273361.286,
  959: 6318514.5,
  960: 7180084.714,
  961: 6187135.833,
  962: 7171217.857,
  963: 7113582.286,
  964: 7018920.714,
  965: 7040805.571,
  966: 6784616.857,
  967: 7009910.571,
  968: 6341447.286,
  969: 7117840.286,
  970: 6261402.714,
  971: 6185839.125,
  972: 6989869.375,
  973: 7605024.875,
  974: 7161297.125,
  975: 7208481.0,
  976: 7030704.0,
  977: 7182192.857,
  978: 6524291.286,
  979: 7100283.286,
  980: 6273531.75,
  981: 6318668.286,
  982: 7180305.375,
  983: 6187332.833,
  984: 7171443.75,
  985: 7113793.5,
  986: 7019137.375,
  987: 7041052.0,
  988: 6784832.875,
  989: 7010169.25,
  990: 6341651.375,
  991: 7118073.25,
  992: 6261553.5,
  993: 6186022.571,
  994: 6990054.857,
  995: 7605202.0,
  996: 7161482.714,
  997: 7208660.286,
  998: 7030890.0,
  999: 7182360.875,
  ...}}
]

In [1]:
p1_def= pd.read_csv("def.csv")

NameError: name 'pd' is not defined

In [2]:
import pandas as pd 
import time
from datetime import datetime as dt
p1_def= pd.read_csv("def.csv")

In [3]:
p1_def.dropna(inplace = True)
p1_def_dict = p1_def.to_dict()
p1_def_dict

{'dda;plant': {0: 'DATE_TIME;15/05/2020 0:00',
  1: 'PLANT_ID;4135001',
  2: 'SOURCE_KEY;1BY6WEcLGh8j5v7',
  3: 'DC_POWER;0',
  4: 'AC_POWER;0',
  5: 'DAILY_YIELD;0',
  6: 'TOTAL_YIELD;6259559'}}

In [4]:
import pandas as pd 
import time
from datetime import datetime as dt
p1_CAT= pd.read_csv("CAT.csv")
p1_CAT.dropna(inplace = True)
p1_CAT_dict = p1_CAT.to_dict()
p1_CAT_dict

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 5


In [5]:
import pandas as pd 
import time
from datetime import datetime as dt
p1_CAT= pd.read_csv("CAT.csv")

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 5


In [6]:
#SENTENCIAS
# PRIMER REQUERIMIENTO

# TERCER  REQUERIMIENTO

print('obtener la potencia de la planta uno con la SOURCE_KEY es un horario determinado)
key = 'SOURCE_KEY#adLQvlD726eNBSB'.encode()

col_filter = row_filters.ColumnQualifierRegexFilter('SOURCE_KEY#adLQvlD726eNBSB'.encode())

row = table.read_row(key, filter_ = col_filter)
print_row(row)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 3, saw 5


In [ ]:
#2 requerimiento
[
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,34702E+16",
        "MODULE_TEMPERATURE": "2,28296E+16",
        "IRRADIATION": "6,39194E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,36733E+16",
        "MODULE_TEMPERATURE": "2,33385E+16",
        "IRRADIATION": "2,09014E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,39792E+16",
        "MODULE_TEMPERATURE": "2,38863E+16",
        "IRRADIATION": "4,68731E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,42097E+16",
        "MODULE_TEMPERATURE": "2,5162E+16",
        "IRRADIATION": "6,97067E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,45673E+16",
        "MODULE_TEMPERATURE": "2,65871E+16",
        "IRRADIATION": "6,46242E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,48356E+16",
        "MODULE_TEMPERATURE": "2,7842E+16",
        "IRRADIATION": "1504371014"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,53616E+16",
        "MODULE_TEMPERATURE": "3,02081E+16",
        "IRRADIATION": "1,6287E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "255112201",
        "MODULE_TEMPERATURE": "2,96953E+16",
        "IRRADIATION": "1,3207E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,56891E+16",
        "MODULE_TEMPERATURE": "2,96067E+16",
        "IRRADIATION": "161106645"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,58384E+16",
        "MODULE_TEMPERATURE": "2,96984E+16",
        "IRRADIATION": "2,17574E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,6106E+15",
        "MODULE_TEMPERATURE": "3,0992E+16",
        "IRRADIATION": "2,81624E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,67854E+16",
        "MODULE_TEMPERATURE": "3,53831E+16",
        "IRRADIATION": "5,47219E+14"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,75159E+16",
        "MODULE_TEMPERATURE": "3,97507E+15",
        "IRRADIATION": "6,60321E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,82458E+16",
        "MODULE_TEMPERATURE": "4,18187E+16",
        "IRRADIATION": "6,98736E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2905904",
        "MODULE_TEMPERATURE": "4,63607E+16",
        "IRRADIATION": "8,13793E+15"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,93947E+16",
        "MODULE_TEMPERATURE": "4,93506E+16",
        "IRRADIATION": "7,88344E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,93017E+16",
        "MODULE_TEMPERATURE": "4,75594E+16",
        "IRRADIATION": "6,25476E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "2,98226E+16",
        "MODULE_TEMPERATURE": "504896434",
        "IRRADIATION": "9,18448E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,01605E+16",
        "MODULE_TEMPERATURE": "4,97954E+15",
        "IRRADIATION": "6,88386E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,00995E+16",
        "MODULE_TEMPERATURE": "439329496",
        "IRRADIATION": "3,2E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,01654E+16",
        "MODULE_TEMPERATURE": "4,12362E+16",
        "IRRADIATION": "3,16755E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,05977E+16",
        "MODULE_TEMPERATURE": "4,32035E+16",
        "IRRADIATION": "4,54342E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "309631358",
        "MODULE_TEMPERATURE": "4,31681E+16",
        "IRRADIATION": "3,52625E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,07022E+16",
        "MODULE_TEMPERATURE": "3,96788E+16",
        "IRRADIATION": "4,8591E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,20973E+16",
        "MODULE_TEMPERATURE": "5,34779E+16",
        "IRRADIATION": "1,07978E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,23246E+15",
        "MODULE_TEMPERATURE": "5,60867E+16",
        "IRRADIATION": "7,52138E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,27136E+16",
        "MODULE_TEMPERATURE": "4,93169E+16",
        "IRRADIATION": "9,17015E+16"
    },
    {
        "DATE_TIME": "lunes, 1 de junio de 2020",
        "PLANT_ID": "4136001",
        "SOURCE_KEY": "iq8k7ZNt4Mwm3w0",
        "AMBIENT_TEMPERATURE": "3,29141E+16",
        "MODULE_TEMPERATURE": "4,85172E+16",
        "IRRADIATION": "5,20088E+16"
    }
]

In [ ]:
# TERCER_REQUERIMIENTO
[
    {
        "DATE_TIME": "15/05/2020 0:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 0:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 0:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 0:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 1:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 1:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 1:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 1:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 2:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 2:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 2:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 2:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 3:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 3:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 3:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 3:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 4:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 4:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 4:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 4:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 5:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 5:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 5:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 5:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "0",
        "AC_POWER": "0",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 6:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "56125",
        "AC_POWER": "5425",
        "DAILY_YIELD": "0",
        "TOTAL_YIELD": "6271355"
    },
    {
        "DATE_TIME": "15/05/2020 6:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "3151428571",
        "AC_POWER": "3044285714",
        "DAILY_YIELD": "3428571429",
        "TOTAL_YIELD": "6271358429"
    },
    {
        "DATE_TIME": "15/05/2020 6:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "71925",
        "AC_POWER": "697375",
        "DAILY_YIELD": "155",
        "TOTAL_YIELD": "62713705"
    },
    {
        "DATE_TIME": "15/05/2020 6:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "1348142857",
        "AC_POWER": "1315",
        "DAILY_YIELD": "39",
        "TOTAL_YIELD": "6271394"
    },
    {
        "DATE_TIME": "15/05/2020 7:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "2009625",
        "AC_POWER": "197",
        "DAILY_YIELD": "82375",
        "TOTAL_YIELD": "6271437375"
    },
    {
        "DATE_TIME": "15/05/2020 7:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "2143714286",
        "AC_POWER": "2101714286",
        "DAILY_YIELD": "1341428571",
        "TOTAL_YIELD": "6271489143"
    },
    {
        "DATE_TIME": "15/05/2020 7:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "2109875",
        "AC_POWER": "20685",
        "DAILY_YIELD": "1875",
        "TOTAL_YIELD": "62715425"
    },
    {
        "DATE_TIME": "15/05/2020 7:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "2176857143",
        "AC_POWER": "2134285714",
        "DAILY_YIELD": "2377142857",
        "TOTAL_YIELD": "6271592714"
    },
    {
        "DATE_TIME": "15/05/2020 8:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "281625",
        "AC_POWER": "2764625",
        "DAILY_YIELD": "296375",
        "TOTAL_YIELD": "6271651375"
    },
    {
        "DATE_TIME": "15/05/2020 8:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "4617142857",
        "AC_POWER": "4531",
        "DAILY_YIELD": "3857142857",
        "TOTAL_YIELD": "6271740714"
    },
    {
        "DATE_TIME": "15/05/2020 8:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "4948375",
        "AC_POWER": "48535",
        "DAILY_YIELD": "512375",
        "TOTAL_YIELD": "6271867375"
    },
    {
        "DATE_TIME": "15/05/2020 8:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "5279428571",
        "AC_POWER": "5177142857",
        "DAILY_YIELD": "6218571429",
        "TOTAL_YIELD": "6271976857"
    },
    {
        "DATE_TIME": "15/05/2020 9:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "8274875",
        "AC_POWER": "8098",
        "DAILY_YIELD": "809125",
        "TOTAL_YIELD": "6272164125"
    },
    {
        "DATE_TIME": "15/05/2020 9:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "3395714286",
        "AC_POWER": "3336428571",
        "DAILY_YIELD": "9547142857",
        "TOTAL_YIELD": "6272309714"
    },
    {
        "DATE_TIME": "15/05/2020 9:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "3996875",
        "AC_POWER": "3919375",
        "DAILY_YIELD": "10345",
        "TOTAL_YIELD": "62723895"
    },
    {
        "DATE_TIME": "15/05/2020 9:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "8003857143",
        "AC_POWER": "7828",
        "DAILY_YIELD": "1170428571",
        "TOTAL_YIELD": "6272525429"
    },
    {
        "DATE_TIME": "15/05/2020 10:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "8481142857",
        "AC_POWER": "8289428571",
        "DAILY_YIELD": "1393857143",
        "TOTAL_YIELD": "6272748857"
    },
    {
        "DATE_TIME": "15/05/2020 10:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "6276875",
        "AC_POWER": "61505",
        "DAILY_YIELD": "1542",
        "TOTAL_YIELD": "6272897"
    },
    {
        "DATE_TIME": "15/05/2020 10:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "5257571429",
        "AC_POWER": "5155714286",
        "DAILY_YIELD": "1712428571",
        "TOTAL_YIELD": "6273067429"
    },
    {
        "DATE_TIME": "15/05/2020 10:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "4962",
        "AC_POWER": "48655",
        "DAILY_YIELD": "18425",
        "TOTAL_YIELD": "62731975"
    },
    {
        "DATE_TIME": "15/05/2020 11:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "7679",
        "AC_POWER": "7511714286",
        "DAILY_YIELD": "2006285714",
        "TOTAL_YIELD": "6273361286"
    },
    {
        "DATE_TIME": "15/05/2020 11:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "5966875",
        "AC_POWER": "58465",
        "DAILY_YIELD": "217675",
        "TOTAL_YIELD": "627353175"
    },
    {
        "DATE_TIME": "15/05/2020 11:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "8083857143",
        "AC_POWER": "7902",
        "DAILY_YIELD": "2365142857",
        "TOTAL_YIELD": "6273720143"
    },
    {
        "DATE_TIME": "15/05/2020 11:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "8465375",
        "AC_POWER": "827575",
        "DAILY_YIELD": "253725",
        "TOTAL_YIELD": "627389225"
    },
    {
        "DATE_TIME": "15/05/2020 12:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "6857857143",
        "AC_POWER": "6710285714",
        "DAILY_YIELD": "2746142857",
        "TOTAL_YIELD": "6274101143"
    },
    {
        "DATE_TIME": "15/05/2020 12:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "902275",
        "AC_POWER": "8819375",
        "DAILY_YIELD": "2949",
        "TOTAL_YIELD": "6274304"
    },
    {
        "DATE_TIME": "15/05/2020 12:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "1009171429",
        "AC_POWER": "9852142857",
        "DAILY_YIELD": "3178142857",
        "TOTAL_YIELD": "6274533143"
    },
    {
        "DATE_TIME": "15/05/2020 12:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "92495",
        "AC_POWER": "9043125",
        "DAILY_YIELD": "3411625",
        "TOTAL_YIELD": "6274766625"
    },
    {
        "DATE_TIME": "15/05/2020 13:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "6446285714",
        "AC_POWER": "6314142857",
        "DAILY_YIELD": "3590857143",
        "TOTAL_YIELD": "6274945857"
    },
    {
        "DATE_TIME": "15/05/2020 13:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "7089571429",
        "AC_POWER": "6942857143",
        "DAILY_YIELD": "3753142857",
        "TOTAL_YIELD": "6275108143"
    },
    {
        "DATE_TIME": "15/05/2020 13:30",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "8289125",
        "AC_POWER": "8112375",
        "DAILY_YIELD": "3940625",
        "TOTAL_YIELD": "6275295625"
    },
    {
        "DATE_TIME": "15/05/2020 13:45",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "7920571429",
        "AC_POWER": "7752857143",
        "DAILY_YIELD": "4136857143",
        "TOTAL_YIELD": "6275491857"
    },
    {
        "DATE_TIME": "15/05/2020 14:00",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "10456125",
        "AC_POWER": "10220875",
        "DAILY_YIELD": "435675",
        "TOTAL_YIELD": "627571175"
    },
    {
        "DATE_TIME": "15/05/2020 14:15",
        "PLANT_ID": "4135001",
        "SOURCE_KEY": "adLQvlD726eNBSB",
        "DC_POWER": "1231914286",
        "AC_POWER": "1201442857",
        "DAILY_YIELD": "4633142857",
        "TOTAL_YIELD": "6275988143"
    }
]